In [1]:
%env CUDA_VISIBLE_DEVICES=0

device='cuda'

env: CUDA_VISIBLE_DEVICES=0


In [2]:
import os
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn import preprocessing, metrics
from sklearn.ensemble import RandomForestClassifier
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from copy import deepcopy
from tqdm import tqdm

from mtcnn import MTCNN

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential, load_model, model_from_json
from tensorflow.keras.optimizers import Adam, SGD

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from torchvision import datasets, transforms
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, classification_report

print(f"Torch: {torch.__version__}")

2024-08-05 09:22:03.732363: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-05 09:22:03.770302: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-05 09:22:03.770328: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-05 09:22:03.770367: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-05 09:22:03.777585: I tensorflow/core/platform/cpu_feature_g

Torch: 2.1.0+cu121


In [3]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Dropout, Dense, Activation, Concatenate, Reshape
from tensorflow.keras.layers import Flatten, RepeatVector, Permute, TimeDistributed
from tensorflow.keras.layers import Multiply, Lambda, Softmax

In [4]:
DATA_DIR = '/media/deelvin_disk/echuraev/Workspace/HSE/engagement/'
BATCH_SIZE = 128
TABLE_NAME = None

# Helpers

In [5]:
import pickle

def save_features(file_name, features):
    if os.path.isfile(file_name):
        print("Error! Cannot save features because file already exists")
        return
    with open(file_name, 'wb') as f:
        pickle.dump(features, f) # , protocol=pickle.HIGHEST_PROTOCOL)
    
def load_features(file_name):
    with open(file_name, 'rb') as f:
        return pickle.load(f)

def save_weights(model, file_name):
    if os.path.isfile(file_name):
        print("Error! Cannot save features because file already exists")
        return
    model.save_weights(file_name)

def load_weights(model, file_name):
    model.load_weights(file_name)

In [6]:
def draw_confusion_matrix(all_preds, all_labels):
    from sklearn.metrics import confusion_matrix
    import pandas as pd
    import seaborn as sn

    preds = []
    labels = []
    for p in all_preds:
        preds.append(float2int[p])
    for l in all_labels:
        labels.append(float2int[l])

    labels = np.array(labels)
    preds = np.array(preds)
    data=confusion_matrix(labels, preds)

    df_cm = pd.DataFrame(data, columns=labels_list)
    df_cm.index.name = 'Actual'
    df_cm.columns.name = 'Predicted'
    plt.figure(figsize = (13,10))
    sn.set(font_scale=3)#for label size
    sn.heatmap(df_cm, cmap="Blues", annot=True,annot_kws={"size": 16*2.5}, fmt='g')# font size
    plt.yticks(np.arange(len(labels_list))+0.5, labels_list, rotation=0, va="center")

In [7]:
def transform_sequences2norm_features(features):
    ret = []
    for i in range(len(features)):
        ret.append(stat_func(features[i], axis=0))
    return ret

In [8]:
def pred2label(pred):
    if pred < 0.25:
        return 0.0
    if pred < 0.5:
        return 0.33
    if pred < 0.75:
        return 0.66
    return 1.0

def get_prediction(model, file2feat, with_processing=False):
    y_pred = []
    exp_pred = []
    for fn, x in file2feat.items():
        #print(x.shape)
        if len(x)==0:
            continue
        if CONCATENATE_STAT:
            mean_x=np.repeat(stat_func(x,axis=0).reshape((1,-1)),len(x),axis=0)            
            #preds=model.predict(np.expand_dims(np.concatenate((mean_x,x-mean_x),axis=1), axis=0))[0]
            pred=model.predict(np.expand_dims(np.concatenate((mean_x,x),axis=1), axis=0), verbose=0)
        else:
            pred=model.predict(np.expand_dims(x, axis=0), verbose=0)
        #y_pred.append(np.argmax(pred))
        #if with_processing:
        #    y_pred.append(pred2label(pred[0]))
        #else:
        #    y_pred.append(float(pred[0]))
        y_pred.append(float(pred[0]))
        exp_pred.append(video2label[fn])
    return np.array(y_pred), np.array(exp_pred)

def prediction2bin(preds, threshold):
    y_pred = []
    for y in preds:
        if N_CLASSES == 2:
            y = 0 if y < threshold else 1
        y_pred.append(y)
    return y_pred

def dump_to_table(table_name, metric_name, acc, mse, uar, recall, precision, f1, num_comma=","):
    def _to_str(num, num_comma):
        if num_comma == ",":
            return str(num).replace(".", ",")
        return str(num).replace(",", ".")
    
    import pandas as pd
    if os.path.isfile(table_name):
        df = pd.read_excel(open(table_name,'rb'))
    else:
        df = pd.DataFrame(columns=['metric_name', 'accuracy', 'mse', 'uar', 'recall', 'precision', 'f1'])

    idx = len(df)
    # Remove previous recordings
    df = df[df.metric_name != metric_name]
    df.loc[idx] = [
        metric_name,
        _to_str(acc, num_comma),
        _to_str(mse, num_comma),
        _to_str(uar, num_comma),
        _to_str(recall, num_comma),
        _to_str(precision, num_comma), 
        _to_str(f1, num_comma)
    ] 
    df.to_excel(table_name, index=False) 
    

def print_results(pred, labels, table_name=None, metric_name=None, from_int=False):
    if from_int is False:
        float2int={0:0,0.33:1,0.66:2,1:3}
        if N_CLASSES == 2:
            float2int={0:1,0.33:1,0.66:0,1:0}
        pred_int = np.array([float2int[pred2label(l)] for l in pred])
        labels_int = np.array([float2int[l] for l in labels])
        #labels_float = np.array([int2float[l] for l in labels])
        #pred_float = np.array([int2float[l] for l in pred])
        acc = (labels_int==pred_int).mean()
        #if N_CLASSES == 2:
        #    mse = ((labels-pred)**2).mean()
        #else:
        #    mse = ((labels_float-pred_float)**2).mean()
        mse = ((labels-pred)**2).mean()
        uar = recall_score(y_true=labels_int,y_pred=pred_int, average='macro')
        if N_CLASSES == 2:
            recall = recall_score(y_true=labels,y_pred=pred, average='binary',pos_label=1)
            precision = precision_score(y_true=labels,y_pred=pred, average='binary',pos_label=1)
            f1 = f1_score(y_true=labels,y_pred=pred, average='binary',pos_label=1)
        else:
            recall = 'N/A'
            precision = 'N/A'
            f1 = f1_score(y_true=labels_int,y_pred=pred_int, average='macro')
    else:
        int2float = {0:0.0, 1: 0.33, 2:0.66, 3: 1.0}
        labels_float = np.array([int2float[l] for l in labels])
        pred_float = np.array([int2float[l] for l in pred])
        acc = (labels==pred).mean()
        if N_CLASSES == 2:
            mse = ((labels-pred)**2).mean()
        else:
            mse = ((labels_float-pred_float)**2).mean()
        uar = recall_score(y_true=labels,y_pred=pred, average='macro')
        if N_CLASSES == 2:
            recall = recall_score(y_true=labels,y_pred=pred, average='binary',pos_label=1)
            precision = precision_score(y_true=labels,y_pred=pred, average='binary',pos_label=1)
            f1 = f1_score(y_true=labels,y_pred=pred, average='binary',pos_label=1)
        else:
            recall = 'N/A'
            precision = 'N/A'
            f1 = f1_score(y_true=labels,y_pred=pred, average='macro')
    print('Metric_name: ', metric_name,
          'Accuracy: ', acc,
          'MSE: ', mse,
          'UAR: ', uar,
          'Recall: ', recall,
          'Precision: ', precision,
          'F1: ', f1)
    if table_name is not None and metric_name is not None:
        dump_to_table(table_name, metric_name, acc, mse, uar, recall, precision, f1)
    return acc, mse, uar, recall, precision, f1

def print_mse(preds, labels):
    diff_correct,num_total=0.0,0
    for i in range(len(preds)):
        diff_correct+=(preds[i]-labels[i])**2
        num_total+=1
    print(num_total,diff_correct/num_total)

In [9]:
def define_dataset(engage_wild_dataset, bin_classification):
    ENGAGE_WILD_DATASET = engage_wild_dataset
    BIN_CLASSIFICATION = bin_classification
    if BIN_CLASSIFICATION is True:
        N_CLASSES = 2
    else:
        N_CLASSES = 4
    if ENGAGE_WILD_DATASET is True:
        ext = "png"
        DATASET_NAME = 'EngageWild'
        BASE_DATASET_DIR = '/home/HDD6TB/datasets/emotions/EmotiW/engagement/'
        BASE_DATASET_DIR = '/media/deelvin_disk/echuraev/Workspace/HSE/datasets/EngageWild'
    else:
        ext = "jpg"
        DATASET_NAME = 'DAiSEE'
        BASE_DATASET_DIR = '/home/HDD6TB/datasets/emotions/DAiSEE/'
        BASE_DATASET_DIR = '/media/deelvin_disk/echuraev/Workspace/HSE/datasets/DAiSEE'

    return ENGAGE_WILD_DATASET, BIN_CLASSIFICATION, N_CLASSES, DATASET_NAME, BASE_DATASET_DIR, ext

In [177]:
def get_video2label():
    labels_list = ['very distracted', 'distracted', 'engaged', 'very engaged']
    float2int={0:0,0.33:1,0.66:2,1:3}
    if ENGAGE_WILD_DATASET is True:
        import csv
        video2label={}
        with open(os.path.join(BASE_DATASET_DIR,'Engagement_Labels_Engagement.csv'), mode='r') as csvfile:
            labels_reader = csv.reader(csvfile, delimiter='\t')
            for i,row in enumerate(labels_reader):
                if i==0:
                    continue
                #videoname,label=row[0],float2int[float(row[1])]
                videoname,label=row[0],float(row[1])
                video2label[videoname]=label
                #print(videoname,label)
                #if (videoname not in filename2features_val) and (videoname not in filename2features_train):
                #    print(videoname,label)
        #check if fix is incorrect
        video2label['subject_87_Vid_3']=video2label['subject_77_Vid_6']
        #video2label = to_categorical(video2label)
        #print(len(video2label))
        #print(video2label)
    else:
        import pandas as pd
        df=pd.read_csv(os.path.join(BASE_DATASET_DIR,'Labels/AllLabels.csv'))
        df.columns = df.columns.str.replace(' ', '')
        df.head()
        labels2fileAndValues=df.set_index('ClipID').to_dict()
        video2label={os.path.splitext(video)[0]:val for video,val in labels2fileAndValues['Engagement'].items()}
        int2float = {0:0.0, 1: 0.33, 2:0.66, 3: 1.0}
        for k, v in video2label.items():
            video2label[k] = int2float[video2label[k]]
        #print(video2label)
    #print(video2label)
    if N_CLASSES == 2:
        labels_list = ['engaged', 'distracted']
        bin_labels = {0:1, 1:1, 2:0, 3:0}
        for k in video2label.keys():
            video2label[k] = bin_labels[float2int[video2label[k]]]
    return video2label, labels_list

In [11]:
def get_weights_path(metric_name):
    return WEIGHTS_DIR + "{}.h5".format(metric_name)
    
def get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, aggregator, classificator):
    return "{}_{}_{}_{}_{}".format(base_model_key, DATASET_NAME, N_CLASSES, aggregator, classificator)

In [12]:
def print_classes(labels):
    min_num = len(labels)
    for i in np.unique(labels):
        count = labels.count(i)
        if count < min_num:
            min_num = count
        percent = count * 100 / len(labels)
        print("{} {}/{}: {}%".format(i, count, len(labels), percent))

def get_labels(fn2feat):
    labels = []
    for fn in fn2feat.keys():
        labels.append(video2label[fn])
    return labels

## Dataset balancing

In [13]:
def create_dataset(filename2features):
    x = []
    y = []
    ind=0
    for fn in filename2features:
        features=filename2features[fn]
        total_features=None
        if USE_ALL_FEATURES:
            #prev=features[0].shape
            cur_features=features[0][features[-1]==1]
            #if filename2features_val==filename2features:
            #    print(cur_features.shape)
            #    cur_features=cur_features[2:]
            #print(prev,features.shape)
        else:
            cur_features=features
        
        total_features=stat_func(cur_features)
        if total_features is not None:
            x.append(total_features)
            y.append(video2label[fn])
    x=np.array(x)
    y=np.array(y)
    print(x.shape,y.shape)
    return x,y

In [14]:
import random
random.seed(7)

def get_train_test_indices(x_all, y_all, SPLIT_RATIO):
    inds_train, inds_test = [], []
    inds = np.arange(len(y_all))
    
    for lbl in np.unique(y_all):
        tmp_inds = inds[y_all == lbl]
        random.shuffle(tmp_inds)
        
        ind=int(len(tmp_inds) * SPLIT_RATIO)
        inds_train.append(tmp_inds[:ind])
        inds_test.append(tmp_inds[ind:])
    
    inds_train = np.concatenate(inds_train, axis=0)
    inds_test = np.concatenate(inds_test, axis=0)
    return inds_train, inds_test

def get_train_test():
    inds_train, inds_test = get_train_test_indices()
    X_train=x_all[inds_train]
    y_train=y_all[inds_train]
    X_test=x_all[inds_test]
    y_test=y_all[inds_test]
    return X_train,y_train,X_test,y_test

In [15]:
def prepare_new_dataset(filename2features_train, filename2features_test, SPLIT_RATIO): # =0.767):
    x_train, y_train = create_dataset(filename2features_train)
    x_test, y_test = create_dataset(filename2features_test)

    x_all=np.concatenate((x_train,x_test),axis=0)
    y_all=np.concatenate((y_train,y_test),axis=0)
    print(x_all.shape,y_all.shape)
    
    labels,counts=np.unique(y_all,return_counts=True)
    print(labels,counts)

    filenames_all=list(filename2features_train.keys())+list(filename2features_test.keys())
    print(len(filenames_all))
    
    random.seed(7)
    inds_train, inds_val = get_train_test_indices(x_all, y_all, SPLIT_RATIO)
    
    def get_filename2features_new(inds):
        filename2features_new={}
        for ind in inds:
            filename=filenames_all[ind]
            if filename in filename2features_train:
                filename2features_new[filename]=filename2features_train[filename]
            else:
                filename2features_new[filename]=filename2features_test[filename]
        print(len(filename2features_new))
        return filename2features_new
    filename2features_train_new=get_filename2features_new(inds_train)
    filename2features_val_new=get_filename2features_new(inds_val)

    return filename2features_train_new, filename2features_val_new

In [16]:
def compute_descriptor(cur_features, axis=0):
    #cur_features=cur_features[1000:]
    #cur_features=cur_features[30:-30]
    #mean_features=features.mean(axis=0)
    mean_features = np.mean(cur_features, axis=0)
    std_features = np.std(cur_features, axis=0)
    max_features = np.max(cur_features, axis=0)
    min_features = np.min(cur_features, axis=0)

    # join several features together
    #feature = np.concatenate((mean_features, std_features, min_features, max_features), axis=None)                    
    #feature = np.concatenate((mean_features, std_features, max_features), axis=None)
    #feature = np.concatenate((mean_features, min_features, max_features), axis=None)
    feature = np.concatenate((mean_features, std_features), axis=None)
    #feature = np.concatenate((max_features, std_features), axis=None)

    #feature=std_features
    #feature=mean_features
    #feature=np.percentile(cur_features, 100,axis=0)
    return feature

## Attention specific

In [17]:
IMAGE_SET_SIZE=128 #32 #20
USE_GENERATORS = False
USE_ALL_FEATURES = False

In [18]:
def get_samples(features_list, labels_list):
    X_subsample,y_subsample=[],[]
    for i in range(len(features_list)):
        features=features_list[i]
        label = labels_list[i]
        total_features=None
        if USE_ALL_FEATURES:
            x=features[0][features[-1]==1]
        else:
            x=features
        max_ind=len(x)-IMAGE_SET_SIZE
        if max_ind<=0:
            continue
        stat_x=stat_func(x,axis=0)
        num_samples=max(max_ind//(IMAGE_SET_SIZE),1)
        for frame_ind in random.sample(range(max_ind),k=num_samples):
            if CONCATENATE_STAT:
                X_current=[np.concatenate((stat_x,x[frame_ind+i])) for i in range(IMAGE_SET_SIZE)]
            else:
                X_current=[x[frame_ind+i] for i in range(IMAGE_SET_SIZE)]
            X_subsample.append(X_current)
            y_subsample.append(label)
    
 
    X_subsample = np.array(X_subsample)
    y_subsample=np.array(y_subsample)
    print(X_subsample.shape,y_subsample.shape)
    return X_subsample,y_subsample

def get_samples(filename2features):
    X_subsample,y_subsample=[],[]
    for fn in filename2features:
        features=filename2features[fn]
        total_features=None
        if USE_ALL_FEATURES:
            x=features[0][features[-1]==1]
        else:
            x=features
        max_ind=len(x)-IMAGE_SET_SIZE
        if max_ind<=0:
            continue
        stat_x=stat_func(x,axis=0)
        num_samples=max(max_ind//(IMAGE_SET_SIZE),1)
        for frame_ind in random.sample(range(max_ind),k=num_samples):
            if CONCATENATE_STAT:
                X_current=[np.concatenate((stat_x,x[frame_ind+i])) for i in range(IMAGE_SET_SIZE)]
            else:
                X_current=[x[frame_ind+i] for i in range(IMAGE_SET_SIZE)]
            X_subsample.append(X_current)
            y_subsample.append(video2label[fn])
    
 
    X_subsample = np.array(X_subsample)
    y_subsample=np.array(y_subsample)
    print(X_subsample.shape,y_subsample.shape)
    return X_subsample,y_subsample

In [19]:
def generate_batch(features, labels):
    X_subsample = []
    Y_subsample = []
    while True:
        idx = random.randrange(len(labels))
        x = features[idx]
        img_count=len(x)
        num_per_part = img_count // IMAGE_SET_SIZE

        if img_count < IMAGE_SET_SIZE:
            continue

        mean_x=stat_func(x,axis=0)
        for j in range(img_count):
            if CONCATENATE_STAT:
                #X_current=[np.concatenate((mean_x,x[frame_ind+i]-mean_x)) for i in range(IMAGE_SET_SIZE)]
                X_current=[np.concatenate((mean_x,x[random.randint(num_per_part*j, min(num_per_part*(j+1),img_count-1))])) for j in range(IMAGE_SET_SIZE)]
            else:
                X_current=[x[random.randint(num_per_part*j, min(num_per_part*(j+1),img_count-1))] for j in range(IMAGE_SET_SIZE)]
            X_subsample.append(X_current)
            Y_subsample.append(labels[idx])
            if len(Y_subsample) >= BATCH_SIZE:
                X_subsample = np.array(X_subsample)
                Y_subsample = np.array(Y_subsample)
                yield X_subsample, Y_subsample
                X_subsample = []
                Y_subsample = []


def get_num_samples(features):
    num_samples = 0
    for idx in range(len(features)):
        x = features[idx]
        img_count=len(x)
        num_per_part = img_count // IMAGE_SET_SIZE

        if img_count < IMAGE_SET_SIZE:
            continue

        num_samples += img_count

    if num_samples % BATCH_SIZE > 0:
        num_samples += BATCH_SIZE

    if CONCATENATE_STAT:
        mean_x=stat_func(x,axis=0)
        #X_current=[np.concatenate((mean_x,x[frame_ind+i]-mean_x)) for i in range(IMAGE_SET_SIZE)]
        vector_dim = len(np.concatenate((mean_x,x[random.randint(0, min(num_per_part,img_count-1))])))
    else:
        vector_dim = len(x[random.randint(0, min(num_per_part,img_count-1))])
    return num_samples, vector_dim

In [20]:
from tensorflow.keras.models import load_model,Model
class SaveBestModel(tf.keras.callbacks.Callback):
    def __init__(self, save_best_metric='val_loss', this_max=False):
        self.save_best_metric = save_best_metric
        self.max = this_max
        if this_max:
            self.best = float('-inf')
        else:
            self.best = float('inf')

    def on_epoch_end(self, epoch, logs=None):
        metric_value = logs[self.save_best_metric]
        if self.max:
            if metric_value > self.best:
                self.best = metric_value
                self.best_model_weights = self.model.get_weights()

        else:
            if metric_value < self.best:
                self.best = metric_value
                self.best_model_weights = self.model.get_weights()

In [21]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [22]:
def get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES):
    inputs = Input(shape=(None, FEATURE_VECTOR_DIM),name='image_set')  # (batch, samples, features)
    e = Dense(1, activation='linear', name='e')(inputs)
    e = Reshape([-1], name='alignment')(e)
    alpha = Activation('softmax', name='alpha')(e)
    
    alpha_repeated = Permute([2, 1],name='alpha_repeated')(RepeatVector(FEATURE_VECTOR_DIM, name='repeat')(alpha))
    
    c = Multiply(name='c')([inputs, alpha_repeated])
    x = Lambda(lambda xin: K.sum(xin, axis=1), output_shape=(FEATURE_VECTOR_DIM,), name='context')(c)
    
    #x = Dropout(0.3)(x)
    x = Dense(units=512, activation='relu', name='hidden_FC')(x)  # (batch, units) #128 64
    #x = tf.keras.activations.gelu(Dense(512, activation='linear')(x))

    #pred=Dense(N_CLASSES,activation='softmax')(x)
    pred=Dense(1,activation='sigmoid')(x)
    modelAtn=Model(inputs=inputs,outputs=pred)
    #modelAtn.compile(optimizer=Adam(lr=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    if N_CLASSES == 2:
        #modelAtn.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=['acc', f1_m,precision_m, recall_m])
        metrics=['acc',tf.keras.metrics.AUC(multi_label=True,name='auc'), tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.Recall(),tf.keras.metrics.Precision()]
        modelAtn.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=metrics)
    else:
        modelAtn.compile(optimizer=Adam(lr=1e-4), loss='mse', metrics=['mae'])
    modelAtn.summary()

    #save_best_model = SaveBestModel('val_accuracy',True)
    save_best_model = SaveBestModel('val_loss',False)
    return modelAtn, save_best_model

In [23]:
def get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES):
    embeddings_dim=512
    embeddings_dim=FEATURE_VECTOR_DIM
    inputs = Input(shape=(None, FEATURE_VECTOR_DIM),name='image_set')  # (batch, samples, features)
    if False:
        query_seq_encoding, value_seq_encoding=inputs,inputs
    else:
        query_seq_encoding=Dense(embeddings_dim, activation='linear',use_bias=False, name='query')(inputs)
        value_seq_encoding=Dense(embeddings_dim, activation='linear',use_bias=False, name='value')(inputs)
    query_value_attention_seq = tf.keras.layers.Attention()([query_seq_encoding, value_seq_encoding])
    
    e = Dense(1, activation='linear', name='e')(query_value_attention_seq)
    e = Reshape([-1], name='alignment')(e)
    alpha = Activation('softmax', name='alpha')(e)
    
    alpha_repeated = Permute([2, 1],name='alpha_repeated')(RepeatVector(embeddings_dim, name='repeat')(alpha))
    
    c = Multiply(name='c')([query_value_attention_seq, alpha_repeated])
    x = Lambda(lambda xin: K.sum(xin, axis=1), output_shape=(embeddings_dim,), name='context')(c)

    #pred=Dense(N_CLASSES,activation='softmax')(x)
    pred=Dense(1,activation='sigmoid')(x)
    modelAtn=Model(inputs=inputs,outputs=pred)
    #modelAtn.compile(optimizer=Adam(lr=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    if N_CLASSES == 2:
        #modelAtn.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=['acc', f1_m,precision_m, recall_m])
        metrics=['acc',tf.keras.metrics.AUC(multi_label=True,name='auc'), tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.Recall(),tf.keras.metrics.Precision()]
        modelAtn.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=metrics)
    else:
        modelAtn.compile(optimizer=Adam(lr=1e-4), loss='mse', metrics=['mae'])
    modelAtn.summary()

    #save_best_model = SaveBestModel('val_accuracy',True)
    save_best_model = SaveBestModel('val_loss',False)
    return modelAtn, save_best_model

# EngageWild dataset (4 classes)

In [24]:
FEATURES_DIR = DATA_DIR + 'features_EngageWild/'
WEIGHTS_DIR = DATA_DIR + 'weights_EngageWild/'
TABLE_NAME = '02_EngageWild_4_classes.xlsx'

## enet_b0_8_best_afew

In [25]:
base_model_key = 'enet_b0_8_best_afew.pt'

In [26]:
ENGAGE_WILD_DATASET, BIN_CLASSIFICATION, N_CLASSES, DATASET_NAME, BASE_DATASET_DIR, ext = define_dataset(True, False)
video2label, labels_list = get_video2label()
print(labels_list)
print(video2label)

['very distracted', 'distracted', 'engaged', 'very engaged']
{'subject_1_Vid_1': 1.0, 'subject_1_Vid_2': 1.0, 'subject_1_Vid_3': 0.66, 'subject_1_Vid_4': 1.0, 'subject_1_Vid_5': 1.0, 'subject_31_Vid_6': 1.0, 'subject_2_Vid_6': 0.33, 'subject_3_Vid_6': 1.0, 'subject_3_Vid_1': 0.33, 'subject_3_Vid_2': 0.33, 'subject_3_Vid_3': 0.66, 'subject_3_Vid_4': 0.66, 'subject_3_Vid_5': 0.33, 'subject_3_Vid_7': 1.0, 'subject_4_Vid_6': 1.0, 'subject_5_Vid_6': 1.0, 'subject_6_Vid_6': 0.33, 'subject_7_Vid_1': 0.66, 'subject_7_Vid_2': 0.66, 'subject_7_Vid_3': 0.66, 'subject_7_Vid_4': 0.66, 'subject_7_Vid_5': 0.33, 'subject_8_Vid_6': 1.0, 'subject_9_Vid_6': 0.0, 'subject_10_Vid_6': 0.66, 'subject_11_Vid_6': 1.0, 'subject_12_Vid_6': 0.0, 'subject_13_Vid_6': 0.66, 'subject_14_Vid_6': 0.66, 'subject_15_Vid_6': 0.66, 'subject_16_Vid_6': 0.33, 'subject_17_Vid_6': 0.66, 'subject_18_Vid_6': 0.33, 'subject_19_Vid_6': 1.0, 'subject_20_Vid_6': 0.66, 'subject_20_Vid_1': 0.66, 'subject_20_Vid_2': 0.66, 'subject_20_V

### Std

In [27]:
stat_func = np.std
stat_name = "std"

#### Traditional split

In [28]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [29]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [30]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 5/147: 3.401360544217687%
0.33 35/147: 23.80952380952381%
0.66 79/147: 53.74149659863946%
1.0 28/147: 19.047619047619047%
val:
0.0 4/48: 8.333333333333334%
0.33 10/48: 20.833333333333332%
0.66 19/48: 39.583333333333336%
1.0 15/48: 31.25%


##### Single attention

In [31]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_4_std_single_attention_traditional


In [32]:
CONCATENATE_STAT = True

In [33]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2560) (5449,)
(2284, 128, 2560) (2284,)


In [34]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-07-21 20:57:19.794348: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                              

In [35]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20


2024-07-21 20:57:28.377540: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0xcab7b090 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-21 20:57:28.377561: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-07-21 20:57:28.382472: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-21 20:57:28.415997: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-07-21 20:57:28.468254: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


43/43 [==============================] - 6s 102ms/step - loss: 0.0463 - mae: 0.1715 - val_loss: 0.0895 - val_mae: 0.2561
Epoch 2/20
43/43 [==============================] - 1s 32ms/step - loss: 0.0175 - mae: 0.1041 - val_loss: 0.0881 - val_mae: 0.2481
Epoch 3/20
43/43 [==============================] - 1s 31ms/step - loss: 0.0114 - mae: 0.0791 - val_loss: 0.0936 - val_mae: 0.2515
Epoch 4/20
43/43 [==============================] - 1s 31ms/step - loss: 0.0078 - mae: 0.0619 - val_loss: 0.0885 - val_mae: 0.2469
Epoch 5/20
43/43 [==============================] - 1s 32ms/step - loss: 0.0057 - mae: 0.0523 - val_loss: 0.0915 - val_mae: 0.2512
Epoch 6/20
43/43 [==============================] - 1s 32ms/step - loss: 0.0042 - mae: 0.0460 - val_loss: 0.1035 - val_mae: 0.2689
Epoch 7/20
43/43 [==============================] - 1s 32ms/step - loss: 0.0032 - mae: 0.0399 - val_loss: 0.0986 - val_mae: 0.2649
Epoch 8/20
43/43 [==============================] - 1s 32ms/step - loss: 0.0025 - mae: 0.0346

In [36]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3608044/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [37]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_std_single_attention_traditional Accuracy:  0.2916666666666667 MSE:  0.10009201708232872 UAR:  0.3478070175438596 Recall:  N/A Precision:  N/A F1:  0.3157444005270092


In [38]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [39]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3608044/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [40]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_std_single_attention_traditional_best Accuracy:  0.3541666666666667 MSE:  0.08007323866137694 UAR:  0.2850877192982456 Recall:  N/A Precision:  N/A F1:  0.2765014024088434


##### Self attention

In [41]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_4_std_self_attention_traditional


In [42]:
CONCATENATE_STAT = True

In [43]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2560) (5449,)
(2284, 128, 2560) (2284,)


In [44]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention (Attention)       (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [45]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 7s 152ms/step - loss: 0.2071 - mae: 0.3759 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 2/20
43/43 [==============================] - 4s 91ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 3/20
43/43 [==============================] - 4s 91ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 4/20
43/43 [==============================] - 4s 91ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 5/20
43/43 [==============================] - 4s 91ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 6/20
43/43 [==============================] - 4s 91ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 7/20
43/43 [==============================] - 4s 91ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 8/20
43/43 [==============================] - 4s 90ms/step - loss: 0.2089 - 

In [46]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3608044/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [47]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_std_self_attention_traditional Accuracy:  0.3125 MSE:  0.22261249999999996 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.11904761904761904


In [48]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [49]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3608044/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [50]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_std_self_attention_traditional_best Accuracy:  0.3125 MSE:  0.22261249999999996 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.11904761904761904


#### Balanced split

In [51]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [52]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [53]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 5/147: 3.401360544217687%
0.33 35/147: 23.80952380952381%
0.66 79/147: 53.74149659863946%
1.0 28/147: 19.047619047619047%
val:
0.0 4/48: 8.333333333333334%
0.33 10/48: 20.833333333333332%
0.66 19/48: 39.583333333333336%
1.0 15/48: 31.25%


In [54]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.767)

(147,) (147,)
(48,) (48,)
(195,) (195,)
[0.   0.33 0.66 1.  ] [ 9 45 98 43]
195
147
48


In [55]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 6/147: 4.081632653061225%
0.33 34/147: 23.12925170068027%
0.66 75/147: 51.02040816326531%
1.0 32/147: 21.768707482993197%
val:
0.0 3/48: 6.25%
0.33 11/48: 22.916666666666668%
0.66 23/48: 47.916666666666664%
1.0 11/48: 22.916666666666668%


##### Single attention

In [56]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_4_std_single_attention_balanced


In [57]:
CONCATENATE_STAT = True

In [58]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5641, 128, 2560) (5641,)
(2092, 128, 2560) (2092,)


In [59]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [60]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
45/45 [==============================] - 4s 72ms/step - loss: 0.0421 - mae: 0.1615 - val_loss: 0.0632 - val_mae: 0.2078
Epoch 2/20
45/45 [==============================] - 2s 37ms/step - loss: 0.0124 - mae: 0.0855 - val_loss: 0.0632 - val_mae: 0.2054
Epoch 3/20
45/45 [==============================] - 1s 31ms/step - loss: 0.0072 - mae: 0.0624 - val_loss: 0.0682 - val_mae: 0.2172
Epoch 4/20
45/45 [==============================] - 1s 30ms/step - loss: 0.0053 - mae: 0.0538 - val_loss: 0.0716 - val_mae: 0.2219
Epoch 5/20
45/45 [==============================] - 1s 31ms/step - loss: 0.0035 - mae: 0.0407 - val_loss: 0.0742 - val_mae: 0.2249
Epoch 6/20
45/45 [==============================] - 1s 32ms/step - loss: 0.0028 - mae: 0.0368 - val_loss: 0.0769 - val_mae: 0.2310
Epoch 7/20
45/45 [==============================] - 2s 36ms/step - loss: 0.0024 - mae: 0.0345 - val_loss: 0.0777 - val_mae: 0.2317
Epoch 8/20
45/45 [==============================] - 1s 31ms/step - loss: 0.0017 - m

In [61]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3608044/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [62]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_std_single_attention_balanced Accuracy:  0.3541666666666667 MSE:  0.0828139840055649 UAR:  0.363965744400527 Recall:  N/A Precision:  N/A F1:  0.36019736842105265


In [63]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [64]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3608044/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [65]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_std_single_attention_balanced_best Accuracy:  0.4583333333333333 MSE:  0.056269898658401506 UAR:  0.4183135704874835 Recall:  N/A Precision:  N/A F1:  0.4298611111111111


##### Self attention

In [66]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_4_std_self_attention_balanced


In [67]:
CONCATENATE_STAT = True

In [68]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5641, 128, 2560) (5641,)
(2092, 128, 2560) (2092,)


In [69]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_1 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [70]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
45/45 [==============================] - 7s 138ms/step - loss: 0.2181 - mae: 0.3788 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 2/20
45/45 [==============================] - 4s 88ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 3/20
45/45 [==============================] - 4s 88ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 4/20
45/45 [==============================] - 4s 99ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 5/20
45/45 [==============================] - 4s 89ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 6/20
45/45 [==============================] - 4s 88ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 7/20
45/45 [==============================] - 4s 87ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 8/20
45/45 [==============================] - 4s 88ms/step - loss: 0.2202 - 

In [71]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3608044/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [72]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_std_self_attention_balanced Accuracy:  0.22916666666666666 MSE:  0.2207645833333333 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.09322033898305083


In [73]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [74]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3608044/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [75]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_std_self_attention_balanced_best Accuracy:  0.22916666666666666 MSE:  0.2207645833333333 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.09322033898305083


### Max

In [27]:
stat_func = np.max
stat_name = "max"

#### Traditional split

In [77]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [78]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [79]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 5/147: 3.401360544217687%
0.33 35/147: 23.80952380952381%
0.66 79/147: 53.74149659863946%
1.0 28/147: 19.047619047619047%
val:
0.0 4/48: 8.333333333333334%
0.33 10/48: 20.833333333333332%
0.66 19/48: 39.583333333333336%
1.0 15/48: 31.25%


##### Single attention

In [80]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_4_max_single_attention_traditional


In [81]:
CONCATENATE_STAT = True

In [82]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2560) (5449,)
(2284, 128, 2560) (2284,)


In [83]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [84]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 5s 89ms/step - loss: 0.2061 - mae: 0.3746 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 2/20
43/43 [==============================] - 1s 31ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 3/20
43/43 [==============================] - 1s 31ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 4/20
43/43 [==============================] - 1s 32ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 5/20
43/43 [==============================] - 1s 31ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 6/20
43/43 [==============================] - 1s 32ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 7/20
43/43 [==============================] - 1s 32ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 8/20
43/43 [==============================] - 1s 32ms/step - loss: 0.2089 - m

In [85]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3608044/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [86]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_max_single_attention_traditional Accuracy:  0.3125 MSE:  0.22261250000000113 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.11904761904761904


In [87]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [88]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3608044/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [89]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_max_single_attention_traditional_best Accuracy:  0.3125 MSE:  0.22261250000000113 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.11904761904761904


##### Self attention

In [90]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_4_max_self_attention_traditional


In [91]:
CONCATENATE_STAT = True

In [92]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2560) (5449,)
(2284, 128, 2560) (2284,)


In [93]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_2 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [94]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 7s 135ms/step - loss: 0.2059 - mae: 0.3744 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 2/20
43/43 [==============================] - 4s 91ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 3/20
43/43 [==============================] - 4s 92ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 4/20
43/43 [==============================] - 4s 91ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 5/20
43/43 [==============================] - 4s 91ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 6/20
43/43 [==============================] - 4s 92ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 7/20
43/43 [==============================] - 4s 91ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 8/20
43/43 [==============================] - 4s 90ms/step - loss: 0.2089 - 

In [95]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3608044/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [96]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_max_self_attention_traditional Accuracy:  0.3125 MSE:  0.22261249999999996 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.11904761904761904


In [97]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [98]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3608044/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [99]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_max_self_attention_traditional_best Accuracy:  0.3125 MSE:  0.22261249999999996 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.11904761904761904


#### Balanced split

In [28]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [29]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [30]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 5/147: 3.401360544217687%
0.33 35/147: 23.80952380952381%
0.66 79/147: 53.74149659863946%
1.0 28/147: 19.047619047619047%
val:
0.0 4/48: 8.333333333333334%
0.33 10/48: 20.833333333333332%
0.66 19/48: 39.583333333333336%
1.0 15/48: 31.25%


In [31]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.767)

(147,) (147,)
(48,) (48,)
(195,) (195,)
[0.   0.33 0.66 1.  ] [ 9 45 98 43]
195
147
48


In [32]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 6/147: 4.081632653061225%
0.33 34/147: 23.12925170068027%
0.66 75/147: 51.02040816326531%
1.0 32/147: 21.768707482993197%
val:
0.0 3/48: 6.25%
0.33 11/48: 22.916666666666668%
0.66 23/48: 47.916666666666664%
1.0 11/48: 22.916666666666668%


##### Single attention

In [33]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_4_max_single_attention_balanced


In [34]:
CONCATENATE_STAT = True

In [35]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5641, 128, 2560) (5641,)
(2092, 128, 2560) (2092,)


In [36]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-07-22 05:20:12.882923: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                              

In [37]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20


2024-07-22 05:20:22.010341: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f8a48007850 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-22 05:20:22.010361: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-07-22 05:20:22.015152: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-22 05:20:22.056624: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-07-22 05:20:22.109869: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


45/45 [==============================] - 6s 94ms/step - loss: 0.2172 - mae: 0.3782 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 2/20
45/45 [==============================] - 1s 30ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 3/20
45/45 [==============================] - 1s 30ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 4/20
45/45 [==============================] - 1s 30ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 5/20
45/45 [==============================] - 1s 30ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 6/20
45/45 [==============================] - 1s 30ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 7/20
45/45 [==============================] - 1s 30ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 8/20
45/45 [==============================] - 1s 30ms/step - loss: 0.2202 - mae: 0.3811 

In [38]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3803109/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [39]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_max_single_attention_balanced Accuracy:  0.22916666666666666 MSE:  0.2207645833333333 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.09322033898305083


In [40]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [41]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3803109/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [42]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_max_single_attention_balanced_best Accuracy:  0.22916666666666666 MSE:  0.2207645833333336 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.09322033898305083


##### Self attention

In [43]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_4_max_self_attention_balanced


In [44]:
CONCATENATE_STAT = True

In [45]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5641, 128, 2560) (5641,)
(2092, 128, 2560) (2092,)


In [46]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention (Attention)       (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [47]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
45/45 [==============================] - 8s 152ms/step - loss: 0.2205 - mae: 0.3819 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 2/20
45/45 [==============================] - 4s 88ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 3/20
45/45 [==============================] - 4s 87ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 4/20
45/45 [==============================] - 4s 88ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 5/20
45/45 [==============================] - 4s 87ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 6/20
45/45 [==============================] - 4s 88ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 7/20
45/45 [==============================] - 4s 88ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 8/20
45/45 [==============================] - 4s 87ms/step - loss: 0.2202 - 

In [48]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3803109/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [49]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_max_self_attention_balanced Accuracy:  0.22916666666666666 MSE:  0.2207645833333333 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.09322033898305083


In [50]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [51]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3803109/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [52]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_max_self_attention_balanced_best Accuracy:  0.22916666666666666 MSE:  0.2207645833333333 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.09322033898305083


### STAT

In [27]:
stat_func = compute_descriptor
stat_name = "STAT"

#### Traditional split

In [28]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [29]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [30]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 5/147: 3.401360544217687%
0.33 35/147: 23.80952380952381%
0.66 79/147: 53.74149659863946%
1.0 28/147: 19.047619047619047%
val:
0.0 4/48: 8.333333333333334%
0.33 10/48: 20.833333333333332%
0.66 19/48: 39.583333333333336%
1.0 15/48: 31.25%


##### Single attention

In [31]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_4_STAT_single_attention_traditional


In [32]:
CONCATENATE_STAT = True

In [33]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 3840) (5449,)
(2284, 128, 3840) (2284,)


In [34]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-07-22 06:05:31.848209: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3840)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3841      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                              

In [35]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

In [36]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3828290/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [37]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_STAT_single_attention_traditional Accuracy:  0.25 MSE:  0.1146485906264167 UAR:  0.22631578947368422 Recall:  N/A Precision:  N/A F1:  0.2108556832694764


In [39]:
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    modelAtn.set_weights(best_model_weights)
    save_weights(modelAtn, weights_name)

In [40]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3828290/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [41]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_STAT_single_attention_traditional_best Accuracy:  0.3958333333333333 MSE:  0.08042476034517941 UAR:  0.3078947368421053 Recall:  N/A Precision:  N/A F1:  0.2902121374865736


##### Self attention

In [42]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_4_STAT_self_attention_traditional


In [43]:
CONCATENATE_STAT = True

In [44]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 3840) (5449,)
(2284, 128, 3840) (2284,)


In [45]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3840)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 3840)           1474560   ['image_set[0][0]']           
                                                          0                                       
                                                                                                  
 value (Dense)               (None, None, 3840)           1474560   ['image_set[0][0]']           
                                                          0                                       
                                                                                            

In [46]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

In [47]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3828290/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [48]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_STAT_self_attention_traditional Accuracy:  0.3541666666666667 MSE:  0.10893076012359715 UAR:  0.38728070175438595 Recall:  N/A Precision:  N/A F1:  0.36451612903225805


In [49]:
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    modelAtn.set_weights(best_model_weights)
    save_weights(modelAtn, weights_name)

In [50]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3828290/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [51]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_STAT_self_attention_traditional_best Accuracy:  0.4166666666666667 MSE:  0.07915359221476707 UAR:  0.4232456140350877 Recall:  N/A Precision:  N/A F1:  0.45722610722610724


#### Balanced split

In [52]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [53]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [54]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 5/147: 3.401360544217687%
0.33 35/147: 23.80952380952381%
0.66 79/147: 53.74149659863946%
1.0 28/147: 19.047619047619047%
val:
0.0 4/48: 8.333333333333334%
0.33 10/48: 20.833333333333332%
0.66 19/48: 39.583333333333336%
1.0 15/48: 31.25%


In [55]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.767)

(147, 2560) (147,)
(48, 2560) (48,)
(195, 2560) (195,)
[0.   0.33 0.66 1.  ] [ 9 45 98 43]
195
147
48


In [56]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 6/147: 4.081632653061225%
0.33 34/147: 23.12925170068027%
0.66 75/147: 51.02040816326531%
1.0 32/147: 21.768707482993197%
val:
0.0 3/48: 6.25%
0.33 11/48: 22.916666666666668%
0.66 23/48: 47.916666666666664%
1.0 11/48: 22.916666666666668%


##### Single attention

In [57]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_4_STAT_single_attention_balanced


In [58]:
CONCATENATE_STAT = True

In [59]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5641, 128, 3840) (5641,)
(2092, 128, 3840) (2092,)


In [60]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3840)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3841      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [61]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20


2024-07-22 06:09:52.493356: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f7eec008940 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-22 06:09:52.493378: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-07-22 06:09:52.498201: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-22 06:09:52.535635: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-07-22 06:09:52.588560: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


45/45 [==============================] - 8s 146ms/step - loss: 0.0425 - mae: 0.1595 - val_loss: 0.0671 - val_mae: 0.2109
Epoch 2/20
45/45 [==============================] - 2s 43ms/step - loss: 0.0109 - mae: 0.0782 - val_loss: 0.0690 - val_mae: 0.2114
Epoch 3/20
45/45 [==============================] - 2s 47ms/step - loss: 0.0057 - mae: 0.0518 - val_loss: 0.0778 - val_mae: 0.2242
Epoch 4/20
45/45 [==============================] - 2s 43ms/step - loss: 0.0043 - mae: 0.0455 - val_loss: 0.0797 - val_mae: 0.2309
Epoch 5/20
45/45 [==============================] - 2s 44ms/step - loss: 0.0030 - mae: 0.0362 - val_loss: 0.0795 - val_mae: 0.2317
Epoch 6/20
45/45 [==============================] - 2s 44ms/step - loss: 0.0021 - mae: 0.0302 - val_loss: 0.0808 - val_mae: 0.2333
Epoch 7/20
45/45 [==============================] - 2s 45ms/step - loss: 0.0021 - mae: 0.0313 - val_loss: 0.0903 - val_mae: 0.2447
Epoch 8/20
45/45 [==============================] - 2s 44ms/step - loss: 0.0016 - mae: 0.0276

In [62]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3828290/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [63]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_STAT_single_attention_balanced Accuracy:  0.3541666666666667 MSE:  0.08639217736825654 UAR:  0.363965744400527 Recall:  N/A Precision:  N/A F1:  0.34731934731934727


In [64]:
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    modelAtn.set_weights(best_model_weights)
    save_weights(modelAtn, weights_name)

In [65]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3828290/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [66]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_4_STAT_single_attention_balanced_best Accuracy:  0.4791666666666667 MSE:  0.0586605660204162 UAR:  0.42918313570487476 Recall:  N/A Precision:  N/A F1:  0.4437641723356009


##### Self attention

In [67]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_4_STAT_self_attention_balanced


In [68]:
CONCATENATE_STAT = True

In [69]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5641, 128, 3840) (5641,)
(2092, 128, 3840) (2092,)


In [70]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3840)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 3840)           1474560   ['image_set[0][0]']           
                                                          0                                       
                                                                                                  
 value (Dense)               (None, None, 3840)           1474560   ['image_set[0][0]']           
                                                          0                                       
                                                                                            

In [ ]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
45/45 [==============================] - 13s 270ms/step - loss: 0.2176 - mae: 0.3791 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 2/20
45/45 [==============================] - 8s 170ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 3/20
45/45 [==============================] - 9s 197ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 4/20
45/45 [==============================] - 8s 172ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 5/20
45/45 [==============================] - 8s 171ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 6/20
45/45 [==============================] - 8s 171ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 7/20
45/45 [==============================] - 8s 170ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 8/20
45/45 [==============================] - 8s 170ms/step - loss: 0

In [ ]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

In [ ]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

In [ ]:
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    modelAtn.set_weights(best_model_weights)
    save_weights(modelAtn, weights_name)

In [ ]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

In [ ]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

## MobileNet_7

In [25]:
base_model_key = 'mobilenet_7.h5'

In [26]:
ENGAGE_WILD_DATASET, BIN_CLASSIFICATION, N_CLASSES, DATASET_NAME, BASE_DATASET_DIR, ext = define_dataset(True, False)
video2label, labels_list = get_video2label()
print(labels_list)
print(video2label)

['very distracted', 'distracted', 'engaged', 'very engaged']
{'subject_1_Vid_1': 1.0, 'subject_1_Vid_2': 1.0, 'subject_1_Vid_3': 0.66, 'subject_1_Vid_4': 1.0, 'subject_1_Vid_5': 1.0, 'subject_31_Vid_6': 1.0, 'subject_2_Vid_6': 0.33, 'subject_3_Vid_6': 1.0, 'subject_3_Vid_1': 0.33, 'subject_3_Vid_2': 0.33, 'subject_3_Vid_3': 0.66, 'subject_3_Vid_4': 0.66, 'subject_3_Vid_5': 0.33, 'subject_3_Vid_7': 1.0, 'subject_4_Vid_6': 1.0, 'subject_5_Vid_6': 1.0, 'subject_6_Vid_6': 0.33, 'subject_7_Vid_1': 0.66, 'subject_7_Vid_2': 0.66, 'subject_7_Vid_3': 0.66, 'subject_7_Vid_4': 0.66, 'subject_7_Vid_5': 0.33, 'subject_8_Vid_6': 1.0, 'subject_9_Vid_6': 0.0, 'subject_10_Vid_6': 0.66, 'subject_11_Vid_6': 1.0, 'subject_12_Vid_6': 0.0, 'subject_13_Vid_6': 0.66, 'subject_14_Vid_6': 0.66, 'subject_15_Vid_6': 0.66, 'subject_16_Vid_6': 0.33, 'subject_17_Vid_6': 0.66, 'subject_18_Vid_6': 0.33, 'subject_19_Vid_6': 1.0, 'subject_20_Vid_6': 0.66, 'subject_20_Vid_1': 0.66, 'subject_20_Vid_2': 0.66, 'subject_20_V

### Std

In [27]:
stat_func = np.std
stat_name = "std"

#### Traditional split

In [28]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [29]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [30]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 5/147: 3.401360544217687%
0.33 35/147: 23.80952380952381%
0.66 79/147: 53.74149659863946%
1.0 28/147: 19.047619047619047%
val:
0.0 4/48: 8.333333333333334%
0.33 10/48: 20.833333333333332%
0.66 19/48: 39.583333333333336%
1.0 15/48: 31.25%


##### Single attention

In [31]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

mobilenet_7.h5_EngageWild_4_std_single_attention_traditional


In [32]:
CONCATENATE_STAT = True

In [33]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2048) (5449,)
(2284, 128, 2048) (2284,)


In [34]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-07-23 15:58:54.594011: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                              

In [35]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20


2024-07-23 15:59:01.869079: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x13b0e8840 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-23 15:59:01.869100: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-07-23 15:59:01.873846: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-23 15:59:01.905719: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-07-23 15:59:01.957648: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


43/43 [==============================] - 5s 87ms/step - loss: 0.0493 - mae: 0.1724 - val_loss: 0.0992 - val_mae: 0.2626
Epoch 2/20
43/43 [==============================] - 1s 26ms/step - loss: 0.0181 - mae: 0.1056 - val_loss: 0.0975 - val_mae: 0.2508
Epoch 3/20
43/43 [==============================] - 1s 26ms/step - loss: 0.0098 - mae: 0.0745 - val_loss: 0.1053 - val_mae: 0.2578
Epoch 4/20
43/43 [==============================] - 1s 26ms/step - loss: 0.0065 - mae: 0.0600 - val_loss: 0.1108 - val_mae: 0.2641
Epoch 5/20
43/43 [==============================] - 1s 26ms/step - loss: 0.0044 - mae: 0.0480 - val_loss: 0.1173 - val_mae: 0.2652
Epoch 6/20
43/43 [==============================] - 1s 26ms/step - loss: 0.0033 - mae: 0.0416 - val_loss: 0.1229 - val_mae: 0.2696
Epoch 7/20
43/43 [==============================] - 1s 26ms/step - loss: 0.0024 - mae: 0.0352 - val_loss: 0.1229 - val_mae: 0.2677
Epoch 8/20
43/43 [==============================] - 1s 26ms/step - loss: 0.0017 - mae: 0.0292 

In [36]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3864770/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [37]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_std_single_attention_traditional Accuracy:  0.4375 MSE:  0.11541946616558597 UAR:  0.356578947368421 Recall:  N/A Precision:  N/A F1:  0.3360759953501889


In [38]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [39]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3864770/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [40]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_std_single_attention_traditional_best Accuracy:  0.4166666666666667 MSE:  0.08423179411250159 UAR:  0.30087719298245613 Recall:  N/A Precision:  N/A F1:  0.2812820512820513


##### Self attention

In [41]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

mobilenet_7.h5_EngageWild_4_std_self_attention_traditional


In [42]:
CONCATENATE_STAT = True

In [43]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2048) (5449,)
(2284, 128, 2048) (2284,)


In [44]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 attention (Attention)       (None, None, 2048)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [45]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 6s 129ms/step - loss: 0.2048 - mae: 0.3731 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 2/20
43/43 [==============================] - 3s 65ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 3/20
43/43 [==============================] - 3s 64ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 4/20
43/43 [==============================] - 3s 65ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 5/20
43/43 [==============================] - 3s 65ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 6/20
43/43 [==============================] - 3s 66ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 7/20
43/43 [==============================] - 3s 66ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 8/20
43/43 [==============================] - 3s 65ms/step - loss: 0.2089 - 

In [46]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3864770/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [47]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_std_self_attention_traditional Accuracy:  0.3125 MSE:  0.22261249999999996 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.11904761904761904


In [48]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [49]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3864770/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [50]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_std_self_attention_traditional_best Accuracy:  0.3125 MSE:  0.22261249999999996 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.11904761904761904


#### Balanced split

In [51]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [52]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [53]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 5/147: 3.401360544217687%
0.33 35/147: 23.80952380952381%
0.66 79/147: 53.74149659863946%
1.0 28/147: 19.047619047619047%
val:
0.0 4/48: 8.333333333333334%
0.33 10/48: 20.833333333333332%
0.66 19/48: 39.583333333333336%
1.0 15/48: 31.25%


In [54]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.767)

(147,) (147,)
(48,) (48,)
(195,) (195,)
[0.   0.33 0.66 1.  ] [ 9 45 98 43]
195
147
48


In [55]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 6/147: 4.081632653061225%
0.33 34/147: 23.12925170068027%
0.66 75/147: 51.02040816326531%
1.0 32/147: 21.768707482993197%
val:
0.0 3/48: 6.25%
0.33 11/48: 22.916666666666668%
0.66 23/48: 47.916666666666664%
1.0 11/48: 22.916666666666668%


##### Single attention

In [56]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

mobilenet_7.h5_EngageWild_4_std_single_attention_balanced


In [57]:
CONCATENATE_STAT = True

In [58]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5641, 128, 2048) (5641,)
(2092, 128, 2048) (2092,)


In [59]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [60]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
45/45 [==============================] - 3s 60ms/step - loss: 0.0497 - mae: 0.1738 - val_loss: 0.0687 - val_mae: 0.2079
Epoch 2/20
45/45 [==============================] - 1s 26ms/step - loss: 0.0171 - mae: 0.1051 - val_loss: 0.0673 - val_mae: 0.2081
Epoch 3/20
45/45 [==============================] - 1s 25ms/step - loss: 0.0086 - mae: 0.0705 - val_loss: 0.0686 - val_mae: 0.2080
Epoch 4/20
45/45 [==============================] - 1s 25ms/step - loss: 0.0052 - mae: 0.0536 - val_loss: 0.0678 - val_mae: 0.2071
Epoch 5/20
45/45 [==============================] - 1s 25ms/step - loss: 0.0037 - mae: 0.0443 - val_loss: 0.0726 - val_mae: 0.2110
Epoch 6/20
45/45 [==============================] - 1s 26ms/step - loss: 0.0030 - mae: 0.0398 - val_loss: 0.0736 - val_mae: 0.2154
Epoch 7/20
45/45 [==============================] - 1s 25ms/step - loss: 0.0026 - mae: 0.0379 - val_loss: 0.0758 - val_mae: 0.2195
Epoch 8/20
45/45 [==============================] - 1s 25ms/step - loss: 0.0020 - m

In [61]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3864770/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [62]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_std_single_attention_balanced Accuracy:  0.4583333333333333 MSE:  0.06568898942859348 UAR:  0.3814229249011858 Recall:  N/A Precision:  N/A F1:  0.3470916568742656


In [63]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [64]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3864770/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [65]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_std_single_attention_balanced_best Accuracy:  0.5208333333333334 MSE:  0.05547004883940995 UAR:  0.4627799736495389 Recall:  N/A Precision:  N/A F1:  0.4724120082815735


##### Self attention

In [66]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

mobilenet_7.h5_EngageWild_4_std_self_attention_balanced


In [67]:
CONCATENATE_STAT = True

In [68]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5641, 128, 2048) (5641,)
(2092, 128, 2048) (2092,)


In [69]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 attention_1 (Attention)     (None, None, 2048)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [70]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
45/45 [==============================] - 5s 94ms/step - loss: 0.2169 - mae: 0.3773 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 2/20
45/45 [==============================] - 3s 63ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 3/20
45/45 [==============================] - 3s 63ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 4/20
45/45 [==============================] - 3s 62ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 5/20
45/45 [==============================] - 3s 62ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 6/20
45/45 [==============================] - 3s 63ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 7/20
45/45 [==============================] - 3s 63ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 8/20
45/45 [==============================] - 3s 62ms/step - loss: 0.2202 - m

In [71]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3864770/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [72]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_std_self_attention_balanced Accuracy:  0.22916666666666666 MSE:  0.2207645833333333 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.09322033898305083


In [73]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [74]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3864770/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [75]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_std_self_attention_balanced_best Accuracy:  0.22916666666666666 MSE:  0.2207645833333333 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.09322033898305083


### Max

In [27]:
stat_func = np.max
stat_name = "max"

#### Traditional split

In [77]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [78]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [79]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 5/147: 3.401360544217687%
0.33 35/147: 23.80952380952381%
0.66 79/147: 53.74149659863946%
1.0 28/147: 19.047619047619047%
val:
0.0 4/48: 8.333333333333334%
0.33 10/48: 20.833333333333332%
0.66 19/48: 39.583333333333336%
1.0 15/48: 31.25%


##### Single attention

In [80]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

mobilenet_7.h5_EngageWild_4_max_single_attention_traditional


In [81]:
CONCATENATE_STAT = True

In [82]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2048) (5449,)
(2284, 128, 2048) (2284,)


In [83]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [84]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 4s 89ms/step - loss: 0.2723 - mae: 0.4406 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 2/20
43/43 [==============================] - 1s 26ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 3/20
43/43 [==============================] - 1s 26ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 4/20
43/43 [==============================] - 1s 26ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 5/20
43/43 [==============================] - 1s 26ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 6/20
43/43 [==============================] - 1s 26ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 7/20
43/43 [==============================] - 1s 34ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 8/20
43/43 [==============================] - 1s 27ms/step - loss: 0.2089 - m

In [85]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3864770/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [86]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_max_single_attention_traditional Accuracy:  0.3125 MSE:  0.2226124206764275 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.11904761904761904


In [87]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [88]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3864770/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [89]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_max_single_attention_traditional_best Accuracy:  0.3125 MSE:  0.2226123397141818 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.11904761904761904


##### Self attention

In [90]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

mobilenet_7.h5_EngageWild_4_max_self_attention_traditional


In [91]:
CONCATENATE_STAT = True

In [92]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2048) (5449,)
(2284, 128, 2048) (2284,)


In [93]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 attention_2 (Attention)     (None, None, 2048)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [94]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 7s 131ms/step - loss: 0.4446 - mae: 0.6123 - val_loss: 0.5077 - val_mae: 0.6284
Epoch 2/20
43/43 [==============================] - 3s 65ms/step - loss: 0.4538 - mae: 0.6224 - val_loss: 0.5077 - val_mae: 0.6284
Epoch 3/20
43/43 [==============================] - 3s 65ms/step - loss: 0.4538 - mae: 0.6224 - val_loss: 0.5077 - val_mae: 0.6284
Epoch 4/20
43/43 [==============================] - 3s 65ms/step - loss: 0.4538 - mae: 0.6224 - val_loss: 0.5077 - val_mae: 0.6284
Epoch 5/20
43/43 [==============================] - 3s 65ms/step - loss: 0.4538 - mae: 0.6224 - val_loss: 0.5077 - val_mae: 0.6284
Epoch 6/20
43/43 [==============================] - 3s 65ms/step - loss: 0.4538 - mae: 0.6224 - val_loss: 0.5077 - val_mae: 0.6284
Epoch 7/20
43/43 [==============================] - 3s 65ms/step - loss: 0.4538 - mae: 0.6224 - val_loss: 0.5077 - val_mae: 0.6284
Epoch 8/20
43/43 [==============================] - 3s 65ms/step - loss: 0.4538 - 

In [95]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3864770/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [96]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_max_self_attention_traditional Accuracy:  0.08333333333333333 MSE:  0.5076125 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.038461538461538464


In [97]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [98]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3864770/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [99]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_max_self_attention_traditional_best Accuracy:  0.08333333333333333 MSE:  0.5076125 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.038461538461538464


#### Balanced split

In [28]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [29]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [30]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 5/147: 3.401360544217687%
0.33 35/147: 23.80952380952381%
0.66 79/147: 53.74149659863946%
1.0 28/147: 19.047619047619047%
val:
0.0 4/48: 8.333333333333334%
0.33 10/48: 20.833333333333332%
0.66 19/48: 39.583333333333336%
1.0 15/48: 31.25%


In [31]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.767)

(147,) (147,)
(48,) (48,)
(195,) (195,)
[0.   0.33 0.66 1.  ] [ 9 45 98 43]
195
147
48


In [32]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 6/147: 4.081632653061225%
0.33 34/147: 23.12925170068027%
0.66 75/147: 51.02040816326531%
1.0 32/147: 21.768707482993197%
val:
0.0 3/48: 6.25%
0.33 11/48: 22.916666666666668%
0.66 23/48: 47.916666666666664%
1.0 11/48: 22.916666666666668%


##### Single attention

In [33]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

mobilenet_7.h5_EngageWild_4_max_single_attention_balanced


In [34]:
CONCATENATE_STAT = True

In [35]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5641, 128, 2048) (5641,)
(2092, 128, 2048) (2092,)


In [36]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-07-23 16:10:15.235180: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                              

In [37]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20


2024-07-23 16:10:22.778323: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fb0a4d93cb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-23 16:10:22.778478: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-07-23 16:10:22.782701: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-23 16:10:22.813671: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-07-23 16:10:22.865698: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


45/45 [==============================] - 5s 77ms/step - loss: 0.2181 - mae: 0.3792 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 2/20
45/45 [==============================] - 1s 25ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 3/20
45/45 [==============================] - 1s 25ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 4/20
45/45 [==============================] - 1s 26ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 5/20
45/45 [==============================] - 1s 25ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 6/20
45/45 [==============================] - 1s 25ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 7/20
45/45 [==============================] - 1s 25ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 8/20
45/45 [==============================] - 1s 25ms/step - loss: 0.2202 - mae: 0.3811 

In [38]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3881236/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [39]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_max_single_attention_balanced Accuracy:  0.22916666666666666 MSE:  0.2207645833333333 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.09322033898305083


In [40]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [41]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3881236/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [42]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_max_single_attention_balanced_best Accuracy:  0.22916666666666666 MSE:  0.2207645833333333 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.09322033898305083


##### Self attention

In [43]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

mobilenet_7.h5_EngageWild_4_max_self_attention_balanced


In [44]:
CONCATENATE_STAT = True

In [45]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5641, 128, 2048) (5641,)
(2092, 128, 2048) (2092,)


In [46]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 attention (Attention)       (None, None, 2048)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [47]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
45/45 [==============================] - 6s 118ms/step - loss: 0.2176 - mae: 0.3782 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 2/20
45/45 [==============================] - 3s 63ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 3/20
45/45 [==============================] - 3s 63ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 4/20
45/45 [==============================] - 3s 63ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 5/20
45/45 [==============================] - 3s 63ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 6/20
45/45 [==============================] - 3s 63ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 7/20
45/45 [==============================] - 3s 62ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 8/20
45/45 [==============================] - 3s 63ms/step - loss: 0.2202 - 

In [48]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3881236/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [49]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_max_self_attention_balanced Accuracy:  0.22916666666666666 MSE:  0.2207645833333333 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.09322033898305083


In [50]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [51]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3881236/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [52]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_max_self_attention_balanced_best Accuracy:  0.22916666666666666 MSE:  0.2207645833333333 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.09322033898305083


### STAT

In [27]:
stat_func = compute_descriptor
stat_name = "STAT"

#### Traditional split

In [54]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [55]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [56]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 5/147: 3.401360544217687%
0.33 35/147: 23.80952380952381%
0.66 79/147: 53.74149659863946%
1.0 28/147: 19.047619047619047%
val:
0.0 4/48: 8.333333333333334%
0.33 10/48: 20.833333333333332%
0.66 19/48: 39.583333333333336%
1.0 15/48: 31.25%


##### Single attention

In [57]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

mobilenet_7.h5_EngageWild_4_STAT_single_attention_traditional


In [58]:
CONCATENATE_STAT = True

In [59]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 3072) (5449,)
(2284, 128, 3072) (2284,)


In [60]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3072)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3073      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [61]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 5s 94ms/step - loss: 0.0865 - mae: 0.2254 - val_loss: 0.1025 - val_mae: 0.2601
Epoch 2/20
43/43 [==============================] - 2s 37ms/step - loss: 0.0176 - mae: 0.1052 - val_loss: 0.1070 - val_mae: 0.2633
Epoch 3/20
43/43 [==============================] - 2s 38ms/step - loss: 0.0099 - mae: 0.0749 - val_loss: 0.1113 - val_mae: 0.2645
Epoch 4/20
43/43 [==============================] - 2s 38ms/step - loss: 0.0064 - mae: 0.0582 - val_loss: 0.1196 - val_mae: 0.2699
Epoch 5/20
43/43 [==============================] - 2s 38ms/step - loss: 0.0043 - mae: 0.0473 - val_loss: 0.1217 - val_mae: 0.2721
Epoch 6/20
43/43 [==============================] - 2s 37ms/step - loss: 0.0030 - mae: 0.0385 - val_loss: 0.1276 - val_mae: 0.2794
Epoch 7/20
43/43 [==============================] - 2s 37ms/step - loss: 0.0025 - mae: 0.0363 - val_loss: 0.1302 - val_mae: 0.2828
Epoch 8/20
43/43 [==============================] - 2s 41ms/step - loss: 0.0017 - m

In [62]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3881236/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [63]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_STAT_single_attention_traditional Accuracy:  0.375 MSE:  0.1364128734837705 UAR:  0.3017543859649123 Recall:  N/A Precision:  N/A F1:  0.29008152173913043


In [64]:
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    modelAtn.set_weights(best_model_weights)
    save_weights(modelAtn, weights_name)

In [65]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3881236/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [66]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_STAT_single_attention_traditional_best Accuracy:  0.4791666666666667 MSE:  0.08846163625492254 UAR:  0.34868421052631576 Recall:  N/A Precision:  N/A F1:  0.319724025974026


##### Self attention

In [67]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

mobilenet_7.h5_EngageWild_4_STAT_self_attention_traditional


In [68]:
CONCATENATE_STAT = True

In [69]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 3072) (5449,)
(2284, 128, 3072) (2284,)


In [70]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3072)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 3072)           9437184   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 3072)           9437184   ['image_set[0][0]']           
                                                                                                  
 attention_1 (Attention)     (None, None, 3072)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [71]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 10s 208ms/step - loss: 0.2060 - mae: 0.3745 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 2/20
43/43 [==============================] - 5s 123ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 3/20
43/43 [==============================] - 5s 123ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 4/20
43/43 [==============================] - 5s 122ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 5/20
43/43 [==============================] - 5s 122ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 6/20
43/43 [==============================] - 5s 122ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 7/20
43/43 [==============================] - 5s 122ms/step - loss: 0.2089 - mae: 0.3776 - val_loss: 0.2510 - val_mae: 0.3716
Epoch 8/20
43/43 [==============================] - 5s 122ms/step - loss: 0

In [72]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3881236/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [73]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_STAT_self_attention_traditional Accuracy:  0.3125 MSE:  0.22261249999999996 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.11904761904761904


In [74]:
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    modelAtn.set_weights(best_model_weights)
    save_weights(modelAtn, weights_name)

In [75]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3881236/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [76]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_STAT_self_attention_traditional_best Accuracy:  0.3125 MSE:  0.22261249999999996 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.11904761904761904


#### Balanced split

In [28]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [29]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [30]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 5/147: 3.401360544217687%
0.33 35/147: 23.80952380952381%
0.66 79/147: 53.74149659863946%
1.0 28/147: 19.047619047619047%
val:
0.0 4/48: 8.333333333333334%
0.33 10/48: 20.833333333333332%
0.66 19/48: 39.583333333333336%
1.0 15/48: 31.25%


In [31]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.767)

(147, 2048) (147,)
(48, 2048) (48,)
(195, 2048) (195,)
[0.   0.33 0.66 1.  ] [ 9 45 98 43]
195
147
48


In [32]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0.0 6/147: 4.081632653061225%
0.33 34/147: 23.12925170068027%
0.66 75/147: 51.02040816326531%
1.0 32/147: 21.768707482993197%
val:
0.0 3/48: 6.25%
0.33 11/48: 22.916666666666668%
0.66 23/48: 47.916666666666664%
1.0 11/48: 22.916666666666668%


##### Single attention

In [33]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

mobilenet_7.h5_EngageWild_4_STAT_single_attention_balanced


In [34]:
CONCATENATE_STAT = True

In [35]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5641, 128, 3072) (5641,)
(2092, 128, 3072) (2092,)


In [36]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-07-23 16:25:00.604515: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3072)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3073      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                              

In [37]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20


2024-07-23 16:25:11.002238: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f0bdc043370 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-23 16:25:11.002261: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-07-23 16:25:11.007070: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-23 16:25:11.041331: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-07-23 16:25:11.093106: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


45/45 [==============================] - 7s 116ms/step - loss: 0.0500 - mae: 0.1739 - val_loss: 0.0696 - val_mae: 0.2126
Epoch 2/20
45/45 [==============================] - 2s 36ms/step - loss: 0.0149 - mae: 0.0970 - val_loss: 0.0690 - val_mae: 0.2066
Epoch 3/20
45/45 [==============================] - 2s 36ms/step - loss: 0.0074 - mae: 0.0646 - val_loss: 0.0697 - val_mae: 0.2066
Epoch 4/20
45/45 [==============================] - 2s 35ms/step - loss: 0.0047 - mae: 0.0493 - val_loss: 0.0718 - val_mae: 0.2080
Epoch 5/20
45/45 [==============================] - 2s 36ms/step - loss: 0.0037 - mae: 0.0436 - val_loss: 0.0804 - val_mae: 0.2211
Epoch 6/20
45/45 [==============================] - 2s 35ms/step - loss: 0.0030 - mae: 0.0397 - val_loss: 0.0823 - val_mae: 0.2239
Epoch 7/20
45/45 [==============================] - 2s 35ms/step - loss: 0.0025 - mae: 0.0366 - val_loss: 0.0831 - val_mae: 0.2199
Epoch 8/20
45/45 [==============================] - 2s 35ms/step - loss: 0.0016 - mae: 0.0273

In [38]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3894676/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [39]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_STAT_single_attention_balanced Accuracy:  0.4166666666666667 MSE:  0.07565788414361008 UAR:  0.3596837944664032 Recall:  N/A Precision:  N/A F1:  0.3188892948483741


In [40]:
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    modelAtn.set_weights(best_model_weights)
    save_weights(modelAtn, weights_name)

In [41]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3894676/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [42]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_STAT_single_attention_balanced_best Accuracy:  0.5625 MSE:  0.056693993648760294 UAR:  0.4963768115942029 Recall:  N/A Precision:  N/A F1:  0.5041149068322982


##### Self attention

In [43]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

mobilenet_7.h5_EngageWild_4_STAT_self_attention_balanced


In [44]:
CONCATENATE_STAT = True

In [45]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5641, 128, 3072) (5641,)
(2092, 128, 3072) (2092,)


In [46]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3072)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 3072)           9437184   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 3072)           9437184   ['image_set[0][0]']           
                                                                                                  
 attention (Attention)       (None, None, 3072)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [47]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
45/45 [==============================] - 10s 204ms/step - loss: 0.2163 - mae: 0.3772 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 2/20
45/45 [==============================] - 5s 117ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 3/20
45/45 [==============================] - 5s 117ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 4/20
45/45 [==============================] - 5s 117ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 5/20
45/45 [==============================] - 5s 117ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 6/20
45/45 [==============================] - 5s 118ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 7/20
45/45 [==============================] - 5s 118ms/step - loss: 0.2202 - mae: 0.3811 - val_loss: 0.2241 - val_mae: 0.3616
Epoch 8/20
45/45 [==============================] - 5s 117ms/step - loss: 0

In [48]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3894676/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [49]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_STAT_self_attention_balanced Accuracy:  0.22916666666666666 MSE:  0.2207645833333333 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.09322033898305083


In [50]:
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    modelAtn.set_weights(best_model_weights)
    save_weights(modelAtn, weights_name)

In [51]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_3894676/870029410.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [52]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_EngageWild_4_STAT_self_attention_balanced_best Accuracy:  0.22916666666666666 MSE:  0.2207645833333333 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.09322033898305083


# EngageWild dataset (2 classes)

In [24]:
FEATURES_DIR = DATA_DIR + 'features_EngageWild/'
WEIGHTS_DIR = DATA_DIR + 'weights_EngageWild/'
TABLE_NAME = '02_EngageWild_2_classes.xlsx'

## enet_b0_8_best_afew

In [25]:
base_model_key = 'enet_b0_8_best_afew.pt'

In [26]:
ENGAGE_WILD_DATASET, BIN_CLASSIFICATION, N_CLASSES, DATASET_NAME, BASE_DATASET_DIR, ext = define_dataset(True, True)
video2label, labels_list = get_video2label()
print(labels_list)
print(video2label)

['engaged', 'distracted']
{'subject_1_Vid_1': 0, 'subject_1_Vid_2': 0, 'subject_1_Vid_3': 0, 'subject_1_Vid_4': 0, 'subject_1_Vid_5': 0, 'subject_31_Vid_6': 0, 'subject_2_Vid_6': 1, 'subject_3_Vid_6': 0, 'subject_3_Vid_1': 1, 'subject_3_Vid_2': 1, 'subject_3_Vid_3': 0, 'subject_3_Vid_4': 0, 'subject_3_Vid_5': 1, 'subject_3_Vid_7': 0, 'subject_4_Vid_6': 0, 'subject_5_Vid_6': 0, 'subject_6_Vid_6': 1, 'subject_7_Vid_1': 0, 'subject_7_Vid_2': 0, 'subject_7_Vid_3': 0, 'subject_7_Vid_4': 0, 'subject_7_Vid_5': 1, 'subject_8_Vid_6': 0, 'subject_9_Vid_6': 1, 'subject_10_Vid_6': 0, 'subject_11_Vid_6': 0, 'subject_12_Vid_6': 1, 'subject_13_Vid_6': 0, 'subject_14_Vid_6': 0, 'subject_15_Vid_6': 0, 'subject_16_Vid_6': 1, 'subject_17_Vid_6': 0, 'subject_18_Vid_6': 1, 'subject_19_Vid_6': 0, 'subject_20_Vid_6': 0, 'subject_20_Vid_1': 0, 'subject_20_Vid_2': 0, 'subject_20_Vid_3': 0, 'subject_20_Vid_4': 1, 'subject_20_Vid_5': 0, 'subject_20_Vid_5_1': 0, 'subject_20_Vid_5_2': 0, 'subject_20_Vid_7': 1, 'su

### Std

In [27]:
stat_func = np.std
stat_name = "std"

#### Traditional split

In [29]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [30]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [31]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


##### Single attention

In [32]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_traditional


In [33]:
CONCATENATE_STAT = True

In [34]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2560) (5449,)
(2284, 128, 2560) (2284,)


In [35]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-07-30 07:06:57.092910: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                              

In [36]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20


2024-07-30 07:07:06.933872: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f07b81ff2b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-30 07:07:06.934044: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-07-30 07:07:06.938196: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-30 07:07:06.993674: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-07-30 07:07:07.046402: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


43/43 [==============================] - 8s 109ms/step - loss: 0.3125 - acc: 0.8807 - auc: 0.9209 - binary_accuracy: 0.8807 - recall: 0.6660 - precision: 0.8886 - val_loss: 0.7877 - val_acc: 0.6432 - val_auc: 0.6720 - val_binary_accuracy: 0.6432 - val_recall: 0.5172 - val_precision: 0.4822
Epoch 2/20
43/43 [==============================] - 1s 33ms/step - loss: 0.1500 - acc: 0.9453 - auc: 0.9848 - binary_accuracy: 0.9453 - recall: 0.8760 - precision: 0.9285 - val_loss: 0.8964 - val_acc: 0.7229 - val_auc: 0.7030 - val_binary_accuracy: 0.7229 - val_recall: 0.5019 - val_precision: 0.6195
Epoch 3/20
43/43 [==============================] - 1s 33ms/step - loss: 0.0845 - acc: 0.9754 - auc: 0.9969 - binary_accuracy: 0.9754 - recall: 0.9409 - precision: 0.9721 - val_loss: 0.9080 - val_acc: 0.7053 - val_auc: 0.7138 - val_binary_accuracy: 0.7053 - val_recall: 0.5873 - val_precision: 0.5691
Epoch 4/20
43/43 [==============================] - 1s 32ms/step - loss: 0.0552 - acc: 0.9840 - auc: 0.9991

In [37]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4014914/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [38]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_traditional Accuracy:  0.5416666666666666 MSE:  0.4583333333333333 UAR:  0.5504201680672269 Recall:  0.5714285714285714 Precision:  0.3333333333333333 F1:  0.4210526315789474
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_traditional Accuracy:  0.625 MSE:  0.375 UAR:  0.6092436974789917 Recall:  0.5714285714285714 Precision:  0.4 F1:  0.47058823529411764
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_traditional Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.6239495798319328 Recall:  0.5714285714285714 Precision:  0.42105263157894735 F1:  0.48484848484848486
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_traditional Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6533613445378151 Recall:  0.5714285714285714 Precision:  0.47058823529411764 F1:  0.5161290322580646
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_tradit

In [39]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [40]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4014914/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [41]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_traditional_best Accuracy:  0.5208333333333334 MSE:  0.4791666666666667 UAR:  0.5777310924369747 Recall:  0.7142857142857143 Precision:  0.3448275862068966 F1:  0.46511627906976755
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_traditional_best Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.5735294117647058 Recall:  0.5 Precision:  0.3684210526315789 F1:  0.4242424242424242
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_traditional_best Accuracy:  0.625 MSE:  0.375 UAR:  0.5882352941176471 Recall:  0.5 Precision:  0.3888888888888889 F1:  0.43750000000000006
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_traditional_best Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.6029411764705883 Recall:  0.5 Precision:  0.4117647058823529 F1:  0.45161290322580644
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_att

##### Self attention

In [42]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_traditional


In [43]:
CONCATENATE_STAT = True

In [44]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2560) (5449,)
(2284, 128, 2560) (2284,)


In [45]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention (Attention)       (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [46]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 7s 140ms/step - loss: 0.5918 - acc: 0.7847 - auc: 0.7871 - binary_accuracy: 0.7847 - recall_1: 0.5607 - precision_1: 0.6410 - val_loss: 0.8495 - val_acc: 0.6475 - val_auc: 0.6537 - val_binary_accuracy: 0.6475 - val_recall_1: 0.5758 - val_precision_1: 0.4892
Epoch 2/20
43/43 [==============================] - 4s 93ms/step - loss: 0.2312 - acc: 0.9095 - auc: 0.9594 - binary_accuracy: 0.9095 - recall_1: 0.7868 - precision_1: 0.8838 - val_loss: 0.7742 - val_acc: 0.7408 - val_auc: 0.7044 - val_binary_accuracy: 0.7408 - val_recall_1: 0.5057 - val_precision_1: 0.6606
Epoch 3/20
43/43 [==============================] - 4s 103ms/step - loss: 0.1426 - acc: 0.9549 - auc: 0.9865 - binary_accuracy: 0.9549 - recall_1: 0.8940 - precision_1: 0.9450 - val_loss: 1.0746 - val_acc: 0.6756 - val_auc: 0.7150 - val_binary_accuracy: 0.6756 - val_recall_1: 0.2140 - val_precision_1: 0.5753
Epoch 4/20
43/43 [==============================] - 4s 93ms/step - loss

In [47]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4014914/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [48]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_traditional Accuracy:  0.8125 MSE:  0.1875 UAR:  0.7626050420168067 Recall:  0.6428571428571429 Precision:  0.6923076923076923 F1:  0.6666666666666666
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_traditional Accuracy:  0.8333333333333334 MSE:  0.16666666666666666 UAR:  0.7773109243697479 Recall:  0.6428571428571429 Precision:  0.75 F1:  0.6923076923076924
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_traditional Accuracy:  0.8333333333333334 MSE:  0.16666666666666666 UAR:  0.7773109243697479 Recall:  0.6428571428571429 Precision:  0.75 F1:  0.6923076923076924
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_traditional Accuracy:  0.8333333333333334 MSE:  0.16666666666666666 UAR:  0.7773109243697479 Recall:  0.6428571428571429 Precision:  0.75 F1:  0.6923076923076924
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_traditional Accura

In [49]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [50]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4014914/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [51]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_traditional_best Accuracy:  0.5625 MSE:  0.4375 UAR:  0.6281512605042017 Recall:  0.7857142857142857 Precision:  0.3793103448275862 F1:  0.5116279069767441
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_traditional_best Accuracy:  0.7291666666666666 MSE:  0.2708333333333333 UAR:  0.703781512605042 Recall:  0.6428571428571429 Precision:  0.5294117647058824 F1:  0.5806451612903226
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_traditional_best Accuracy:  0.7708333333333334 MSE:  0.22916666666666666 UAR:  0.7331932773109244 Recall:  0.6428571428571429 Precision:  0.6 F1:  0.6206896551724138
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_traditional_best Accuracy:  0.8125 MSE:  0.1875 UAR:  0.7626050420168067 Recall:  0.6428571428571429 Precision:  0.6923076923076923 F1:  0.6666666666666666
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attentio

#### Balanced split

In [52]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [53]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [54]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


In [55]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.759)

(147,) (147,)
(48,) (48,)
(195,) (195,)
[0 1] [141  54]
195
147
48


In [56]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


##### Single attention

In [57]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_balanced


In [58]:
CONCATENATE_STAT = True

In [59]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5631, 128, 2560) (5631,)
(2102, 128, 2560) (2102,)


In [60]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [61]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
44/44 [==============================] - 5s 83ms/step - loss: 0.3716 - acc: 0.8430 - auc: 0.8928 - binary_accuracy: 0.8430 - recall_2: 0.6052 - precision_2: 0.8311 - val_loss: 0.7749 - val_acc: 0.6698 - val_auc: 0.6856 - val_binary_accuracy: 0.6698 - val_recall_2: 0.4567 - val_precision_2: 0.4538
Epoch 2/20
44/44 [==============================] - 1s 32ms/step - loss: 0.1450 - acc: 0.9563 - auc: 0.9877 - binary_accuracy: 0.9563 - recall_2: 0.8910 - precision_2: 0.9620 - val_loss: 0.9226 - val_acc: 0.6613 - val_auc: 0.6917 - val_binary_accuracy: 0.6613 - val_recall_2: 0.5039 - val_precision_2: 0.4463
Epoch 3/20
44/44 [==============================] - 1s 32ms/step - loss: 0.0865 - acc: 0.9746 - auc: 0.9960 - binary_accuracy: 0.9746 - recall_2: 0.9426 - precision_2: 0.9728 - val_loss: 1.1081 - val_acc: 0.6613 - val_auc: 0.7015 - val_binary_accuracy: 0.6613 - val_recall_2: 0.4945 - val_precision_2: 0.4454
Epoch 4/20
44/44 [==============================] - 1s 32ms/step - loss: 

In [62]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4014914/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [63]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_balanced Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6323529411764706 Recall:  0.5 Precision:  0.4666666666666667 F1:  0.4827586206896552
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_balanced Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6323529411764706 Recall:  0.5 Precision:  0.4666666666666667 F1:  0.4827586206896552
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_balanced Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6323529411764706 Recall:  0.5 Precision:  0.4666666666666667 F1:  0.4827586206896552
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_balanced Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6323529411764706 Recall:  0.5 Precision:  0.4666666666666667 F1:  0.4827586206896552
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_balanced Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6470588235294117 Recall:  0.5 Precision: 

In [64]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [65]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4014914/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [66]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_balanced_best Accuracy:  0.625 MSE:  0.375 UAR:  0.6932773109243697 Recall:  0.8571428571428571 Precision:  0.42857142857142855 F1:  0.5714285714285714
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_balanced_best Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.7016806722689075 Recall:  0.7857142857142857 Precision:  0.4583333333333333 F1:  0.5789473684210527
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_balanced_best Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.6449579831932774 Recall:  0.6428571428571429 Precision:  0.42857142857142855 F1:  0.5142857142857143
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_single_attention_balanced_best Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.5966386554621849 Recall:  0.42857142857142855 Precision:  0.42857142857142855 F1:  0.42857142857142855
Metric_name:  enet_b0_8_best_

##### Self attention

In [67]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_balanced


In [68]:
CONCATENATE_STAT = True

In [69]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5631, 128, 2560) (5631,)
(2102, 128, 2560) (2102,)


In [70]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_1 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [71]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
44/44 [==============================] - 8s 168ms/step - loss: 0.5993 - acc: 0.7878 - auc: 0.8036 - binary_accuracy: 0.7878 - recall_3: 0.6098 - precision_3: 0.6631 - val_loss: 0.6822 - val_acc: 0.6817 - val_auc: 0.6883 - val_binary_accuracy: 0.6817 - val_recall_3: 0.2850 - val_precision_3: 0.4571
Epoch 2/20
44/44 [==============================] - 4s 91ms/step - loss: 0.2049 - acc: 0.9215 - auc: 0.9697 - binary_accuracy: 0.9215 - recall_3: 0.8278 - precision_3: 0.9052 - val_loss: 0.8615 - val_acc: 0.7122 - val_auc: 0.6816 - val_binary_accuracy: 0.7122 - val_recall_3: 0.3953 - val_precision_3: 0.5318
Epoch 3/20
44/44 [==============================] - 4s 90ms/step - loss: 0.1210 - acc: 0.9549 - auc: 0.9905 - binary_accuracy: 0.9549 - recall_3: 0.9086 - precision_3: 0.9406 - val_loss: 0.8721 - val_acc: 0.7393 - val_auc: 0.7289 - val_binary_accuracy: 0.7393 - val_recall_3: 0.5764 - val_precision_3: 0.5674
Epoch 4/20
44/44 [==============================] - 4s 92ms/step - loss:

In [72]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4014914/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [73]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_balanced Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6176470588235294 Recall:  0.5 Precision:  0.4375 F1:  0.4666666666666667
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_balanced Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6176470588235294 Recall:  0.5 Precision:  0.4375 F1:  0.4666666666666667
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_balanced Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6323529411764706 Recall:  0.5 Precision:  0.4666666666666667 F1:  0.4827586206896552
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_balanced Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6470588235294117 Recall:  0.5 Precision:  0.5 F1:  0.5
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_balanced Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6470588235294117 Recall:  0.5 Pre

In [74]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [75]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4014914/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [76]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_balanced_best Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.6785714285714286 Recall:  0.8571428571428571 Precision:  0.41379310344827586 F1:  0.5581395348837208
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_balanced_best Accuracy:  0.6875 MSE:  0.3125 UAR:  0.7163865546218487 Recall:  0.7857142857142857 Precision:  0.4782608695652174 F1:  0.5945945945945946
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_balanced_best Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6533613445378151 Recall:  0.5714285714285714 Precision:  0.47058823529411764 F1:  0.5161290322580646
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_balanced_best Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6050420168067226 Recall:  0.35714285714285715 Precision:  0.5 F1:  0.41666666666666663
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_std_self_attention_balanc

### Max

In [38]:
stat_func = np.max
stat_name = "max"

#### Traditional split

In [39]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [40]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [41]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


##### Single attention

In [42]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_traditional


In [43]:
CONCATENATE_STAT = True

In [44]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2560) (5449,)
(2284, 128, 2560) (2284,)


In [45]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-07-30 07:17:43.974052: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                              

In [46]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20


2024-07-30 07:17:53.529710: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fbadc14df40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-30 07:17:53.530074: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-07-30 07:17:53.534425: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-30 07:17:53.581539: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-07-30 07:17:53.646642: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


43/43 [==============================] - 8s 110ms/step - loss: 0.3723 - acc: 0.8541 - auc: 0.8832 - binary_accuracy: 0.8541 - recall: 0.6513 - precision: 0.8009 - val_loss: 0.7008 - val_acc: 0.7452 - val_auc: 0.7242 - val_binary_accuracy: 0.7452 - val_recall: 0.5045 - val_precision: 0.6723
Epoch 2/20
43/43 [==============================] - 2s 38ms/step - loss: 0.1272 - acc: 0.9638 - auc: 0.9940 - binary_accuracy: 0.9638 - recall: 0.9056 - precision: 0.9658 - val_loss: 0.7590 - val_acc: 0.7303 - val_auc: 0.7296 - val_binary_accuracy: 0.7303 - val_recall: 0.5682 - val_precision: 0.6169
Epoch 3/20
43/43 [==============================] - 1s 33ms/step - loss: 0.0557 - acc: 0.9890 - auc: 0.9997 - binary_accuracy: 0.9890 - recall: 0.9769 - precision: 0.9845 - val_loss: 0.8781 - val_acc: 0.7570 - val_auc: 0.7387 - val_binary_accuracy: 0.7570 - val_recall: 0.5682 - val_precision: 0.6737
Epoch 4/20
43/43 [==============================] - 1s 32ms/step - loss: 0.0225 - acc: 1.0000 - auc: 1.0000

In [47]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4027200/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [48]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_traditional Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6386554621848739 Recall:  0.5714285714285714 Precision:  0.4444444444444444 F1:  0.5
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_traditional Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6470588235294117 Recall:  0.5 Precision:  0.5 F1:  0.5
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_traditional Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6470588235294117 Recall:  0.5 Precision:  0.5 F1:  0.5
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_traditional Accuracy:  0.7708333333333334 MSE:  0.22916666666666666 UAR:  0.6911764705882353 Recall:  0.5 Precision:  0.6363636363636364 F1:  0.56
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_traditional Accuracy:  0.7708333333333334 MSE:  0.22916666666666666 UAR:  0.69

In [49]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [50]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4027200/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [51]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_traditional_best Accuracy:  0.5208333333333334 MSE:  0.4791666666666667 UAR:  0.5357142857142857 Recall:  0.5714285714285714 Precision:  0.32 F1:  0.41025641025641024
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_traditional_best Accuracy:  0.5833333333333334 MSE:  0.4166666666666667 UAR:  0.5588235294117647 Recall:  0.5 Precision:  0.35 F1:  0.4117647058823529
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_traditional_best Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.5735294117647058 Recall:  0.5 Precision:  0.3684210526315789 F1:  0.4242424242424242
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_traditional_best Accuracy:  0.7291666666666666 MSE:  0.2708333333333333 UAR:  0.6617647058823529 Recall:  0.5 Precision:  0.5384615384615384 F1:  0.5185185185185186
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attenti

##### Self attention

In [52]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_traditional


In [53]:
CONCATENATE_STAT = True

In [54]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2560) (5449,)
(2284, 128, 2560) (2284,)


In [55]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention (Attention)       (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [56]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 8s 155ms/step - loss: 1.5761 - acc: 0.7234 - auc: 0.7081 - binary_accuracy: 0.7234 - recall_1: 0.5414 - precision_1: 0.5153 - val_loss: 1.2094 - val_acc: 0.6493 - val_auc: 0.6822 - val_binary_accuracy: 0.6493 - val_recall_1: 0.1108 - val_precision_1: 0.4579
Epoch 2/20
43/43 [==============================] - 4s 94ms/step - loss: 0.2213 - acc: 0.9189 - auc: 0.9613 - binary_accuracy: 0.9189 - recall_1: 0.8182 - precision_1: 0.8890 - val_loss: 0.8240 - val_acc: 0.6528 - val_auc: 0.6992 - val_binary_accuracy: 0.6528 - val_recall_1: 0.3261 - val_precision_1: 0.4923
Epoch 3/20
43/43 [==============================] - 4s 93ms/step - loss: 0.1190 - acc: 0.9660 - auc: 0.9940 - binary_accuracy: 0.9660 - recall_1: 0.9249 - precision_1: 0.9549 - val_loss: 0.9960 - val_acc: 0.6909 - val_auc: 0.6846 - val_binary_accuracy: 0.6909 - val_recall_1: 0.2586 - val_precision_1: 0.6208
Epoch 4/20
43/43 [==============================] - 4s 92ms/step - loss:

In [57]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4027200/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [58]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_traditional Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6176470588235294 Recall:  0.5 Precision:  0.4375 F1:  0.4666666666666667
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_traditional Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.5966386554621849 Recall:  0.42857142857142855 Precision:  0.42857142857142855 F1:  0.42857142857142855
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_traditional Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6260504201680672 Recall:  0.42857142857142855 Precision:  0.5 F1:  0.4615384615384615
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_traditional Accuracy:  0.75 MSE:  0.25 UAR:  0.6554621848739496 Recall:  0.42857142857142855 Precision:  0.6 F1:  0.5
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_traditional Accuracy:  0.75 MSE:  0.25 UAR:  0.655

In [59]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [60]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4027200/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [61]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_traditional_best Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.6785714285714286 Recall:  0.8571428571428571 Precision:  0.41379310344827586 F1:  0.5581395348837208
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_traditional_best Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6533613445378151 Recall:  0.5714285714285714 Precision:  0.47058823529411764 F1:  0.5161290322580646
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_traditional_best Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6470588235294117 Recall:  0.5 Precision:  0.5 F1:  0.5
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_traditional_best Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6260504201680672 Recall:  0.42857142857142855 Precision:  0.5 F1:  0.4615384615384615
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_traditional_best

#### Balanced split

In [62]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [63]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [64]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


In [65]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.759)

(147,) (147,)
(48,) (48,)
(195,) (195,)
[0 1] [141  54]
195
147
48


In [66]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


##### Single attention

In [67]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_balanced


In [68]:
CONCATENATE_STAT = True

In [69]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5631, 128, 2560) (5631,)
(2102, 128, 2560) (2102,)


In [70]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [71]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
44/44 [==============================] - 4s 74ms/step - loss: 0.4721 - acc: 0.7949 - auc: 0.8205 - binary_accuracy: 0.7949 - recall_2: 0.5489 - precision_2: 0.7088 - val_loss: 0.6953 - val_acc: 0.5951 - val_auc: 0.7154 - val_binary_accuracy: 0.5951 - val_recall_2: 0.3984 - val_precision_2: 0.3504
Epoch 2/20
44/44 [==============================] - 1s 32ms/step - loss: 0.1780 - acc: 0.9469 - auc: 0.9814 - binary_accuracy: 0.9469 - recall_2: 0.8541 - precision_2: 0.9668 - val_loss: 0.9529 - val_acc: 0.6066 - val_auc: 0.7030 - val_binary_accuracy: 0.6066 - val_recall_2: 0.4787 - val_precision_2: 0.3800
Epoch 3/20
44/44 [==============================] - 1s 32ms/step - loss: 0.0862 - acc: 0.9798 - auc: 0.9976 - binary_accuracy: 0.9798 - recall_2: 0.9432 - precision_2: 0.9896 - val_loss: 1.1213 - val_acc: 0.6560 - val_auc: 0.6707 - val_binary_accuracy: 0.6560 - val_recall_2: 0.3984 - val_precision_2: 0.4259
Epoch 4/20
44/44 [==============================] - 1s 32ms/step - loss: 

In [72]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4027200/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [73]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_balanced Accuracy:  0.625 MSE:  0.375 UAR:  0.5672268907563025 Recall:  0.42857142857142855 Precision:  0.375 F1:  0.39999999999999997
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_balanced Accuracy:  0.625 MSE:  0.375 UAR:  0.5672268907563025 Recall:  0.42857142857142855 Precision:  0.375 F1:  0.39999999999999997
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_balanced Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.5966386554621849 Recall:  0.42857142857142855 Precision:  0.42857142857142855 F1:  0.42857142857142855
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_balanced Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.5756302521008403 Recall:  0.35714285714285715 Precision:  0.4166666666666667 F1:  0.3846153846153846
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_balanced Accuracy:  0.66666

In [74]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [75]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4027200/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [76]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_balanced_best Accuracy:  0.5416666666666666 MSE:  0.4583333333333333 UAR:  0.634453781512605 Recall:  0.8571428571428571 Precision:  0.375 F1:  0.5217391304347825
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_balanced_best Accuracy:  0.625 MSE:  0.375 UAR:  0.6302521008403361 Recall:  0.6428571428571429 Precision:  0.4090909090909091 F1:  0.5000000000000001
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_balanced_best Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6596638655462186 Recall:  0.6428571428571429 Precision:  0.45 F1:  0.5294117647058824
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_balanced_best Accuracy:  0.625 MSE:  0.375 UAR:  0.546218487394958 Recall:  0.35714285714285715 Precision:  0.35714285714285715 F1:  0.35714285714285715
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_single_attention_balanced_best A

##### Self attention

In [77]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_balanced


In [78]:
CONCATENATE_STAT = True

In [79]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5631, 128, 2560) (5631,)
(2102, 128, 2560) (2102,)


In [80]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_1 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [81]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
44/44 [==============================] - 8s 168ms/step - loss: 1.2116 - acc: 0.7714 - auc: 0.7812 - binary_accuracy: 0.7714 - recall_3: 0.6333 - precision_3: 0.6206 - val_loss: 1.0200 - val_acc: 0.6042 - val_auc: 0.6351 - val_binary_accuracy: 0.6042 - val_recall_3: 0.5496 - val_precision_3: 0.3899
Epoch 2/20
44/44 [==============================] - 4s 93ms/step - loss: 0.1467 - acc: 0.9515 - auc: 0.9866 - binary_accuracy: 0.9515 - recall_3: 0.8928 - precision_3: 0.9442 - val_loss: 0.9340 - val_acc: 0.6275 - val_auc: 0.6549 - val_binary_accuracy: 0.6275 - val_recall_3: 0.4173 - val_precision_3: 0.3909
Epoch 3/20
44/44 [==============================] - 4s 91ms/step - loss: 0.0680 - acc: 0.9876 - auc: 0.9990 - binary_accuracy: 0.9876 - recall_3: 0.9690 - precision_3: 0.9898 - val_loss: 1.1035 - val_acc: 0.6213 - val_auc: 0.6565 - val_binary_accuracy: 0.6213 - val_recall_3: 0.3732 - val_precision_3: 0.3732
Epoch 4/20
44/44 [==============================] - 5s 103ms/step - loss

In [82]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4027200/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [83]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_balanced Accuracy:  0.625 MSE:  0.375 UAR:  0.5882352941176471 Recall:  0.5 Precision:  0.3888888888888889 F1:  0.43750000000000006
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_balanced Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.6029411764705883 Recall:  0.5 Precision:  0.4117647058823529 F1:  0.45161290322580644
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_balanced Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.5966386554621849 Recall:  0.42857142857142855 Precision:  0.42857142857142855 F1:  0.42857142857142855
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_balanced Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6260504201680672 Recall:  0.42857142857142855 Precision:  0.5 F1:  0.4615384615384615
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_balanced Accuracy:  0.6875 MSE: 

In [84]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [85]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4027200/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [86]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_balanced_best Accuracy:  0.5625 MSE:  0.4375 UAR:  0.5861344537815126 Recall:  0.6428571428571429 Precision:  0.36 F1:  0.4615384615384615
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_balanced_best Accuracy:  0.625 MSE:  0.375 UAR:  0.6302521008403361 Recall:  0.6428571428571429 Precision:  0.4090909090909091 F1:  0.5000000000000001
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_balanced_best Accuracy:  0.625 MSE:  0.375 UAR:  0.6092436974789917 Recall:  0.5714285714285714 Precision:  0.4 F1:  0.47058823529411764
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_balanced_best Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6386554621848739 Recall:  0.5714285714285714 Precision:  0.4444444444444444 F1:  0.5
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_max_self_attention_balanced_best Accuracy:  0.6458333333333334 MSE:  0.35416666666666

### STAT

In [28]:
stat_func = compute_descriptor
stat_name = "STAT"

#### Traditional split

In [29]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [30]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [31]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


##### Single attention

In [31]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_traditional


In [32]:
CONCATENATE_STAT = True

In [33]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 3840) (5449,)
(2284, 128, 3840) (2284,)


In [34]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-07-30 07:24:49.986587: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3840)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3841      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                              

In [35]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20


2024-07-30 07:25:03.152418: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f2ec8232af0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-30 07:25:03.152594: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-07-30 07:25:03.156757: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-30 07:25:03.198857: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-07-30 07:25:03.250678: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


43/43 [==============================] - 10s 159ms/step - loss: 0.4052 - acc: 0.8391 - auc: 0.8601 - binary_accuracy: 0.8391 - recall: 0.5382 - precision: 0.8414 - val_loss: 0.7601 - val_acc: 0.7084 - val_auc: 0.6645 - val_binary_accuracy: 0.7084 - val_recall: 0.2459 - val_precision: 0.7228
Epoch 2/20
43/43 [==============================] - 2s 47ms/step - loss: 0.1796 - acc: 0.9352 - auc: 0.9801 - binary_accuracy: 0.9352 - recall: 0.8343 - precision: 0.9319 - val_loss: 0.9309 - val_acc: 0.6537 - val_auc: 0.6570 - val_binary_accuracy: 0.6537 - val_recall: 0.5070 - val_precision: 0.4963
Epoch 3/20
43/43 [==============================] - 2s 46ms/step - loss: 0.1088 - acc: 0.9635 - auc: 0.9935 - binary_accuracy: 0.9635 - recall: 0.9242 - precision: 0.9467 - val_loss: 1.0701 - val_acc: 0.6821 - val_auc: 0.6794 - val_binary_accuracy: 0.6821 - val_recall: 0.4013 - val_precision: 0.5517
Epoch 4/20
43/43 [==============================] - 2s 47ms/step - loss: 0.0668 - acc: 0.9846 - auc: 0.998

In [36]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4038218/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [37]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_traditional Accuracy:  0.5208333333333334 MSE:  0.4791666666666667 UAR:  0.5357142857142857 Recall:  0.5714285714285714 Precision:  0.32 F1:  0.41025641025641024
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_traditional Accuracy:  0.5416666666666666 MSE:  0.4583333333333333 UAR:  0.5504201680672269 Recall:  0.5714285714285714 Precision:  0.3333333333333333 F1:  0.4210526315789474
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_traditional Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.5945378151260504 Recall:  0.5714285714285714 Precision:  0.38095238095238093 F1:  0.4571428571428571
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_traditional Accuracy:  0.5833333333333334 MSE:  0.4166666666666667 UAR:  0.5588235294117647 Recall:  0.5 Precision:  0.35 F1:  0.4117647058823529
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STA

In [38]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [39]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4038218/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [40]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_traditional_best Accuracy:  0.5625 MSE:  0.4375 UAR:  0.5651260504201681 Recall:  0.5714285714285714 Precision:  0.34782608695652173 F1:  0.4324324324324324
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_traditional_best Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6176470588235294 Recall:  0.5 Precision:  0.4375 F1:  0.4666666666666667
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_traditional_best Accuracy:  0.6875 MSE:  0.3125 UAR:  0.5903361344537815 Recall:  0.35714285714285715 Precision:  0.45454545454545453 F1:  0.4
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_traditional_best Accuracy:  0.75 MSE:  0.25 UAR:  0.6134453781512605 Recall:  0.2857142857142857 Precision:  0.6666666666666666 F1:  0.4
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_traditional_best Accuracy:  0.7291666666666666 MSE

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### Self attention

In [41]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_traditional


In [42]:
CONCATENATE_STAT = True

In [43]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 3840) (5449,)
(2284, 128, 3840) (2284,)


In [44]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3840)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 3840)           1474560   ['image_set[0][0]']           
                                                          0                                       
                                                                                                  
 value (Dense)               (None, None, 3840)           1474560   ['image_set[0][0]']           
                                                          0                                       
                                                                                            

In [45]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 13s 267ms/step - loss: 0.8178 - acc: 0.7937 - auc: 0.7978 - binary_accuracy: 0.7937 - recall_1: 0.6275 - precision_1: 0.6423 - val_loss: 0.9124 - val_acc: 0.7469 - val_auc: 0.6501 - val_binary_accuracy: 0.7469 - val_recall_1: 0.4089 - val_precision_1: 0.7379
Epoch 2/20
43/43 [==============================] - 9s 204ms/step - loss: 0.1831 - acc: 0.9378 - auc: 0.9740 - binary_accuracy: 0.9378 - recall_1: 0.8401 - precision_1: 0.9356 - val_loss: 0.9893 - val_acc: 0.6865 - val_auc: 0.6679 - val_binary_accuracy: 0.6865 - val_recall_1: 0.4255 - val_precision_1: 0.5576
Epoch 3/20
43/43 [==============================] - 8s 178ms/step - loss: 0.1127 - acc: 0.9642 - auc: 0.9914 - binary_accuracy: 0.9642 - recall_1: 0.9274 - precision_1: 0.9463 - val_loss: 1.2325 - val_acc: 0.6773 - val_auc: 0.6565 - val_binary_accuracy: 0.6773 - val_recall_1: 0.4408 - val_precision_1: 0.5373
Epoch 4/20
43/43 [==============================] - 8s 179ms/step - l

In [46]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4038218/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [47]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_traditional Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.6239495798319328 Recall:  0.5714285714285714 Precision:  0.42105263157894735 F1:  0.48484848484848486
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_traditional Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.6029411764705883 Recall:  0.5 Precision:  0.4117647058823529 F1:  0.45161290322580644
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_traditional Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.5315126050420168 Recall:  0.35714285714285715 Precision:  0.3333333333333333 F1:  0.3448275862068965
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_traditional Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.5315126050420168 Recall:  0.35714285714285715 Precision:  0.3333333333333333 F1:  0.3448275862068965
Metric_name:  enet_b0_8_best_a

In [48]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [49]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4038218/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [50]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_traditional_best Accuracy:  0.5833333333333334 MSE:  0.4166666666666667 UAR:  0.6008403361344539 Recall:  0.6428571428571429 Precision:  0.375 F1:  0.4736842105263159
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_traditional_best Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.5945378151260504 Recall:  0.5714285714285714 Precision:  0.38095238095238093 F1:  0.4571428571428571
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_traditional_best Accuracy:  0.625 MSE:  0.375 UAR:  0.6092436974789917 Recall:  0.5714285714285714 Precision:  0.4 F1:  0.47058823529411764
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_traditional_best Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.6029411764705883 Recall:  0.5 Precision:  0.4117647058823529 F1:  0.45161290322580644
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attent

#### Balanced split

In [32]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [33]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [34]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


In [35]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.759)

(147, 2560) (147,)
(48, 2560) (48,)
(195, 2560) (195,)
[0 1] [141  54]
195
147
48


In [36]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


##### Single attention

In [37]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_balanced


In [38]:
CONCATENATE_STAT = True

In [39]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5631, 128, 3840) (5631,)
(2102, 128, 3840) (2102,)


In [40]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-07-30 07:39:08.167930: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3840)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3841      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                              

In [41]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

In [42]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [43]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_balanced Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6533613445378151 Recall:  0.5714285714285714 Precision:  0.47058823529411764 F1:  0.5161290322580646
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_balanced Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6680672268907563 Recall:  0.5714285714285714 Precision:  0.5 F1:  0.5333333333333333
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_balanced Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6323529411764706 Recall:  0.5 Precision:  0.4666666666666667 F1:  0.4827586206896552
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_balanced Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.5966386554621849 Recall:  0.42857142857142855 Precision:  0.42857142857142855 F1:  0.42857142857142855
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_balanced Accuracy:  0.6666

In [44]:
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    modelAtn.set_weights(best_model_weights)
    save_weights(modelAtn, weights_name)

In [45]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [46]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_balanced_best Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.6785714285714286 Recall:  0.8571428571428571 Precision:  0.41379310344827586 F1:  0.5581395348837208
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_balanced_best Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.7016806722689075 Recall:  0.7857142857142857 Precision:  0.4583333333333333 F1:  0.5789473684210527
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_balanced_best Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.6449579831932774 Recall:  0.6428571428571429 Precision:  0.42857142857142855 F1:  0.5142857142857143
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_single_attention_balanced_best Accuracy:  0.7291666666666666 MSE:  0.2708333333333333 UAR:  0.6827731092436975 Recall:  0.5714285714285714 Precision:  0.5333333333333333 F1:  0.5517241379310344
Me

##### Self attention

In [47]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_balanced


In [48]:
CONCATENATE_STAT = True

In [49]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5631, 128, 3840) (5631,)
(2102, 128, 3840) (2102,)


In [50]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3840)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 3840)           1474560   ['image_set[0][0]']           
                                                          0                                       
                                                                                                  
 value (Dense)               (None, None, 3840)           1474560   ['image_set[0][0]']           
                                                          0                                       
                                                                                            

In [51]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20


2024-07-30 07:39:49.605252: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x25ee422a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-30 07:39:49.605415: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-07-30 07:39:49.609556: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-30 07:39:49.656270: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-07-30 07:39:49.708038: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


44/44 [==============================] - 16s 319ms/step - loss: 0.5446 - acc: 0.8439 - auc: 0.8782 - binary_accuracy: 0.8439 - recall_1: 0.7293 - precision_1: 0.7491 - val_loss: 1.0620 - val_acc: 0.5718 - val_auc: 0.6492 - val_binary_accuracy: 0.5718 - val_recall_1: 0.5780 - val_precision_1: 0.3674
Epoch 2/20
44/44 [==============================] - 8s 178ms/step - loss: 0.1441 - acc: 0.9499 - auc: 0.9835 - binary_accuracy: 0.9499 - recall_1: 0.8910 - precision_1: 0.9406 - val_loss: 0.8972 - val_acc: 0.7350 - val_auc: 0.7195 - val_binary_accuracy: 0.7350 - val_recall_1: 0.4819 - val_precision_1: 0.5730
Epoch 3/20
44/44 [==============================] - 8s 179ms/step - loss: 0.0828 - acc: 0.9705 - auc: 0.9953 - binary_accuracy: 0.9705 - recall_1: 0.9397 - precision_1: 0.9622 - val_loss: 1.0392 - val_acc: 0.6946 - val_auc: 0.7199 - val_binary_accuracy: 0.6946 - val_recall_1: 0.5480 - val_precision_1: 0.4950
Epoch 4/20
44/44 [==============================] - 8s 177ms/step - loss: 0.0497

In [52]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [53]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_balanced Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6596638655462186 Recall:  0.6428571428571429 Precision:  0.45 F1:  0.5294117647058824
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_balanced Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6386554621848739 Recall:  0.5714285714285714 Precision:  0.4444444444444444 F1:  0.5
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_balanced Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6533613445378151 Recall:  0.5714285714285714 Precision:  0.47058823529411764 F1:  0.5161290322580646
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_balanced Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6533613445378151 Recall:  0.5714285714285714 Precision:  0.47058823529411764 F1:  0.5161290322580646
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_balanced Accuracy:  0.6875 MSE:  0.31

In [54]:
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    modelAtn.set_weights(best_model_weights)
    save_weights(modelAtn, weights_name)

In [55]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [56]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_balanced_best Accuracy:  0.625 MSE:  0.375 UAR:  0.6302521008403361 Recall:  0.6428571428571429 Precision:  0.4090909090909091 F1:  0.5000000000000001
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_balanced_best Accuracy:  0.625 MSE:  0.375 UAR:  0.6092436974789917 Recall:  0.5714285714285714 Precision:  0.4 F1:  0.47058823529411764
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_balanced_best Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6386554621848739 Recall:  0.5714285714285714 Precision:  0.4444444444444444 F1:  0.5
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_balanced_best Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6680672268907563 Recall:  0.5714285714285714 Precision:  0.5 F1:  0.5333333333333333
Metric_name:  enet_b0_8_best_afew.pt_EngageWild_2_STAT_self_attention_balanced_best Accuracy:  0.6875 MSE:  

## MobileNet_7

In [25]:
base_model_key = 'mobilenet_7.h5'

In [26]:
ENGAGE_WILD_DATASET, BIN_CLASSIFICATION, N_CLASSES, DATASET_NAME, BASE_DATASET_DIR, ext = define_dataset(True, True)
video2label, labels_list = get_video2label()
print(labels_list)
print(video2label)

['engaged', 'distracted']
{'subject_1_Vid_1': 0, 'subject_1_Vid_2': 0, 'subject_1_Vid_3': 0, 'subject_1_Vid_4': 0, 'subject_1_Vid_5': 0, 'subject_31_Vid_6': 0, 'subject_2_Vid_6': 1, 'subject_3_Vid_6': 0, 'subject_3_Vid_1': 1, 'subject_3_Vid_2': 1, 'subject_3_Vid_3': 0, 'subject_3_Vid_4': 0, 'subject_3_Vid_5': 1, 'subject_3_Vid_7': 0, 'subject_4_Vid_6': 0, 'subject_5_Vid_6': 0, 'subject_6_Vid_6': 1, 'subject_7_Vid_1': 0, 'subject_7_Vid_2': 0, 'subject_7_Vid_3': 0, 'subject_7_Vid_4': 0, 'subject_7_Vid_5': 1, 'subject_8_Vid_6': 0, 'subject_9_Vid_6': 1, 'subject_10_Vid_6': 0, 'subject_11_Vid_6': 0, 'subject_12_Vid_6': 1, 'subject_13_Vid_6': 0, 'subject_14_Vid_6': 0, 'subject_15_Vid_6': 0, 'subject_16_Vid_6': 1, 'subject_17_Vid_6': 0, 'subject_18_Vid_6': 1, 'subject_19_Vid_6': 0, 'subject_20_Vid_6': 0, 'subject_20_Vid_1': 0, 'subject_20_Vid_2': 0, 'subject_20_Vid_3': 0, 'subject_20_Vid_4': 1, 'subject_20_Vid_5': 0, 'subject_20_Vid_5_1': 0, 'subject_20_Vid_5_2': 0, 'subject_20_Vid_7': 1, 'su

### Std

In [59]:
stat_func = np.std
stat_name = "std"

#### Traditional split

In [60]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [61]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [62]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


##### Single attention

In [63]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

mobilenet_7.h5_EngageWild_2_std_single_attention_traditional


In [64]:
CONCATENATE_STAT = True

In [65]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2048) (5449,)
(2284, 128, 2048) (2284,)


In [66]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [67]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 5s 88ms/step - loss: 0.4456 - acc: 0.8064 - auc: 0.8214 - binary_accuracy: 0.8064 - recall_2: 0.4817 - precision_2: 0.7515 - val_loss: 0.6893 - val_acc: 0.6475 - val_auc: 0.6929 - val_binary_accuracy: 0.6475 - val_recall_2: 0.2701 - val_precision_2: 0.4775
Epoch 2/20
43/43 [==============================] - 1s 28ms/step - loss: 0.2183 - acc: 0.9169 - auc: 0.9671 - binary_accuracy: 0.9169 - recall_2: 0.8112 - precision_2: 0.8882 - val_loss: 0.8184 - val_acc: 0.6690 - val_auc: 0.6946 - val_binary_accuracy: 0.6690 - val_recall_2: 0.1427 - val_precision_2: 0.5744
Epoch 3/20
43/43 [==============================] - 1s 27ms/step - loss: 0.1261 - acc: 0.9552 - auc: 0.9915 - binary_accuracy: 0.9552 - recall_2: 0.8844 - precision_2: 0.9556 - val_loss: 0.7918 - val_acc: 0.6957 - val_auc: 0.7102 - val_binary_accuracy: 0.6957 - val_recall_2: 0.3376 - val_precision_2: 0.6023
Epoch 4/20
43/43 [==============================] - 1s 28ms/step - loss: 

In [68]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [69]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_traditional Accuracy:  0.5416666666666666 MSE:  0.4583333333333333 UAR:  0.5084033613445378 Recall:  0.42857142857142855 Precision:  0.3 F1:  0.3529411764705882
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_traditional Accuracy:  0.625 MSE:  0.375 UAR:  0.5672268907563025 Recall:  0.42857142857142855 Precision:  0.375 F1:  0.39999999999999997
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_traditional Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.5315126050420168 Recall:  0.35714285714285715 Precision:  0.3333333333333333 F1:  0.3448275862068965
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_traditional Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.5315126050420168 Recall:  0.35714285714285715 Precision:  0.3333333333333333 F1:  0.3448275862068965
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_traditional Accuracy:  0.604166666

In [70]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [71]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [72]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_traditional_best Accuracy:  0.4791666666666667 MSE:  0.5208333333333334 UAR:  0.5903361344537815 Recall:  0.8571428571428571 Precision:  0.34285714285714286 F1:  0.4897959183673469
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_traditional_best Accuracy:  0.7291666666666666 MSE:  0.2708333333333333 UAR:  0.7247899159663866 Recall:  0.7142857142857143 Precision:  0.5263157894736842 F1:  0.6060606060606061
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_traditional_best Accuracy:  0.75 MSE:  0.25 UAR:  0.6554621848739496 Recall:  0.42857142857142855 Precision:  0.6 F1:  0.5
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_traditional_best Accuracy:  0.7291666666666666 MSE:  0.2708333333333333 UAR:  0.5987394957983193 Recall:  0.2857142857142857 Precision:  0.5714285714285714 F1:  0.38095238095238093
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_traditional_best Acc

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### Self attention

In [73]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

mobilenet_7.h5_EngageWild_2_std_self_attention_traditional


In [74]:
CONCATENATE_STAT = True

In [75]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2048) (5449,)
(2284, 128, 2048) (2284,)


In [76]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 attention_1 (Attention)     (None, None, 2048)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [77]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 7s 137ms/step - loss: 0.5493 - acc: 0.7932 - auc: 0.7957 - binary_accuracy: 0.7932 - recall_3: 0.5549 - precision_3: 0.6656 - val_loss: 0.6722 - val_acc: 0.6848 - val_auc: 0.6613 - val_binary_accuracy: 0.6848 - val_recall_3: 0.4255 - val_precision_3: 0.5539
Epoch 2/20
43/43 [==============================] - 3s 66ms/step - loss: 0.2095 - acc: 0.9286 - auc: 0.9708 - binary_accuracy: 0.9286 - recall_3: 0.8420 - precision_3: 0.9017 - val_loss: 0.7188 - val_acc: 0.7097 - val_auc: 0.6857 - val_binary_accuracy: 0.7097 - val_recall_3: 0.3465 - val_precision_3: 0.6445
Epoch 3/20
43/43 [==============================] - 3s 67ms/step - loss: 0.1524 - acc: 0.9435 - auc: 0.9828 - binary_accuracy: 0.9435 - recall_3: 0.8741 - precision_3: 0.9240 - val_loss: 0.7936 - val_acc: 0.6515 - val_auc: 0.6645 - val_binary_accuracy: 0.6515 - val_recall_3: 0.4229 - val_precision_3: 0.4919
Epoch 4/20
43/43 [==============================] - 3s 67ms/step - loss:

In [78]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [79]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_traditional Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.6365546218487395 Recall:  0.7142857142857143 Precision:  0.4 F1:  0.5128205128205129
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_traditional Accuracy:  0.625 MSE:  0.375 UAR:  0.6512605042016807 Recall:  0.7142857142857143 Precision:  0.4166666666666667 F1:  0.5263157894736842
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_traditional Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6596638655462186 Recall:  0.6428571428571429 Precision:  0.45 F1:  0.5294117647058824
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_traditional Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.6239495798319328 Recall:  0.5714285714285714 Precision:  0.42105263157894735 F1:  0.48484848484848486
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_traditional Accuracy:  0.6458333333333334 MSE:  

In [80]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [81]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [82]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_traditional_best Accuracy:  0.375 MSE:  0.625 UAR:  0.5378151260504201 Recall:  0.9285714285714286 Precision:  0.30952380952380953 F1:  0.46428571428571436
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_traditional_best Accuracy:  0.5 MSE:  0.5 UAR:  0.6260504201680672 Recall:  0.9285714285714286 Precision:  0.3611111111111111 F1:  0.52
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_traditional_best Accuracy:  0.5416666666666666 MSE:  0.4583333333333333 UAR:  0.6554621848739496 Recall:  0.9285714285714286 Precision:  0.38235294117647056 F1:  0.5416666666666667
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_traditional_best Accuracy:  0.6875 MSE:  0.3125 UAR:  0.7584033613445378 Recall:  0.9285714285714286 Precision:  0.48148148148148145 F1:  0.6341463414634146
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_traditional_best Accuracy:  0.7083333333333334 MSE:  0.2916666666

#### Balanced split

In [83]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [84]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [85]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


In [86]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.759)

(147,) (147,)
(48,) (48,)
(195,) (195,)
[0 1] [141  54]
195
147
48


In [87]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


##### Single attention

In [88]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

mobilenet_7.h5_EngageWild_2_std_single_attention_balanced


In [89]:
CONCATENATE_STAT = True

In [90]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5631, 128, 2048) (5631,)
(2102, 128, 2048) (2102,)


In [91]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [92]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
44/44 [==============================] - 4s 61ms/step - loss: 0.3941 - acc: 0.8283 - auc: 0.8757 - binary_accuracy: 0.8283 - recall_4: 0.5595 - precision_4: 0.8162 - val_loss: 0.6503 - val_acc: 0.7174 - val_auc: 0.6976 - val_binary_accuracy: 0.7174 - val_recall_4: 0.5465 - val_precision_4: 0.5314
Epoch 2/20
44/44 [==============================] - 1s 27ms/step - loss: 0.1789 - acc: 0.9357 - auc: 0.9808 - binary_accuracy: 0.9357 - recall_4: 0.8535 - precision_4: 0.9286 - val_loss: 0.8561 - val_acc: 0.6422 - val_auc: 0.6693 - val_binary_accuracy: 0.6422 - val_recall_4: 0.5449 - val_precision_4: 0.4277
Epoch 3/20
44/44 [==============================] - 1s 26ms/step - loss: 0.0990 - acc: 0.9663 - auc: 0.9958 - binary_accuracy: 0.9663 - recall_4: 0.9080 - precision_4: 0.9792 - val_loss: 0.9587 - val_acc: 0.6584 - val_auc: 0.6736 - val_binary_accuracy: 0.6584 - val_recall_4: 0.4945 - val_precision_4: 0.4416
Epoch 4/20
44/44 [==============================] - 1s 28ms/step - loss: 

In [93]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [94]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_balanced Accuracy:  0.75 MSE:  0.25 UAR:  0.6764705882352942 Recall:  0.5 Precision:  0.5833333333333334 F1:  0.5384615384615384
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_balanced Accuracy:  0.7291666666666666 MSE:  0.2708333333333333 UAR:  0.6197478991596639 Recall:  0.35714285714285715 Precision:  0.5555555555555556 F1:  0.43478260869565216
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_balanced Accuracy:  0.75 MSE:  0.25 UAR:  0.634453781512605 Recall:  0.35714285714285715 Precision:  0.625 F1:  0.45454545454545453
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_balanced Accuracy:  0.7291666666666666 MSE:  0.2708333333333333 UAR:  0.5987394957983193 Recall:  0.2857142857142857 Precision:  0.5714285714285714 F1:  0.38095238095238093
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_balanced Accuracy:  0.7291666666666666 MSE:  0.2708333333333333 UAR:  0.59873

In [95]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [96]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [97]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_balanced_best Accuracy:  0.5 MSE:  0.5 UAR:  0.6260504201680672 Recall:  0.9285714285714286 Precision:  0.3611111111111111 F1:  0.52
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_balanced_best Accuracy:  0.5833333333333334 MSE:  0.4166666666666667 UAR:  0.6428571428571428 Recall:  0.7857142857142857 Precision:  0.39285714285714285 F1:  0.5238095238095237
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_balanced_best Accuracy:  0.5833333333333334 MSE:  0.4166666666666667 UAR:  0.6008403361344539 Recall:  0.6428571428571429 Precision:  0.375 F1:  0.4736842105263159
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_balanced_best Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6890756302521008 Recall:  0.6428571428571429 Precision:  0.5 F1:  0.5625000000000001
Metric_name:  mobilenet_7.h5_EngageWild_2_std_single_attention_balanced_best Accuracy:  0.75 MSE:  0.25 UAR:  0

##### Self attention

In [98]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

mobilenet_7.h5_EngageWild_2_std_self_attention_balanced


In [99]:
CONCATENATE_STAT = True

In [100]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5631, 128, 2048) (5631,)
(2102, 128, 2048) (2102,)


In [101]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 attention_2 (Attention)     (None, None, 2048)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [102]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
44/44 [==============================] - 6s 111ms/step - loss: 0.5453 - acc: 0.7906 - auc: 0.8068 - binary_accuracy: 0.7906 - recall_5: 0.5946 - precision_5: 0.6758 - val_loss: 0.6830 - val_acc: 0.6603 - val_auc: 0.6268 - val_binary_accuracy: 0.6603 - val_recall_5: 0.4031 - val_precision_5: 0.4332
Epoch 2/20
44/44 [==============================] - 3s 66ms/step - loss: 0.1889 - acc: 0.9352 - auc: 0.9804 - binary_accuracy: 0.9352 - recall_5: 0.8436 - precision_5: 0.9363 - val_loss: 0.7894 - val_acc: 0.6456 - val_auc: 0.6385 - val_binary_accuracy: 0.6456 - val_recall_5: 0.4331 - val_precision_5: 0.4167
Epoch 3/20
44/44 [==============================] - 3s 66ms/step - loss: 0.1139 - acc: 0.9639 - auc: 0.9934 - binary_accuracy: 0.9639 - recall_5: 0.9203 - precision_5: 0.9591 - val_loss: 0.8956 - val_acc: 0.6618 - val_auc: 0.6308 - val_binary_accuracy: 0.6618 - val_recall_5: 0.3701 - val_precision_5: 0.4304
Epoch 4/20
44/44 [==============================] - 3s 66ms/step - loss:

In [103]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [104]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_balanced Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.6449579831932774 Recall:  0.6428571428571429 Precision:  0.42857142857142855 F1:  0.5142857142857143
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_balanced Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.6029411764705883 Recall:  0.5 Precision:  0.4117647058823529 F1:  0.45161290322580644
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_balanced Accuracy:  0.625 MSE:  0.375 UAR:  0.5672268907563025 Recall:  0.42857142857142855 Precision:  0.375 F1:  0.39999999999999997
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_balanced Accuracy:  0.625 MSE:  0.375 UAR:  0.5672268907563025 Recall:  0.42857142857142855 Precision:  0.375 F1:  0.39999999999999997
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_balanced Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.5819327731092437 Recal

In [105]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [106]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [107]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_balanced_best Accuracy:  0.375 MSE:  0.625 UAR:  0.5378151260504201 Recall:  0.9285714285714286 Precision:  0.30952380952380953 F1:  0.46428571428571436
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_balanced_best Accuracy:  0.5416666666666666 MSE:  0.4583333333333333 UAR:  0.6554621848739496 Recall:  0.9285714285714286 Precision:  0.38235294117647056 F1:  0.5416666666666667
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_balanced_best Accuracy:  0.5625 MSE:  0.4375 UAR:  0.6281512605042017 Recall:  0.7857142857142857 Precision:  0.3793103448275862 F1:  0.5116279069767441
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_balanced_best Accuracy:  0.5625 MSE:  0.4375 UAR:  0.5651260504201681 Recall:  0.5714285714285714 Precision:  0.34782608695652173 F1:  0.4324324324324324
Metric_name:  mobilenet_7.h5_EngageWild_2_std_self_attention_balanced_best Accuracy:  0.5833333333333334 MSE:  0.41666

### Max

In [108]:
stat_func = np.max
stat_name = "max"

#### Traditional split

In [109]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [110]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [111]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


##### Single attention

In [112]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

mobilenet_7.h5_EngageWild_2_max_single_attention_traditional


In [113]:
CONCATENATE_STAT = True

In [114]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2048) (5449,)
(2284, 128, 2048) (2284,)


In [115]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [116]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 4s 66ms/step - loss: 0.6302 - acc: 0.7288 - auc: 0.6793 - binary_accuracy: 0.7288 - recall_6: 0.3455 - precision_6: 0.5396 - val_loss: 0.6465 - val_acc: 0.5898 - val_auc: 0.6219 - val_binary_accuracy: 0.5898 - val_recall_6: 0.0994 - val_precision_6: 0.2532
Epoch 2/20
43/43 [==============================] - 1s 34ms/step - loss: 0.3227 - acc: 0.8873 - auc: 0.9345 - binary_accuracy: 0.8873 - recall_6: 0.6808 - precision_6: 0.9006 - val_loss: 0.7450 - val_acc: 0.6116 - val_auc: 0.6218 - val_binary_accuracy: 0.6116 - val_recall_6: 0.1465 - val_precision_6: 0.3464
Epoch 3/20
43/43 [==============================] - 1s 34ms/step - loss: 0.2100 - acc: 0.9312 - auc: 0.9718 - binary_accuracy: 0.9312 - recall_6: 0.8247 - precision_6: 0.9264 - val_loss: 0.8559 - val_acc: 0.6620 - val_auc: 0.5774 - val_binary_accuracy: 0.6620 - val_recall_6: 0.1631 - val_precision_6: 0.5267
Epoch 4/20
43/43 [==============================] - 1s 34ms/step - loss: 

In [117]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [118]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_traditional Accuracy:  0.5625 MSE:  0.4375 UAR:  0.5441176470588236 Recall:  0.5 Precision:  0.3333333333333333 F1:  0.4
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_traditional Accuracy:  0.5833333333333334 MSE:  0.4166666666666667 UAR:  0.5378151260504201 Recall:  0.42857142857142855 Precision:  0.3333333333333333 F1:  0.375
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_traditional Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.5315126050420168 Recall:  0.35714285714285715 Precision:  0.3333333333333333 F1:  0.3448275862068965
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_traditional Accuracy:  0.625 MSE:  0.375 UAR:  0.546218487394958 Recall:  0.35714285714285715 Precision:  0.35714285714285715 F1:  0.35714285714285715
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_traditional Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0

In [119]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [120]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [121]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_traditional_best Accuracy:  0.3125 MSE:  0.6875 UAR:  0.5147058823529411 Recall:  1.0 Precision:  0.2978723404255319 F1:  0.45901639344262296
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_traditional_best Accuracy:  0.5208333333333334 MSE:  0.4791666666666667 UAR:  0.6407563025210085 Recall:  0.9285714285714286 Precision:  0.37142857142857144 F1:  0.5306122448979592
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_traditional_best Accuracy:  0.5625 MSE:  0.4375 UAR:  0.5441176470588236 Recall:  0.5 Precision:  0.3333333333333333 F1:  0.4
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_traditional_best Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.5336134453781513 Recall:  0.21428571428571427 Precision:  0.375 F1:  0.2727272727272727
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_traditional_best Accuracy:  0.6666666666666666 MSE:  0.3333333333333

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### Self attention

In [122]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

mobilenet_7.h5_EngageWild_2_max_self_attention_traditional


In [123]:
CONCATENATE_STAT = True

In [124]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 2048) (5449,)
(2284, 128, 2048) (2284,)


In [125]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 attention_3 (Attention)     (None, None, 2048)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [126]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 6s 110ms/step - loss: 3.1838 - acc: 0.6818 - auc: 0.6361 - binary_accuracy: 0.6818 - recall_7: 0.4483 - precision_7: 0.4437 - val_loss: 1.0201 - val_acc: 0.6502 - val_auc: 0.6093 - val_binary_accuracy: 0.6502 - val_recall_7: 0.1172 - val_precision_7: 0.4646
Epoch 2/20
43/43 [==============================] - 3s 66ms/step - loss: 0.3023 - acc: 0.8789 - auc: 0.9268 - binary_accuracy: 0.8789 - recall_7: 0.7405 - precision_7: 0.8183 - val_loss: 0.7808 - val_acc: 0.6467 - val_auc: 0.5626 - val_binary_accuracy: 0.6467 - val_recall_7: 0.3146 - val_precision_7: 0.4787
Epoch 3/20
43/43 [==============================] - 3s 66ms/step - loss: 0.1850 - acc: 0.9383 - auc: 0.9809 - binary_accuracy: 0.9383 - recall_7: 0.8504 - precision_7: 0.9278 - val_loss: 0.8676 - val_acc: 0.6454 - val_auc: 0.5438 - val_binary_accuracy: 0.6454 - val_recall_7: 0.1885 - val_precision_7: 0.4611
Epoch 4/20
43/43 [==============================] - 3s 66ms/step - loss:

In [127]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [128]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_traditional Accuracy:  0.5625 MSE:  0.4375 UAR:  0.5861344537815126 Recall:  0.6428571428571429 Precision:  0.36 F1:  0.4615384615384615
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_traditional Accuracy:  0.625 MSE:  0.375 UAR:  0.6302521008403361 Recall:  0.6428571428571429 Precision:  0.4090909090909091 F1:  0.5000000000000001
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_traditional Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.5525210084033614 Recall:  0.42857142857142855 Precision:  0.35294117647058826 F1:  0.3870967741935484
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_traditional Accuracy:  0.625 MSE:  0.375 UAR:  0.5672268907563025 Recall:  0.42857142857142855 Precision:  0.375 F1:  0.39999999999999997
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_traditional Accuracy:  0.625 MSE:  0.375 UAR:  0.5672268907563025 Recall:  0.42857142857142855

In [129]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [130]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [131]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_traditional_best Accuracy:  0.3541666666666667 MSE:  0.6458333333333334 UAR:  0.5441176470588235 Recall:  1.0 Precision:  0.3111111111111111 F1:  0.4745762711864407
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_traditional_best Accuracy:  0.375 MSE:  0.625 UAR:  0.5168067226890756 Recall:  0.8571428571428571 Precision:  0.3 F1:  0.4444444444444444
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_traditional_best Accuracy:  0.4583333333333333 MSE:  0.5416666666666666 UAR:  0.5756302521008403 Recall:  0.8571428571428571 Precision:  0.3333333333333333 F1:  0.48
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_traditional_best Accuracy:  0.625 MSE:  0.375 UAR:  0.6302521008403361 Recall:  0.6428571428571429 Precision:  0.4090909090909091 F1:  0.5000000000000001
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_traditional_best Accuracy:  0.7291666666666666 MSE:  0.2708333333333333

#### Balanced split

In [132]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [133]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [134]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


In [135]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.759)

(147,) (147,)
(48,) (48,)
(195,) (195,)
[0 1] [141  54]
195
147
48


In [136]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


##### Single attention

In [137]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

mobilenet_7.h5_EngageWild_2_max_single_attention_balanced


In [138]:
CONCATENATE_STAT = True

In [139]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5631, 128, 2048) (5631,)
(2102, 128, 2048) (2102,)


In [140]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [141]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
44/44 [==============================] - 4s 61ms/step - loss: 0.6190 - acc: 0.7223 - auc: 0.7111 - binary_accuracy: 0.7223 - recall_8: 0.3849 - precision_8: 0.5611 - val_loss: 0.6488 - val_acc: 0.6151 - val_auc: 0.5859 - val_binary_accuracy: 0.6151 - val_recall_8: 0.0945 - val_precision_8: 0.2041
Epoch 2/20
44/44 [==============================] - 1s 27ms/step - loss: 0.3018 - acc: 0.8863 - auc: 0.9412 - binary_accuracy: 0.8863 - recall_8: 0.7036 - precision_8: 0.8996 - val_loss: 0.6827 - val_acc: 0.6813 - val_auc: 0.6146 - val_binary_accuracy: 0.6813 - val_recall_8: 0.3638 - val_precision_8: 0.4648
Epoch 3/20
44/44 [==============================] - 1s 27ms/step - loss: 0.1704 - acc: 0.9533 - auc: 0.9885 - binary_accuracy: 0.9533 - recall_8: 0.8735 - precision_8: 0.9694 - val_loss: 0.8436 - val_acc: 0.6518 - val_auc: 0.6007 - val_binary_accuracy: 0.6518 - val_recall_8: 0.3370 - val_precision_8: 0.4076
Epoch 4/20
44/44 [==============================] - 1s 28ms/step - loss: 

In [142]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [143]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_balanced Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6176470588235294 Recall:  0.5 Precision:  0.4375 F1:  0.4666666666666667
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_balanced Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.5819327731092437 Recall:  0.42857142857142855 Precision:  0.4 F1:  0.4137931034482759
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_balanced Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.5819327731092437 Recall:  0.42857142857142855 Precision:  0.4 F1:  0.4137931034482759
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_balanced Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.5966386554621849 Recall:  0.42857142857142855 Precision:  0.42857142857142855 F1:  0.42857142857142855
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_balanced Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6

In [144]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [145]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [146]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_balanced_best Accuracy:  0.4166666666666667 MSE:  0.5833333333333334 UAR:  0.5882352941176471 Recall:  1.0 Precision:  0.3333333333333333 F1:  0.5
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_balanced_best Accuracy:  0.5 MSE:  0.5 UAR:  0.5840336134453781 Recall:  0.7857142857142857 Precision:  0.34375 F1:  0.4782608695652174
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_balanced_best Accuracy:  0.5 MSE:  0.5 UAR:  0.45798319327731096 Recall:  0.35714285714285715 Precision:  0.25 F1:  0.2941176470588235
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_balanced_best Accuracy:  0.5833333333333334 MSE:  0.4166666666666667 UAR:  0.4747899159663866 Recall:  0.21428571428571427 Precision:  0.25 F1:  0.23076923076923075
Metric_name:  mobilenet_7.h5_EngageWild_2_max_single_attention_balanced_best Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.5126050420168067 Recall: 

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### Self attention

In [147]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

mobilenet_7.h5_EngageWild_2_max_self_attention_balanced


In [148]:
CONCATENATE_STAT = True

In [149]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5631, 128, 2048) (5631,)
(2102, 128, 2048) (2102,)


In [150]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 attention_4 (Attention)     (None, None, 2048)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [151]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
44/44 [==============================] - 6s 108ms/step - loss: 1.0655 - acc: 0.7862 - auc: 0.7845 - binary_accuracy: 0.7862 - recall_9: 0.6233 - precision_9: 0.6548 - val_loss: 0.8465 - val_acc: 0.6089 - val_auc: 0.5998 - val_binary_accuracy: 0.6089 - val_recall_9: 0.3543 - val_precision_9: 0.3532
Epoch 2/20
44/44 [==============================] - 3s 65ms/step - loss: 0.1544 - acc: 0.9494 - auc: 0.9865 - binary_accuracy: 0.9494 - recall_9: 0.8822 - precision_9: 0.9472 - val_loss: 0.9989 - val_acc: 0.5966 - val_auc: 0.6011 - val_binary_accuracy: 0.5966 - val_recall_9: 0.1969 - val_precision_9: 0.2700
Epoch 3/20
44/44 [==============================] - 3s 65ms/step - loss: 0.0840 - acc: 0.9805 - auc: 0.9981 - binary_accuracy: 0.9805 - recall_9: 0.9514 - precision_9: 0.9836 - val_loss: 1.2105 - val_acc: 0.5861 - val_auc: 0.5777 - val_binary_accuracy: 0.5861 - val_recall_9: 0.1591 - val_precision_9: 0.2311
Epoch 4/20
44/44 [==============================] - 3s 66ms/step - loss:

In [152]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [153]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_balanced Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.5399159663865546 Recall:  0.2857142857142857 Precision:  0.36363636363636365 F1:  0.32
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_balanced Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.4684873949579832 Recall:  0.14285714285714285 Precision:  0.2222222222222222 F1:  0.17391304347826086
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_balanced Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.4684873949579832 Recall:  0.14285714285714285 Precision:  0.2222222222222222 F1:  0.17391304347826086
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_balanced Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.4684873949579832 Recall:  0.14285714285714285 Precision:  0.2222222222222222 F1:  0.17391304347826086
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_balanced Accur

In [154]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [155]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [156]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_balanced_best Accuracy:  0.4791666666666667 MSE:  0.5208333333333334 UAR:  0.5903361344537815 Recall:  0.8571428571428571 Precision:  0.34285714285714286 F1:  0.4897959183673469
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_balanced_best Accuracy:  0.5208333333333334 MSE:  0.4791666666666667 UAR:  0.5987394957983193 Recall:  0.7857142857142857 Precision:  0.3548387096774194 F1:  0.48888888888888893
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_balanced_best Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.657563025210084 Recall:  0.7857142857142857 Precision:  0.4074074074074074 F1:  0.5365853658536585
Metric_name:  mobilenet_7.h5_EngageWild_2_max_self_attention_balanced_best Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.6239495798319328 Recall:  0.5714285714285714 Precision:  0.42105263157894735 F1:  0.48484848484848486
Metric_name:  mobilenet_7.h5_EngageWild_2_max

### STAT

In [27]:
stat_func = compute_descriptor
stat_name = "STAT"

#### Traditional split

In [158]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [159]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [160]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


##### Single attention

In [161]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

mobilenet_7.h5_EngageWild_2_STAT_single_attention_traditional


In [162]:
CONCATENATE_STAT = True

In [163]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 3072) (5449,)
(2284, 128, 3072) (2284,)


In [164]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3072)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3073      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                           

In [165]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 5s 94ms/step - loss: 0.4465 - acc: 0.8012 - auc: 0.8244 - binary_accuracy: 0.8012 - recall_10: 0.5067 - precision_10: 0.7147 - val_loss: 0.6960 - val_acc: 0.6585 - val_auc: 0.6723 - val_binary_accuracy: 0.6585 - val_recall_10: 0.3019 - val_precision_10: 0.5053
Epoch 2/20
43/43 [==============================] - 2s 39ms/step - loss: 0.2180 - acc: 0.9095 - auc: 0.9672 - binary_accuracy: 0.9095 - recall_10: 0.7938 - precision_10: 0.8778 - val_loss: 0.8028 - val_acc: 0.6546 - val_auc: 0.6677 - val_binary_accuracy: 0.6546 - val_recall_10: 0.1962 - val_precision_10: 0.4936
Epoch 3/20
43/43 [==============================] - 2s 38ms/step - loss: 0.1432 - acc: 0.9424 - auc: 0.9876 - binary_accuracy: 0.9424 - recall_10: 0.8696 - precision_10: 0.9242 - val_loss: 0.7691 - val_acc: 0.6563 - val_auc: 0.6781 - val_binary_accuracy: 0.6563 - val_recall_10: 0.2803 - val_precision_10: 0.5000
Epoch 4/20
43/43 [==============================] - 2s 38ms/s

In [166]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [167]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_traditional Accuracy:  0.5625 MSE:  0.4375 UAR:  0.523109243697479 Recall:  0.42857142857142855 Precision:  0.3157894736842105 F1:  0.36363636363636365
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_traditional Accuracy:  0.5833333333333334 MSE:  0.4166666666666667 UAR:  0.5168067226890757 Recall:  0.35714285714285715 Precision:  0.3125 F1:  0.3333333333333333
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_traditional Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.5105042016806722 Recall:  0.2857142857142857 Precision:  0.3076923076923077 F1:  0.29629629629629634
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_traditional Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.5399159663865546 Recall:  0.2857142857142857 Precision:  0.36363636363636365 F1:  0.32
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_traditional Accuracy:  0.6

In [168]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [169]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [170]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_traditional_best Accuracy:  0.4791666666666667 MSE:  0.5208333333333334 UAR:  0.6113445378151261 Recall:  0.9285714285714286 Precision:  0.35135135135135137 F1:  0.5098039215686275
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_traditional_best Accuracy:  0.6458333333333334 MSE:  0.3541666666666667 UAR:  0.6659663865546219 Recall:  0.7142857142857143 Precision:  0.43478260869565216 F1:  0.5405405405405405
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_traditional_best Accuracy:  0.7708333333333334 MSE:  0.22916666666666666 UAR:  0.6911764705882353 Recall:  0.5 Precision:  0.6363636363636364 F1:  0.56
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_traditional_best Accuracy:  0.7291666666666666 MSE:  0.2708333333333333 UAR:  0.5987394957983193 Recall:  0.2857142857142857 Precision:  0.5714285714285714 F1:  0.38095238095238093
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_si

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### Self attention

In [171]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

mobilenet_7.h5_EngageWild_2_STAT_self_attention_traditional


In [172]:
CONCATENATE_STAT = True

In [173]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5449, 128, 3072) (5449,)
(2284, 128, 3072) (2284,)


In [174]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3072)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 3072)           9437184   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 3072)           9437184   ['image_set[0][0]']           
                                                                                                  
 attention_5 (Attention)     (None, None, 3072)           0         ['query[0][0]',               
                                                                     'value[0][0]']        

In [175]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 9s 181ms/step - loss: 0.8626 - acc: 0.7851 - auc: 0.7786 - binary_accuracy: 0.7851 - recall_11: 0.5793 - precision_11: 0.6361 - val_loss: 0.7451 - val_acc: 0.6213 - val_auc: 0.6604 - val_binary_accuracy: 0.6213 - val_recall_11: 0.1554 - val_precision_11: 0.3765
Epoch 2/20
43/43 [==============================] - 5s 125ms/step - loss: 0.1910 - acc: 0.9284 - auc: 0.9751 - binary_accuracy: 0.9284 - recall_11: 0.8382 - precision_11: 0.9044 - val_loss: 0.7915 - val_acc: 0.6694 - val_auc: 0.6388 - val_binary_accuracy: 0.6694 - val_recall_11: 0.3389 - val_precision_11: 0.5299
Epoch 3/20
43/43 [==============================] - 6s 130ms/step - loss: 0.1192 - acc: 0.9604 - auc: 0.9929 - binary_accuracy: 0.9604 - recall_11: 0.9094 - precision_11: 0.9497 - val_loss: 0.7957 - val_acc: 0.6677 - val_auc: 0.6517 - val_binary_accuracy: 0.6677 - val_recall_11: 0.3643 - val_precision_11: 0.5238
Epoch 4/20
43/43 [==============================] - 5s 124

In [176]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [177]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_traditional Accuracy:  0.5416666666666666 MSE:  0.4583333333333333 UAR:  0.592436974789916 Recall:  0.7142857142857143 Precision:  0.35714285714285715 F1:  0.4761904761904762
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_traditional Accuracy:  0.5208333333333334 MSE:  0.4791666666666667 UAR:  0.5567226890756303 Recall:  0.6428571428571429 Precision:  0.3333333333333333 F1:  0.43902439024390244
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_traditional Accuracy:  0.5 MSE:  0.5 UAR:  0.5210084033613445 Recall:  0.5714285714285714 Precision:  0.3076923076923077 F1:  0.4
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_traditional Accuracy:  0.5208333333333334 MSE:  0.4791666666666667 UAR:  0.5357142857142857 Recall:  0.5714285714285714 Precision:  0.32 F1:  0.41025641025641024
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_traditional Accuracy:  0.5208333333333334 MSE: 

In [178]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [179]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4051215/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [180]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_traditional_best Accuracy:  0.5 MSE:  0.5 UAR:  0.5840336134453781 Recall:  0.7857142857142857 Precision:  0.34375 F1:  0.4782608695652174
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_traditional_best Accuracy:  0.625 MSE:  0.375 UAR:  0.6302521008403361 Recall:  0.6428571428571429 Precision:  0.4090909090909091 F1:  0.5000000000000001
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_traditional_best Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6470588235294117 Recall:  0.5 Precision:  0.5 F1:  0.5
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_traditional_best Accuracy:  0.7708333333333334 MSE:  0.22916666666666666 UAR:  0.6911764705882353 Recall:  0.5 Precision:  0.6363636363636364 F1:  0.56
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_traditional_best Accuracy:  0.7291666666666666 MSE:  0.2708333333333333 UAR:  0.5987394957983193 Recall:  0.28

#### Balanced split

In [28]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [29]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  147
Test:  48


In [30]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


In [31]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.759)

(147, 2048) (147,)
(48, 2048) (48,)
(195, 2048) (195,)
[0 1] [141  54]
195
147
48


In [32]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 107/147: 72.78911564625851%
1 40/147: 27.210884353741495%
val:
0 34/48: 70.83333333333333%
1 14/48: 29.166666666666668%


##### Single attention

In [35]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

mobilenet_7.h5_EngageWild_2_STAT_single_attention_balanced


In [36]:
CONCATENATE_STAT = True

In [37]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5631, 128, 3072) (5631,)
(2102, 128, 3072) (2102,)


In [38]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-07-30 08:14:20.359963: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3072)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3073      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                              

In [39]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20


2024-07-30 08:14:31.460675: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1f36bbf0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-30 08:14:31.460844: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-07-30 08:14:31.464965: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-30 08:14:31.510415: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-07-30 08:14:31.563412: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


44/44 [==============================] - 8s 123ms/step - loss: 0.3858 - acc: 0.8304 - auc: 0.8784 - binary_accuracy: 0.8304 - recall: 0.6104 - precision: 0.7823 - val_loss: 0.7626 - val_acc: 0.6665 - val_auc: 0.6457 - val_binary_accuracy: 0.6665 - val_recall: 0.3291 - val_precision: 0.4318
Epoch 2/20
44/44 [==============================] - 2s 37ms/step - loss: 0.1524 - acc: 0.9433 - auc: 0.9869 - binary_accuracy: 0.9433 - recall: 0.8670 - precision: 0.9415 - val_loss: 0.9998 - val_acc: 0.6056 - val_auc: 0.6323 - val_binary_accuracy: 0.6056 - val_recall: 0.2520 - val_precision: 0.3113
Epoch 3/20
44/44 [==============================] - 2s 37ms/step - loss: 0.0812 - acc: 0.9718 - auc: 0.9975 - binary_accuracy: 0.9718 - recall: 0.9262 - precision: 0.9796 - val_loss: 1.1166 - val_acc: 0.6598 - val_auc: 0.6427 - val_binary_accuracy: 0.6598 - val_recall: 0.3669 - val_precision: 0.4267
Epoch 4/20
44/44 [==============================] - 2s 37ms/step - loss: 0.0486 - acc: 0.9874 - auc: 0.9995

In [40]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4087404/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [41]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_balanced Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6176470588235294 Recall:  0.5 Precision:  0.4375 F1:  0.4666666666666667
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_balanced Accuracy:  0.7291666666666666 MSE:  0.2708333333333333 UAR:  0.6407563025210083 Recall:  0.42857142857142855 Precision:  0.5454545454545454 F1:  0.4799999999999999
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_balanced Accuracy:  0.75 MSE:  0.25 UAR:  0.6554621848739496 Recall:  0.42857142857142855 Precision:  0.6 F1:  0.5
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_balanced Accuracy:  0.75 MSE:  0.25 UAR:  0.634453781512605 Recall:  0.35714285714285715 Precision:  0.625 F1:  0.45454545454545453
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_balanced Accuracy:  0.7291666666666666 MSE:  0.2708333333333333 UAR:  0.5987394957983193 Recall:  0.285714285714285

In [42]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [43]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4087404/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [44]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_balanced_best Accuracy:  0.5208333333333334 MSE:  0.4791666666666667 UAR:  0.5777310924369747 Recall:  0.7142857142857143 Precision:  0.3448275862068966 F1:  0.46511627906976755
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_balanced_best Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6596638655462186 Recall:  0.6428571428571429 Precision:  0.45 F1:  0.5294117647058824
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_balanced_best Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6533613445378151 Recall:  0.5714285714285714 Precision:  0.47058823529411764 F1:  0.5161290322580646
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_balanced_best Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.5966386554621849 Recall:  0.42857142857142855 Precision:  0.42857142857142855 F1:  0.42857142857142855
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_single_attention_balan

##### Self attention

In [33]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

mobilenet_7.h5_EngageWild_2_STAT_self_attention_balanced


In [34]:
CONCATENATE_STAT = True

In [35]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5631, 128, 3072) (5631,)
(2102, 128, 3072) (2102,)


In [36]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-07-30 08:30:39.929705: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3072)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 3072)           9437184   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 3072)           9437184   ['image_set[0][0]']           
                                                                                                  
 attention (Attention)       (None, None, 3072)           0         ['query[0][0]',               
                                                                     'value[0][0]']           

In [37]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

In [38]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4095000/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [39]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_balanced Accuracy:  0.6666666666666666 MSE:  0.3333333333333333 UAR:  0.6386554621848739 Recall:  0.5714285714285714 Precision:  0.4444444444444444 F1:  0.5
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_balanced Accuracy:  0.6875 MSE:  0.3125 UAR:  0.6533613445378151 Recall:  0.5714285714285714 Precision:  0.47058823529411764 F1:  0.5161290322580646
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_balanced Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6470588235294117 Recall:  0.5 Precision:  0.5 F1:  0.5
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_balanced Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6470588235294117 Recall:  0.5 Precision:  0.5 F1:  0.5
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_balanced Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.6470588235294117 Recall:  0.5 Precision:  0.5 F1:  0.5
Met

In [41]:
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    modelAtn.set_weights(best_model_weights)
    save_weights(modelAtn, weights_name)

In [42]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_4095000/1902217451.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [43]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_balanced_best Accuracy:  0.4375 MSE:  0.5625 UAR:  0.5819327731092437 Recall:  0.9285714285714286 Precision:  0.3333333333333333 F1:  0.4905660377358491
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_balanced_best Accuracy:  0.6041666666666666 MSE:  0.3958333333333333 UAR:  0.6785714285714286 Recall:  0.8571428571428571 Precision:  0.41379310344827586 F1:  0.5581395348837208
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_balanced_best Accuracy:  0.7083333333333334 MSE:  0.2916666666666667 UAR:  0.73109243697479 Recall:  0.7857142857142857 Precision:  0.5 F1:  0.6111111111111112
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_balanced_best Accuracy:  0.7291666666666666 MSE:  0.2708333333333333 UAR:  0.6827731092436975 Recall:  0.5714285714285714 Precision:  0.5333333333333333 F1:  0.5517241379310344
Metric_name:  mobilenet_7.h5_EngageWild_2_STAT_self_attention_balanced_best Accuracy

# DAiSEE dataset (4 classes)

In [24]:
FEATURES_DIR = DATA_DIR + 'features_DAiSEE/'
WEIGHTS_DIR = DATA_DIR + 'weights_DAiSEE/'
TABLE_NAME = '02_DAiSEE_4_classes.xlsx'

## enet_b0_8_best_afew

In [25]:
base_model_key = 'enet_b0_8_best_afew.pt'

In [26]:
ENGAGE_WILD_DATASET, BIN_CLASSIFICATION, N_CLASSES, DATASET_NAME, BASE_DATASET_DIR, ext = define_dataset(False, False)
video2label, labels_list = get_video2label()
print(labels_list)
print(video2label)

['very distracted', 'distracted', 'engaged', 'very engaged']
{'1100011002': 0.66, '1100011003': 0.66, '1100011004': 1.0, '1100011005': 1.0, '1100011006': 1.0, '1100011007': 0.66, '1100011008': 1.0, '1100011009': 0.66, '1100011010': 1.0, '1100011011': 1.0, '1100011012': 0.66, '1100011013': 1.0, '1100011014': 1.0, '1100011015': 1.0, '1100011016': 1.0, '1100011017': 1.0, '1100011018': 1.0, '1100011019': 1.0, '1100011020': 1.0, '1100011021': 1.0, '1100011022': 1.0, '1100011023': 1.0, '1100011025': 1.0, '1100011026': 1.0, '1100011027': 1.0, '1100011028': 1.0, '1100011029': 1.0, '1100011031': 1.0, '1100011032': 1.0, '1100011034': 1.0, '1100011035': 1.0, '1100011037': 1.0, '1100011038': 1.0, '1100011040': 0.66, '1100011046': 1.0, '1100011047': 1.0, '1100011048': 0.66, '1100011049': 1.0, '1100011050': 1.0, '1100011051': 1.0, '1100011052': 1.0, '1100011053': 1.0, '1100011054': 1.0, '1100011055': 1.0, '1100011056': 1.0, '1100011057': 1.0, '1100011058': 1.0, '1100011059': 1.0, '1100011060': 1.0, 

### Std

In [27]:
stat_func = np.std
stat_name = "std"

#### Traditional split

In [28]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
val_pickle = FEATURES_DIR + 'val_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)
filename2features_val = load_features(val_pickle)

In [29]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))
print("Val: ", len(filename2features_val))

Train:  5482
Test:  1723
Val:  1720


In [30]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))
print("val:")
print_classes(get_labels(filename2features_val))

train:
0.0 34/5482: 0.6202116016052536%
0.33 214/5482: 3.903684786574243%
0.66 2649/5482: 48.32178037212696%
1.0 2585/5482: 47.15432323969354%
test:
0.0 4/1723: 0.2321532211259431%
0.33 81/1723: 4.7011027278003485%
0.66 861/1723: 49.97098084735926%
1.0 777/1723: 45.095763203714455%
val:
0.0 23/1720: 1.3372093023255813%
0.33 160/1720: 9.30232558139535%
0.66 912/1720: 53.02325581395349%
1.0 625/1720: 36.33720930232558%


##### Single attention

In [31]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_std_single_attention_traditional


In [32]:
CONCATENATE_STAT = True

In [33]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2560) (5482,)
(1723, 128, 2560) (1723,)
(1720, 128, 2560) (1720,)


In [34]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-08-05 07:01:38.946316: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                              

In [35]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20


2024-08-05 07:01:51.048901: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7eee0bd8e490 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-05 07:01:51.049059: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-08-05 07:01:51.053350: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-05 07:01:51.086992: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-08-05 07:01:51.138453: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


43/43 [==============================] - 5s 84ms/step - loss: 0.0467 - mae: 0.1792 - val_loss: 0.0499 - val_mae: 0.1833
Epoch 2/20
43/43 [==============================] - 1s 29ms/step - loss: 0.0352 - mae: 0.1603 - val_loss: 0.0498 - val_mae: 0.1834
Epoch 3/20
43/43 [==============================] - 1s 29ms/step - loss: 0.0342 - mae: 0.1571 - val_loss: 0.0511 - val_mae: 0.1879
Epoch 4/20
43/43 [==============================] - 1s 29ms/step - loss: 0.0334 - mae: 0.1539 - val_loss: 0.0523 - val_mae: 0.1896
Epoch 5/20
43/43 [==============================] - 1s 29ms/step - loss: 0.0329 - mae: 0.1528 - val_loss: 0.0544 - val_mae: 0.1955
Epoch 6/20
43/43 [==============================] - 1s 29ms/step - loss: 0.0324 - mae: 0.1524 - val_loss: 0.0507 - val_mae: 0.1853
Epoch 7/20
43/43 [==============================] - 1s 29ms/step - loss: 0.0322 - mae: 0.1508 - val_loss: 0.0499 - val_mae: 0.1849
Epoch 8/20
43/43 [==============================] - 1s 29ms/step - loss: 0.0318 - mae: 0.1496 

In [36]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [37]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_single_attention_traditional Accuracy:  0.5217643644805572 MSE:  0.038047092321387664 UAR:  0.2804014816209938 Recall:  N/A Precision:  N/A F1:  0.25784831847775197


(0.5217643644805572,
 0.038047092321387664,
 0.2804014816209938,
 'N/A',
 'N/A',
 0.25784831847775197)

In [38]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [39]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [40]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_single_attention_traditional_best Accuracy:  0.5368543238537434 MSE:  0.03966343271742339 UAR:  0.29005864168465795 Recall:  N/A Precision:  N/A F1:  0.2859987189068005


(0.5368543238537434,
 0.03966343271742339,
 0.29005864168465795,
 'N/A',
 'N/A',
 0.2859987189068005)

##### Self attention

In [41]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_traditional


In [42]:
CONCATENATE_STAT = True

In [43]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2560) (5482,)
(1723, 128, 2560) (1723,)
(1720, 128, 2560) (1720,)


In [44]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention (Attention)       (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [45]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 6s 120ms/step - loss: 0.0812 - mae: 0.2005 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 2/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 3/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 4/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 5/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 6/20
43/43 [==============================] - 4s 88ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 7/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 8/20
43/43 [==============================] - 4s 85ms/step - loss: 0.0796 - 

In [46]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [47]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_traditional Accuracy:  0.4509576320371445 MSE:  0.08119123621590248 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.1554


(0.4509576320371445, 0.08119123621590248, 0.25, 'N/A', 'N/A', 0.1554)

In [48]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [49]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [50]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_traditional_best Accuracy:  0.4509576320371445 MSE:  0.08119123621590248 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.1554


(0.4509576320371445, 0.08119123621590248, 0.25, 'N/A', 'N/A', 0.1554)

#### Balanced split

In [60]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [61]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  5482
Test:  1723


In [62]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))

train:
0.0 34/5482: 0.6202116016052536%
0.33 214/5482: 3.903684786574243%
0.66 2649/5482: 48.32178037212696%
1.0 2585/5482: 47.15432323969354%
test:
0.0 4/1723: 0.2321532211259431%
0.33 81/1723: 4.7011027278003485%
0.66 861/1723: 49.97098084735926%
1.0 777/1723: 45.095763203714455%


In [63]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.76)

(5482,) (5482,)
(1723,) (1723,)
(7205,) (7205,)
[0.   0.33 0.66 1.  ] [  38  295 3510 3362]
7205
5474
1731


In [64]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))

train:
0.0 28/5474: 0.5115089514066496%
0.33 224/5474: 4.092071611253197%
0.66 2667/5474: 48.72122762148338%
1.0 2555/5474: 46.67519181585678%
test:
0.0 10/1731: 0.5777007510109763%
0.33 71/1731: 4.101675332177932%
0.66 843/1731: 48.7001733102253%
1.0 807/1731: 46.62045060658579%


##### Single attention

In [65]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_std_single_attention_balanced


In [66]:
CONCATENATE_STAT = True

In [67]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5474, 128, 2560) (5474,)
(1731, 128, 2560) (1731,)


In [68]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [69]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 4s 62ms/step - loss: 0.0455 - mae: 0.1805 - val_loss: 0.0361 - val_mae: 0.1671
Epoch 2/20
43/43 [==============================] - 1s 30ms/step - loss: 0.0358 - mae: 0.1640 - val_loss: 0.0343 - val_mae: 0.1609
Epoch 3/20
43/43 [==============================] - 1s 30ms/step - loss: 0.0346 - mae: 0.1598 - val_loss: 0.0336 - val_mae: 0.1579
Epoch 4/20
43/43 [==============================] - 1s 29ms/step - loss: 0.0337 - mae: 0.1572 - val_loss: 0.0336 - val_mae: 0.1591
Epoch 5/20
43/43 [==============================] - 1s 29ms/step - loss: 0.0331 - mae: 0.1559 - val_loss: 0.0339 - val_mae: 0.1540
Epoch 6/20
43/43 [==============================] - 1s 30ms/step - loss: 0.0331 - mae: 0.1555 - val_loss: 0.0343 - val_mae: 0.1544
Epoch 7/20
43/43 [==============================] - 1s 30ms/step - loss: 0.0328 - mae: 0.1542 - val_loss: 0.0335 - val_mae: 0.1533
Epoch 8/20
43/43 [==============================] - 1s 30ms/step - loss: 0.0322 - m

In [70]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [71]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_single_attention_balanced Accuracy:  0.5771230502599654 MSE:  0.03434189749156464 UAR:  0.32464645258206026 Recall:  N/A Precision:  N/A F1:  0.31959519092782207


(0.5771230502599654,
 0.03434189749156464,
 0.32464645258206026,
 'N/A',
 'N/A',
 0.31959519092782207)

In [72]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [73]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [74]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_single_attention_balanced_best Accuracy:  0.6077411900635471 MSE:  0.03344626485229473 UAR:  0.3238047218162814 Recall:  N/A Precision:  N/A F1:  0.313621256796059


(0.6077411900635471,
 0.03344626485229473,
 0.3238047218162814,
 'N/A',
 'N/A',
 0.313621256796059)

##### Self attention

In [75]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_balanced


In [76]:
CONCATENATE_STAT = True

In [77]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5474, 128, 2560) (5474,)
(1731, 128, 2560) (1731,)


In [78]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_1 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [79]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 6s 125ms/step - loss: 0.0812 - mae: 0.2012 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 2/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 3/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 4/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 5/20
43/43 [==============================] - 4s 88ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 6/20
43/43 [==============================] - 4s 88ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 7/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 8/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - 

In [80]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [81]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_balanced Accuracy:  0.4662045060658579 MSE:  0.08048682842287694 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048682842287694,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

In [82]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [83]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [84]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_std_self_attention_balanced_best Accuracy:  0.4662045060658579 MSE:  0.08048682842287694 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048682842287694,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

### Max

In [85]:
stat_func = np.max
stat_name = "max"

#### Traditional split

In [86]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
val_pickle = FEATURES_DIR + 'val_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)
filename2features_val = load_features(val_pickle)

In [87]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))
print("Val: ", len(filename2features_val))

Train:  5482
Test:  1723
Val:  1720


In [88]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))
print("val:")
print_classes(get_labels(filename2features_val))

train:
0.0 34/5482: 0.6202116016052536%
0.33 214/5482: 3.903684786574243%
0.66 2649/5482: 48.32178037212696%
1.0 2585/5482: 47.15432323969354%
test:
0.0 4/1723: 0.2321532211259431%
0.33 81/1723: 4.7011027278003485%
0.66 861/1723: 49.97098084735926%
1.0 777/1723: 45.095763203714455%
val:
0.0 23/1720: 1.3372093023255813%
0.33 160/1720: 9.30232558139535%
0.66 912/1720: 53.02325581395349%
1.0 625/1720: 36.33720930232558%


##### Single attention

In [89]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_max_single_attention_traditional


In [90]:
CONCATENATE_STAT = True

In [91]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2560) (5482,)
(1723, 128, 2560) (1723,)
(1720, 128, 2560) (1720,)


In [92]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [93]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 3s 62ms/step - loss: 0.0808 - mae: 0.2002 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 2/20
43/43 [==============================] - 1s 30ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 3/20
43/43 [==============================] - 1s 30ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 4/20
43/43 [==============================] - 1s 29ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 5/20
43/43 [==============================] - 1s 29ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 6/20
43/43 [==============================] - 1s 29ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 7/20
43/43 [==============================] - 1s 29ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 8/20
43/43 [==============================] - 1s 29ms/step - loss: 0.0796 - m

In [94]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [95]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_max_single_attention_traditional Accuracy:  0.5252466627974464 MSE:  0.03956373653398682 UAR:  0.28726490718360637 Recall:  N/A Precision:  N/A F1:  0.2729057554923391


(0.5252466627974464,
 0.03956373653398682,
 0.28726490718360637,
 'N/A',
 'N/A',
 0.2729057554923391)

In [96]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [97]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [98]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_max_single_attention_traditional_best Accuracy:  0.5206035983749274 MSE:  0.03961097842999725 UAR:  0.28506758791311637 Recall:  N/A Precision:  N/A F1:  0.2698112420225823


(0.5206035983749274,
 0.03961097842999725,
 0.28506758791311637,
 'N/A',
 'N/A',
 0.2698112420225823)

##### Self attention

In [99]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_max_self_attention_traditional


In [100]:
CONCATENATE_STAT = True

In [101]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2560) (5482,)
(1723, 128, 2560) (1723,)
(1720, 128, 2560) (1720,)


In [102]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_2 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [103]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 6s 120ms/step - loss: 0.0815 - mae: 0.2004 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 2/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 3/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 4/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 5/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 6/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 7/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 8/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0796 - 

In [104]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [105]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_max_self_attention_traditional Accuracy:  0.4509576320371445 MSE:  0.08119123621590248 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.1554


(0.4509576320371445, 0.08119123621590248, 0.25, 'N/A', 'N/A', 0.1554)

In [106]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [107]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [108]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_max_self_attention_traditional_best Accuracy:  0.4509576320371445 MSE:  0.08119123621590248 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.1554


(0.4509576320371445, 0.08119123621590248, 0.25, 'N/A', 'N/A', 0.1554)

#### Balanced split

In [109]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [110]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  5482
Test:  1723


In [111]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))

train:
0.0 34/5482: 0.6202116016052536%
0.33 214/5482: 3.903684786574243%
0.66 2649/5482: 48.32178037212696%
1.0 2585/5482: 47.15432323969354%
test:
0.0 4/1723: 0.2321532211259431%
0.33 81/1723: 4.7011027278003485%
0.66 861/1723: 49.97098084735926%
1.0 777/1723: 45.095763203714455%


In [112]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.76)

(5482,) (5482,)
(1723,) (1723,)
(7205,) (7205,)
[0.   0.33 0.66 1.  ] [  38  295 3510 3362]
7205
5474
1731


In [113]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))

train:
0.0 28/5474: 0.5115089514066496%
0.33 224/5474: 4.092071611253197%
0.66 2667/5474: 48.72122762148338%
1.0 2555/5474: 46.67519181585678%
test:
0.0 10/1731: 0.5777007510109763%
0.33 71/1731: 4.101675332177932%
0.66 843/1731: 48.7001733102253%
1.0 807/1731: 46.62045060658579%


##### Single attention

In [114]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_max_single_attention_balanced


In [115]:
CONCATENATE_STAT = True

In [116]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5474, 128, 2560) (5474,)
(1731, 128, 2560) (1731,)


In [117]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [118]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 4s 63ms/step - loss: 0.0801 - mae: 0.2012 - val_loss: 0.0793 - val_mae: 0.1980
Epoch 2/20
43/43 [==============================] - 1s 30ms/step - loss: 0.0594 - mae: 0.1835 - val_loss: 0.0372 - val_mae: 0.1688
Epoch 3/20
43/43 [==============================] - 1s 31ms/step - loss: 0.0361 - mae: 0.1611 - val_loss: 0.0343 - val_mae: 0.1594
Epoch 4/20
43/43 [==============================] - 1s 29ms/step - loss: 0.0344 - mae: 0.1577 - val_loss: 0.0337 - val_mae: 0.1567
Epoch 5/20
43/43 [==============================] - 1s 29ms/step - loss: 0.0341 - mae: 0.1569 - val_loss: 0.0341 - val_mae: 0.1557
Epoch 6/20
43/43 [==============================] - 1s 30ms/step - loss: 0.0334 - mae: 0.1545 - val_loss: 0.0344 - val_mae: 0.1556
Epoch 7/20
43/43 [==============================] - 1s 29ms/step - loss: 0.0330 - mae: 0.1540 - val_loss: 0.0337 - val_mae: 0.1540
Epoch 8/20
43/43 [==============================] - 1s 29ms/step - loss: 0.0325 - m

In [119]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [120]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_max_single_attention_balanced Accuracy:  0.5597920277296361 MSE:  0.03518569922096237 UAR:  0.30947250441400515 Recall:  N/A Precision:  N/A F1:  0.2954889341453528


(0.5597920277296361,
 0.03518569922096237,
 0.30947250441400515,
 'N/A',
 'N/A',
 0.2954889341453528)

In [121]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [122]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [123]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_max_single_attention_balanced_best Accuracy:  0.560369728480647 MSE:  0.03360795267187779 UAR:  0.2972544506034829 Recall:  N/A Precision:  N/A F1:  0.26869345457200394


(0.560369728480647,
 0.03360795267187779,
 0.2972544506034829,
 'N/A',
 'N/A',
 0.26869345457200394)

##### Self attention

In [124]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_max_self_attention_balanced


In [125]:
CONCATENATE_STAT = True

In [126]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5474, 128, 2560) (5474,)
(1731, 128, 2560) (1731,)


In [127]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_3 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [128]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 6s 121ms/step - loss: 0.0814 - mae: 0.2014 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 2/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 3/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 4/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 5/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 6/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 7/20
43/43 [==============================] - 4s 87ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 8/20
43/43 [==============================] - 4s 86ms/step - loss: 0.0798 - 

In [129]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [130]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_max_self_attention_balanced Accuracy:  0.4662045060658579 MSE:  0.08048682842287694 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048682842287694,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

In [131]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [132]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [133]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_max_self_attention_balanced_best Accuracy:  0.4662045060658579 MSE:  0.08048682842287694 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048682842287694,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

### STAT

In [134]:
stat_func = compute_descriptor
stat_name = "STAT"

#### Traditional split

In [135]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
val_pickle = FEATURES_DIR + 'val_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)
filename2features_val = load_features(val_pickle)

In [136]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))
print("Val: ", len(filename2features_val))

Train:  5482
Test:  1723
Val:  1720


In [137]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))
print("val:")
print_classes(get_labels(filename2features_val))

train:
0.0 34/5482: 0.6202116016052536%
0.33 214/5482: 3.903684786574243%
0.66 2649/5482: 48.32178037212696%
1.0 2585/5482: 47.15432323969354%
test:
0.0 4/1723: 0.2321532211259431%
0.33 81/1723: 4.7011027278003485%
0.66 861/1723: 49.97098084735926%
1.0 777/1723: 45.095763203714455%
val:
0.0 23/1720: 1.3372093023255813%
0.33 160/1720: 9.30232558139535%
0.66 912/1720: 53.02325581395349%
1.0 625/1720: 36.33720930232558%


##### Single attention

In [138]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_STAT_single_attention_traditional


In [139]:
CONCATENATE_STAT = True

In [140]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 3840) (5482,)
(1723, 128, 3840) (1723,)
(1720, 128, 3840) (1720,)


In [141]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3840)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3841      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [142]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 6s 95ms/step - loss: 0.0802 - mae: 0.1991 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 2/20
43/43 [==============================] - 2s 42ms/step - loss: 0.0796 - mae: 0.1966 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 3/20
43/43 [==============================] - 2s 43ms/step - loss: 0.0796 - mae: 0.1966 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 4/20
43/43 [==============================] - 2s 48ms/step - loss: 0.0771 - mae: 0.1934 - val_loss: 0.1094 - val_mae: 0.2470
Epoch 5/20
43/43 [==============================] - 2s 43ms/step - loss: 0.0609 - mae: 0.1796 - val_loss: 0.0515 - val_mae: 0.1784
Epoch 6/20
43/43 [==============================] - 2s 42ms/step - loss: 0.0357 - mae: 0.1600 - val_loss: 0.0515 - val_mae: 0.1875
Epoch 7/20
43/43 [==============================] - 2s 42ms/step - loss: 0.0338 - mae: 0.1547 - val_loss: 0.0500 - val_mae: 0.1845
Epoch 8/20
43/43 [==============================] - 2s 42ms/step - loss: 0.0331 - m

In [143]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [144]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_STAT_single_attention_traditional Accuracy:  0.5502031340684852 MSE:  0.04595219703537471 UAR:  0.3021430257202615 Recall:  N/A Precision:  N/A F1:  0.3037438085935406


(0.5502031340684852,
 0.04595219703537471,
 0.3021430257202615,
 'N/A',
 'N/A',
 0.3037438085935406)

In [145]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [146]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [147]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_STAT_single_attention_traditional_best Accuracy:  0.522344747533372 MSE:  0.037993142095166056 UAR:  0.28458888621490247 Recall:  N/A Precision:  N/A F1:  0.2748328939305298


(0.522344747533372,
 0.037993142095166056,
 0.28458888621490247,
 'N/A',
 'N/A',
 0.2748328939305298)

##### Self attention

In [148]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_STAT_self_attention_traditional


In [149]:
CONCATENATE_STAT = True

In [150]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 3840) (5482,)
(1723, 128, 3840) (1723,)
(1720, 128, 3840) (1720,)


In [151]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3840)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 3840)           1474560   ['image_set[0][0]']           
                                                          0                                       
                                                                                                  
 value (Dense)               (None, None, 3840)           1474560   ['image_set[0][0]']           
                                                          0                                       
                                                                                            

In [152]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 10s 215ms/step - loss: 0.0822 - mae: 0.2014 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 2/20
43/43 [==============================] - 7s 168ms/step - loss: 0.0798 - mae: 0.1969 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 3/20
43/43 [==============================] - 7s 168ms/step - loss: 0.0798 - mae: 0.1969 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 4/20
43/43 [==============================] - 7s 171ms/step - loss: 0.0798 - mae: 0.1969 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 5/20
43/43 [==============================] - 7s 168ms/step - loss: 0.0798 - mae: 0.1969 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 6/20
43/43 [==============================] - 7s 169ms/step - loss: 0.0798 - mae: 0.1969 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 7/20
43/43 [==============================] - 7s 168ms/step - loss: 0.0798 - mae: 0.1969 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 8/20
43/43 [==============================] - 7s 169ms/step - loss: 0

In [153]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [154]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_STAT_self_attention_traditional Accuracy:  0.4509576320371445 MSE:  0.08119123621590248 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.1554


(0.4509576320371445, 0.08119123621590248, 0.25, 'N/A', 'N/A', 0.1554)

In [155]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [156]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [157]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_STAT_self_attention_traditional_best Accuracy:  0.4509576320371445 MSE:  0.08119123621590248 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.1554


(0.4509576320371445, 0.08119123621590248, 0.25, 'N/A', 'N/A', 0.1554)

#### Balanced split

In [158]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [159]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  5482
Test:  1723


In [160]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))

train:
0.0 34/5482: 0.6202116016052536%
0.33 214/5482: 3.903684786574243%
0.66 2649/5482: 48.32178037212696%
1.0 2585/5482: 47.15432323969354%
test:
0.0 4/1723: 0.2321532211259431%
0.33 81/1723: 4.7011027278003485%
0.66 861/1723: 49.97098084735926%
1.0 777/1723: 45.095763203714455%


In [161]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.76)

(5482, 2560) (5482,)
(1723, 2560) (1723,)
(7205, 2560) (7205,)
[0.   0.33 0.66 1.  ] [  38  295 3510 3362]
7205
5474
1731


In [162]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))

train:
0.0 28/5474: 0.5115089514066496%
0.33 224/5474: 4.092071611253197%
0.66 2667/5474: 48.72122762148338%
1.0 2555/5474: 46.67519181585678%
test:
0.0 10/1731: 0.5777007510109763%
0.33 71/1731: 4.101675332177932%
0.66 843/1731: 48.7001733102253%
1.0 807/1731: 46.62045060658579%


##### Single attention

In [163]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_STAT_single_attention_balanced


In [164]:
CONCATENATE_STAT = True

In [165]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5474, 128, 3840) (5474,)
(1731, 128, 3840) (1731,)


In [166]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3840)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3841      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                           

In [167]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 5s 91ms/step - loss: 0.0772 - mae: 0.1978 - val_loss: 0.0693 - val_mae: 0.1895
Epoch 2/20
43/43 [==============================] - 2s 43ms/step - loss: 0.0427 - mae: 0.1726 - val_loss: 0.0354 - val_mae: 0.1619
Epoch 3/20
43/43 [==============================] - 2s 42ms/step - loss: 0.0351 - mae: 0.1594 - val_loss: 0.0342 - val_mae: 0.1568
Epoch 4/20
43/43 [==============================] - 2s 48ms/step - loss: 0.0344 - mae: 0.1578 - val_loss: 0.0347 - val_mae: 0.1605
Epoch 5/20
43/43 [==============================] - 2s 42ms/step - loss: 0.0338 - mae: 0.1570 - val_loss: 0.0338 - val_mae: 0.1580
Epoch 6/20
43/43 [==============================] - 2s 48ms/step - loss: 0.0336 - mae: 0.1555 - val_loss: 0.0347 - val_mae: 0.1563
Epoch 7/20
43/43 [==============================] - 2s 42ms/step - loss: 0.0331 - mae: 0.1537 - val_loss: 0.0353 - val_mae: 0.1608
Epoch 8/20
43/43 [==============================] - 2s 43ms/step - loss: 0.0330 - m

In [168]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [169]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_STAT_single_attention_balanced Accuracy:  0.561525129982669 MSE:  0.03476576609587698 UAR:  0.3544130253569821 Recall:  N/A Precision:  N/A F1:  0.36654019677001815


(0.561525129982669,
 0.03476576609587698,
 0.3544130253569821,
 'N/A',
 'N/A',
 0.36654019677001815)

In [170]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [171]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [172]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_STAT_single_attention_balanced_best Accuracy:  0.5927209705372617 MSE:  0.03347231848271875 UAR:  0.3198743675412443 Recall:  N/A Precision:  N/A F1:  0.3066718194009703


(0.5927209705372617,
 0.03347231848271875,
 0.3198743675412443,
 'N/A',
 'N/A',
 0.3066718194009703)

##### Self attention

In [173]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_4_STAT_self_attention_balanced


In [174]:
CONCATENATE_STAT = True

In [175]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5474, 128, 3840) (5474,)
(1731, 128, 3840) (1731,)


In [176]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3840)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 3840)           1474560   ['image_set[0][0]']           
                                                          0                                       
                                                                                                  
 value (Dense)               (None, None, 3840)           1474560   ['image_set[0][0]']           
                                                          0                                       
                                                                                           

In [177]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 10s 215ms/step - loss: 0.0813 - mae: 0.2015 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 2/20
43/43 [==============================] - 7s 168ms/step - loss: 0.0800 - mae: 0.1984 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 3/20
43/43 [==============================] - 7s 168ms/step - loss: 0.0800 - mae: 0.1984 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 4/20
43/43 [==============================] - 7s 169ms/step - loss: 0.0800 - mae: 0.1984 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 5/20
43/43 [==============================] - 7s 168ms/step - loss: 0.0800 - mae: 0.1984 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 6/20
43/43 [==============================] - 7s 169ms/step - loss: 0.0800 - mae: 0.1984 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 7/20
43/43 [==============================] - 7s 167ms/step - loss: 0.0800 - mae: 0.1984 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 8/20
43/43 [==============================] - 7s 168ms/step - loss: 0

In [178]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [179]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_STAT_self_attention_balanced Accuracy:  0.4662045060658579 MSE:  0.08048682842287694 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048682842287694,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

In [180]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [181]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_813884/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [182]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_4_STAT_self_attention_balanced_best Accuracy:  0.4662045060658579 MSE:  0.08048682842287694 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048682842287694,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

## MobileNet_7

In [25]:
base_model_key = 'mobilenet_7.h5'

In [26]:
ENGAGE_WILD_DATASET, BIN_CLASSIFICATION, N_CLASSES, DATASET_NAME, BASE_DATASET_DIR, ext = define_dataset(False, False)
video2label, labels_list = get_video2label()
print(labels_list)
print(video2label)

['very distracted', 'distracted', 'engaged', 'very engaged']
{'1100011002': 0.66, '1100011003': 0.66, '1100011004': 1.0, '1100011005': 1.0, '1100011006': 1.0, '1100011007': 0.66, '1100011008': 1.0, '1100011009': 0.66, '1100011010': 1.0, '1100011011': 1.0, '1100011012': 0.66, '1100011013': 1.0, '1100011014': 1.0, '1100011015': 1.0, '1100011016': 1.0, '1100011017': 1.0, '1100011018': 1.0, '1100011019': 1.0, '1100011020': 1.0, '1100011021': 1.0, '1100011022': 1.0, '1100011023': 1.0, '1100011025': 1.0, '1100011026': 1.0, '1100011027': 1.0, '1100011028': 1.0, '1100011029': 1.0, '1100011031': 1.0, '1100011032': 1.0, '1100011034': 1.0, '1100011035': 1.0, '1100011037': 1.0, '1100011038': 1.0, '1100011040': 0.66, '1100011046': 1.0, '1100011047': 1.0, '1100011048': 0.66, '1100011049': 1.0, '1100011050': 1.0, '1100011051': 1.0, '1100011052': 1.0, '1100011053': 1.0, '1100011054': 1.0, '1100011055': 1.0, '1100011056': 1.0, '1100011057': 1.0, '1100011058': 1.0, '1100011059': 1.0, '1100011060': 1.0, 

### Std

In [27]:
stat_func = np.std
stat_name = "std"

#### Traditional split

In [28]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
val_pickle = FEATURES_DIR + 'val_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)
filename2features_val = load_features(val_pickle)

In [29]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))
print("Val: ", len(filename2features_val))

Train:  5482
Test:  1723
Val:  1720


In [30]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))
print("val:")
print_classes(get_labels(filename2features_val))

train:
0.0 34/5482: 0.6202116016052536%
0.33 214/5482: 3.903684786574243%
0.66 2649/5482: 48.32178037212696%
1.0 2585/5482: 47.15432323969354%
test:
0.0 4/1723: 0.2321532211259431%
0.33 81/1723: 4.7011027278003485%
0.66 861/1723: 49.97098084735926%
1.0 777/1723: 45.095763203714455%
val:
0.0 23/1720: 1.3372093023255813%
0.33 160/1720: 9.30232558139535%
0.66 912/1720: 53.02325581395349%
1.0 625/1720: 36.33720930232558%


##### Single attention

In [31]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

mobilenet_7.h5_DAiSEE_4_std_single_attention_traditional


In [32]:
CONCATENATE_STAT = True

In [33]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2048) (5482,)
(1723, 128, 2048) (1723,)
(1720, 128, 2048) (1720,)


In [34]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

2024-08-05 09:22:57.873860: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38431 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:11:00.0, compute capability: 8.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                              

In [35]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20


2024-08-05 09:23:04.989567: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f9b0003e950 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-05 09:23:04.989590: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB, Compute Capability 8.0
2024-08-05 09:23:04.994404: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-05 09:23:05.027287: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-08-05 09:23:05.078870: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


43/43 [==============================] - 5s 71ms/step - loss: 0.0664 - mae: 0.1934 - val_loss: 0.0591 - val_mae: 0.2072
Epoch 2/20
43/43 [==============================] - 1s 24ms/step - loss: 0.0376 - mae: 0.1689 - val_loss: 0.0504 - val_mae: 0.1872
Epoch 3/20
43/43 [==============================] - 1s 25ms/step - loss: 0.0345 - mae: 0.1596 - val_loss: 0.0501 - val_mae: 0.1863
Epoch 4/20
43/43 [==============================] - 1s 24ms/step - loss: 0.0338 - mae: 0.1575 - val_loss: 0.0521 - val_mae: 0.1888
Epoch 5/20
43/43 [==============================] - 1s 24ms/step - loss: 0.0334 - mae: 0.1550 - val_loss: 0.0483 - val_mae: 0.1812
Epoch 6/20
43/43 [==============================] - 1s 24ms/step - loss: 0.0326 - mae: 0.1533 - val_loss: 0.0469 - val_mae: 0.1788
Epoch 7/20
43/43 [==============================] - 1s 24ms/step - loss: 0.0322 - mae: 0.1518 - val_loss: 0.0473 - val_mae: 0.1784
Epoch 8/20
43/43 [==============================] - 1s 24ms/step - loss: 0.0321 - mae: 0.1519 

In [36]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [37]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_DAiSEE_4_std_single_attention_traditional Accuracy:  0.5606500290191526 MSE:  0.04404884499239516 UAR:  0.3336954841019881 Recall:  N/A Precision:  N/A F1:  0.3425016395849529


(0.5606500290191526,
 0.04404884499239516,
 0.3336954841019881,
 'N/A',
 'N/A',
 0.3425016395849529)

In [38]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [39]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [40]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_DAiSEE_4_std_single_attention_traditional_best Accuracy:  0.5449796865931514 MSE:  0.03844683204704938 UAR:  0.28950466145588094 Recall:  N/A Precision:  N/A F1:  0.2780953955832553


(0.5449796865931514,
 0.03844683204704938,
 0.28950466145588094,
 'N/A',
 'N/A',
 0.2780953955832553)

##### Self attention

In [41]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

mobilenet_7.h5_DAiSEE_4_std_self_attention_traditional


In [42]:
CONCATENATE_STAT = True

In [43]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2048) (5482,)
(1723, 128, 2048) (1723,)
(1720, 128, 2048) (1720,)


In [44]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 attention (Attention)       (None, None, 2048)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [45]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 5s 90ms/step - loss: 0.0818 - mae: 0.2007 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 2/20
43/43 [==============================] - 3s 61ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 3/20
43/43 [==============================] - 3s 62ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 4/20
43/43 [==============================] - 3s 63ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 5/20
43/43 [==============================] - 3s 62ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 6/20
43/43 [==============================] - 3s 62ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 7/20
43/43 [==============================] - 3s 62ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 8/20
43/43 [==============================] - 3s 61ms/step - loss: 0.0796 - m

In [46]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [47]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_DAiSEE_4_std_self_attention_traditional Accuracy:  0.4509576320371445 MSE:  0.08119123621590248 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.1554


(0.4509576320371445, 0.08119123621590248, 0.25, 'N/A', 'N/A', 0.1554)

In [48]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [49]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [50]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_DAiSEE_4_std_self_attention_traditional_best Accuracy:  0.4509576320371445 MSE:  0.08119123621590248 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.1554


(0.4509576320371445, 0.08119123621590248, 0.25, 'N/A', 'N/A', 0.1554)

#### Balanced split

In [51]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [52]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  5482
Test:  1723


In [53]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))

train:
0.0 34/5482: 0.6202116016052536%
0.33 214/5482: 3.903684786574243%
0.66 2649/5482: 48.32178037212696%
1.0 2585/5482: 47.15432323969354%
test:
0.0 4/1723: 0.2321532211259431%
0.33 81/1723: 4.7011027278003485%
0.66 861/1723: 49.97098084735926%
1.0 777/1723: 45.095763203714455%


In [54]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.76)

(5482,) (5482,)
(1723,) (1723,)
(7205,) (7205,)
[0.   0.33 0.66 1.  ] [  38  295 3510 3362]
7205
5474
1731


In [55]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))

train:
0.0 28/5474: 0.5115089514066496%
0.33 224/5474: 4.092071611253197%
0.66 2667/5474: 48.72122762148338%
1.0 2555/5474: 46.67519181585678%
test:
0.0 10/1731: 0.5777007510109763%
0.33 71/1731: 4.101675332177932%
0.66 843/1731: 48.7001733102253%
1.0 807/1731: 46.62045060658579%


##### Single attention

In [56]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

mobilenet_7.h5_DAiSEE_4_std_single_attention_balanced


In [57]:
CONCATENATE_STAT = True

In [58]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5474, 128, 2048) (5474,)
(1731, 128, 2048) (1731,)


In [59]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [60]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 3s 60ms/step - loss: 0.0524 - mae: 0.1871 - val_loss: 0.0376 - val_mae: 0.1724
Epoch 2/20
43/43 [==============================] - 1s 24ms/step - loss: 0.0364 - mae: 0.1673 - val_loss: 0.0351 - val_mae: 0.1613
Epoch 3/20
43/43 [==============================] - 1s 24ms/step - loss: 0.0346 - mae: 0.1617 - val_loss: 0.0341 - val_mae: 0.1600
Epoch 4/20
43/43 [==============================] - 1s 25ms/step - loss: 0.0338 - mae: 0.1590 - val_loss: 0.0341 - val_mae: 0.1600
Epoch 5/20
43/43 [==============================] - 1s 24ms/step - loss: 0.0332 - mae: 0.1570 - val_loss: 0.0340 - val_mae: 0.1598
Epoch 6/20
43/43 [==============================] - 1s 24ms/step - loss: 0.0338 - mae: 0.1573 - val_loss: 0.0360 - val_mae: 0.1634
Epoch 7/20
43/43 [==============================] - 1s 25ms/step - loss: 0.0329 - mae: 0.1552 - val_loss: 0.0342 - val_mae: 0.1591
Epoch 8/20
43/43 [==============================] - 1s 25ms/step - loss: 0.0324 - m

In [61]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [62]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_DAiSEE_4_std_single_attention_balanced Accuracy:  0.5517042172154824 MSE:  0.034207547604454616 UAR:  0.3185099808036505 Recall:  N/A Precision:  N/A F1:  0.3051872322134036


(0.5517042172154824,
 0.034207547604454616,
 0.3185099808036505,
 'N/A',
 'N/A',
 0.3051872322134036)

In [63]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [64]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [65]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_DAiSEE_4_std_single_attention_balanced_best Accuracy:  0.5788561525129983 MSE:  0.03347769755234969 UAR:  0.3194045051019359 Recall:  N/A Precision:  N/A F1:  0.30678216509304873


(0.5788561525129983,
 0.03347769755234969,
 0.3194045051019359,
 'N/A',
 'N/A',
 0.30678216509304873)

##### Self attention

In [66]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

mobilenet_7.h5_DAiSEE_4_std_self_attention_balanced


In [67]:
CONCATENATE_STAT = True

In [68]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5474, 128, 2048) (5474,)
(1731, 128, 2048) (1731,)


In [69]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 attention_1 (Attention)     (None, None, 2048)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [70]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 5s 99ms/step - loss: 0.0799 - mae: 0.1991 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 2/20
43/43 [==============================] - 3s 62ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 3/20
43/43 [==============================] - 3s 61ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 4/20
43/43 [==============================] - 3s 61ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 5/20
43/43 [==============================] - 3s 61ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 6/20
43/43 [==============================] - 3s 61ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 7/20
43/43 [==============================] - 3s 61ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 8/20
43/43 [==============================] - 3s 62ms/step - loss: 0.0798 - m

In [71]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [72]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_DAiSEE_4_std_self_attention_balanced Accuracy:  0.4662045060658579 MSE:  0.08048682842287694 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048682842287694,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

In [73]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [74]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [75]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_DAiSEE_4_std_self_attention_balanced_best Accuracy:  0.4662045060658579 MSE:  0.08048682842287694 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048682842287694,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

### Max

In [76]:
stat_func = np.max
stat_name = "max"

#### Traditional split

In [77]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
val_pickle = FEATURES_DIR + 'val_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)
filename2features_val = load_features(val_pickle)

In [78]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))
print("Val: ", len(filename2features_val))

Train:  5482
Test:  1723
Val:  1720


In [79]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))
print("val:")
print_classes(get_labels(filename2features_val))

train:
0.0 34/5482: 0.6202116016052536%
0.33 214/5482: 3.903684786574243%
0.66 2649/5482: 48.32178037212696%
1.0 2585/5482: 47.15432323969354%
test:
0.0 4/1723: 0.2321532211259431%
0.33 81/1723: 4.7011027278003485%
0.66 861/1723: 49.97098084735926%
1.0 777/1723: 45.095763203714455%
val:
0.0 23/1720: 1.3372093023255813%
0.33 160/1720: 9.30232558139535%
0.66 912/1720: 53.02325581395349%
1.0 625/1720: 36.33720930232558%


##### Single attention

In [80]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

mobilenet_7.h5_DAiSEE_4_max_single_attention_traditional


In [81]:
CONCATENATE_STAT = True

In [82]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2048) (5482,)
(1723, 128, 2048) (1723,)
(1720, 128, 2048) (1720,)


In [83]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [84]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 3s 51ms/step - loss: 0.0802 - mae: 0.1985 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 2/20
43/43 [==============================] - 1s 24ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 3/20
43/43 [==============================] - 1s 25ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 4/20
43/43 [==============================] - 1s 27ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 5/20
43/43 [==============================] - 1s 24ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 6/20
43/43 [==============================] - 1s 24ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 7/20
43/43 [==============================] - 1s 25ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 8/20
43/43 [==============================] - 1s 25ms/step - loss: 0.0796 - m

In [85]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [86]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_DAiSEE_4_max_single_attention_traditional Accuracy:  0.4509576320371445 MSE:  0.08119123621590248 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.1554


(0.4509576320371445, 0.08119123621590248, 0.25, 'N/A', 'N/A', 0.1554)

In [87]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [88]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [89]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_DAiSEE_4_max_single_attention_traditional_best Accuracy:  0.4509576320371445 MSE:  0.08119123621590248 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.1554


(0.4509576320371445, 0.08119123621590248, 0.25, 'N/A', 'N/A', 0.1554)

##### Self attention

In [90]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

mobilenet_7.h5_DAiSEE_4_max_self_attention_traditional


In [91]:
CONCATENATE_STAT = True

In [92]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2048) (5482,)
(1723, 128, 2048) (1723,)
(1720, 128, 2048) (1720,)


In [93]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 attention_2 (Attention)     (None, None, 2048)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [94]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 5s 99ms/step - loss: 0.0797 - mae: 0.1976 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 2/20
43/43 [==============================] - 3s 62ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 3/20
43/43 [==============================] - 3s 61ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 4/20
43/43 [==============================] - 3s 61ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 5/20
43/43 [==============================] - 3s 66ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 6/20
43/43 [==============================] - 3s 61ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 7/20
43/43 [==============================] - 3s 61ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 8/20
43/43 [==============================] - 3s 62ms/step - loss: 0.0796 - m

In [95]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [96]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_DAiSEE_4_max_self_attention_traditional Accuracy:  0.4509576320371445 MSE:  0.08119123621590248 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.1554


(0.4509576320371445, 0.08119123621590248, 0.25, 'N/A', 'N/A', 0.1554)

In [97]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [98]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [99]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_DAiSEE_4_max_self_attention_traditional_best Accuracy:  0.4509576320371445 MSE:  0.08119123621590248 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.1554


(0.4509576320371445, 0.08119123621590248, 0.25, 'N/A', 'N/A', 0.1554)

#### Balanced split

In [100]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [101]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  5482
Test:  1723


In [102]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))

train:
0.0 34/5482: 0.6202116016052536%
0.33 214/5482: 3.903684786574243%
0.66 2649/5482: 48.32178037212696%
1.0 2585/5482: 47.15432323969354%
test:
0.0 4/1723: 0.2321532211259431%
0.33 81/1723: 4.7011027278003485%
0.66 861/1723: 49.97098084735926%
1.0 777/1723: 45.095763203714455%


In [103]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.76)

(5482,) (5482,)
(1723,) (1723,)
(7205,) (7205,)
[0.   0.33 0.66 1.  ] [  38  295 3510 3362]
7205
5474
1731


In [104]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))

train:
0.0 28/5474: 0.5115089514066496%
0.33 224/5474: 4.092071611253197%
0.66 2667/5474: 48.72122762148338%
1.0 2555/5474: 46.67519181585678%
test:
0.0 10/1731: 0.5777007510109763%
0.33 71/1731: 4.101675332177932%
0.66 843/1731: 48.7001733102253%
1.0 807/1731: 46.62045060658579%


##### Single attention

In [105]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

mobilenet_7.h5_DAiSEE_4_max_single_attention_balanced


In [106]:
CONCATENATE_STAT = True

In [107]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5474, 128, 2048) (5474,)
(1731, 128, 2048) (1731,)


In [108]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [109]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 3s 52ms/step - loss: 0.0802 - mae: 0.1997 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 2/20
43/43 [==============================] - 1s 25ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 3/20
43/43 [==============================] - 1s 24ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 4/20
43/43 [==============================] - 1s 24ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 5/20
43/43 [==============================] - 1s 24ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 6/20
43/43 [==============================] - 1s 24ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 7/20
43/43 [==============================] - 1s 25ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 8/20
43/43 [==============================] - 1s 25ms/step - loss: 0.0798 - m

In [110]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [111]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_DAiSEE_4_max_single_attention_balanced Accuracy:  0.4662045060658579 MSE:  0.08048682837604716 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048682837604716,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

In [112]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [113]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [114]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_DAiSEE_4_max_single_attention_balanced_best Accuracy:  0.4662045060658579 MSE:  0.08048682837604716 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048682837604716,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

##### Self attention

In [115]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

mobilenet_7.h5_DAiSEE_4_max_self_attention_balanced


In [116]:
CONCATENATE_STAT = True

In [117]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5474, 128, 2048) (5474,)
(1731, 128, 2048) (1731,)


In [118]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 attention_3 (Attention)     (None, None, 2048)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [119]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 5s 92ms/step - loss: 0.0805 - mae: 0.1997 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 2/20
43/43 [==============================] - 3s 62ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 3/20
43/43 [==============================] - 3s 61ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 4/20
43/43 [==============================] - 3s 62ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 5/20
43/43 [==============================] - 3s 61ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 6/20
43/43 [==============================] - 3s 62ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 7/20
43/43 [==============================] - 3s 62ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 8/20
43/43 [==============================] - 3s 61ms/step - loss: 0.0798 - m

In [120]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [121]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_DAiSEE_4_max_self_attention_balanced Accuracy:  0.4662045060658579 MSE:  0.08048682842287694 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048682842287694,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

In [122]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [123]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [124]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_DAiSEE_4_max_self_attention_balanced_best Accuracy:  0.4662045060658579 MSE:  0.08048682842287694 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048682842287694,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

### STAT

In [125]:
stat_func = compute_descriptor
stat_name = "STAT"

#### Traditional split

In [126]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
val_pickle = FEATURES_DIR + 'val_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)
filename2features_val = load_features(val_pickle)

In [127]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))
print("Val: ", len(filename2features_val))

Train:  5482
Test:  1723
Val:  1720


In [128]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))
print("val:")
print_classes(get_labels(filename2features_val))

train:
0.0 34/5482: 0.6202116016052536%
0.33 214/5482: 3.903684786574243%
0.66 2649/5482: 48.32178037212696%
1.0 2585/5482: 47.15432323969354%
test:
0.0 4/1723: 0.2321532211259431%
0.33 81/1723: 4.7011027278003485%
0.66 861/1723: 49.97098084735926%
1.0 777/1723: 45.095763203714455%
val:
0.0 23/1720: 1.3372093023255813%
0.33 160/1720: 9.30232558139535%
0.66 912/1720: 53.02325581395349%
1.0 625/1720: 36.33720930232558%


##### Single attention

In [129]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

mobilenet_7.h5_DAiSEE_4_STAT_single_attention_traditional


In [130]:
CONCATENATE_STAT = True

In [131]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 3072) (5482,)
(1723, 128, 3072) (1723,)
(1720, 128, 3072) (1720,)


In [132]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3072)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3073      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                            

In [133]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 4s 72ms/step - loss: 0.0809 - mae: 0.1998 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 2/20
43/43 [==============================] - 1s 34ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 3/20
43/43 [==============================] - 1s 34ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 4/20
43/43 [==============================] - 2s 41ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 5/20
43/43 [==============================] - 1s 34ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 6/20
43/43 [==============================] - 2s 42ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 7/20
43/43 [==============================] - 1s 35ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 8/20
43/43 [==============================] - 1s 34ms/step - loss: 0.0796 - m

In [134]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [135]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_DAiSEE_4_STAT_single_attention_traditional Accuracy:  0.4509576320371445 MSE:  0.08119122675942898 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.1554


(0.4509576320371445, 0.08119122675942898, 0.25, 'N/A', 'N/A', 0.1554)

In [136]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [137]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [138]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_DAiSEE_4_STAT_single_attention_traditional_best Accuracy:  0.4509576320371445 MSE:  0.08119122483188343 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.1554


(0.4509576320371445, 0.08119122483188343, 0.25, 'N/A', 'N/A', 0.1554)

##### Self attention

In [139]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

mobilenet_7.h5_DAiSEE_4_STAT_self_attention_traditional


In [140]:
CONCATENATE_STAT = True

In [141]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 3072) (5482,)
(1723, 128, 3072) (1723,)
(1720, 128, 3072) (1720,)


In [142]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3072)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 3072)           9437184   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 3072)           9437184   ['image_set[0][0]']           
                                                                                                  
 attention_4 (Attention)     (None, None, 3072)           0         ['query[0][0]',               
                                                                     'value[0][0]']         

In [143]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 8s 155ms/step - loss: 0.0805 - mae: 0.1990 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 2/20
43/43 [==============================] - 5s 116ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 3/20
43/43 [==============================] - 5s 116ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 4/20
43/43 [==============================] - 5s 117ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 5/20
43/43 [==============================] - 5s 115ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 6/20
43/43 [==============================] - 5s 116ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 7/20
43/43 [==============================] - 5s 119ms/step - loss: 0.0796 - mae: 0.1967 - val_loss: 0.1164 - val_mae: 0.2560
Epoch 8/20
43/43 [==============================] - 5s 116ms/step - loss: 0.

In [144]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [145]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_DAiSEE_4_STAT_self_attention_traditional Accuracy:  0.4509576320371445 MSE:  0.08119123621590248 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.1554


(0.4509576320371445, 0.08119123621590248, 0.25, 'N/A', 'N/A', 0.1554)

In [146]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [147]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [148]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_DAiSEE_4_STAT_self_attention_traditional_best Accuracy:  0.4509576320371445 MSE:  0.08119123621590248 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.1554


(0.4509576320371445, 0.08119123621590248, 0.25, 'N/A', 'N/A', 0.1554)

#### Balanced split

In [149]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [150]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  5482
Test:  1723


In [151]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))

train:
0.0 34/5482: 0.6202116016052536%
0.33 214/5482: 3.903684786574243%
0.66 2649/5482: 48.32178037212696%
1.0 2585/5482: 47.15432323969354%
test:
0.0 4/1723: 0.2321532211259431%
0.33 81/1723: 4.7011027278003485%
0.66 861/1723: 49.97098084735926%
1.0 777/1723: 45.095763203714455%


In [152]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.76)

(5482, 2048) (5482,)
(1723, 2048) (1723,)
(7205, 2048) (7205,)
[0.   0.33 0.66 1.  ] [  38  295 3510 3362]
7205
5474
1731


In [153]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))

train:
0.0 28/5474: 0.5115089514066496%
0.33 224/5474: 4.092071611253197%
0.66 2667/5474: 48.72122762148338%
1.0 2555/5474: 46.67519181585678%
test:
0.0 10/1731: 0.5777007510109763%
0.33 71/1731: 4.101675332177932%
0.66 843/1731: 48.7001733102253%
1.0 807/1731: 46.62045060658579%


##### Single attention

In [154]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

mobilenet_7.h5_DAiSEE_4_STAT_single_attention_balanced


In [155]:
CONCATENATE_STAT = True

In [156]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5474, 128, 3072) (5474,)
(1731, 128, 3072) (1731,)


In [157]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3072)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3073      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                           

In [158]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 4s 73ms/step - loss: 0.0799 - mae: 0.1992 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 2/20
43/43 [==============================] - 1s 34ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 3/20
43/43 [==============================] - 1s 34ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 4/20
43/43 [==============================] - 2s 41ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 5/20
43/43 [==============================] - 1s 34ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 6/20
43/43 [==============================] - 1s 35ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 7/20
43/43 [==============================] - 1s 35ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 8/20
43/43 [==============================] - 1s 35ms/step - loss: 0.0798 - m

In [159]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [160]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_DAiSEE_4_STAT_single_attention_balanced Accuracy:  0.4662045060658579 MSE:  0.08048561652532543 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048561652532543,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

In [161]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [162]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [163]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_DAiSEE_4_STAT_single_attention_balanced_best Accuracy:  0.4662045060658579 MSE:  0.08048561652532543 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048561652532543,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

##### Self attention

In [164]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

mobilenet_7.h5_DAiSEE_4_STAT_self_attention_balanced


In [165]:
CONCATENATE_STAT = True

In [166]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5474, 128, 3072) (5474,)
(1731, 128, 3072) (1731,)


In [167]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3072)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 3072)           9437184   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 3072)           9437184   ['image_set[0][0]']           
                                                                                                  
 attention_5 (Attention)     (None, None, 3072)           0         ['query[0][0]',               
                                                                     'value[0][0]']        

In [168]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 8s 155ms/step - loss: 0.0802 - mae: 0.1996 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 2/20
43/43 [==============================] - 5s 116ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 3/20
43/43 [==============================] - 5s 116ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 4/20
43/43 [==============================] - 5s 117ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 5/20
43/43 [==============================] - 5s 116ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 6/20
43/43 [==============================] - 5s 116ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 7/20
43/43 [==============================] - 5s 116ms/step - loss: 0.0798 - mae: 0.1982 - val_loss: 0.0805 - val_mae: 0.1988
Epoch 8/20
43/43 [==============================] - 5s 117ms/step - loss: 0.

In [169]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [170]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_DAiSEE_4_STAT_self_attention_balanced Accuracy:  0.4662045060658579 MSE:  0.08048682842287694 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048682842287694,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

In [171]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [172]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [173]:
print_results(pred, test_labels, table_name=TABLE_NAME, metric_name=metric_name)

Metric_name:  mobilenet_7.h5_DAiSEE_4_STAT_self_attention_balanced_best Accuracy:  0.4662045060658579 MSE:  0.08048682842287694 UAR:  0.25 Recall:  N/A Precision:  N/A F1:  0.15898345153664303


(0.4662045060658579,
 0.08048682842287694,
 0.25,
 'N/A',
 'N/A',
 0.15898345153664303)

# DAiSEE dataset (2 classes)

In [174]:
FEATURES_DIR = DATA_DIR + 'features_DAiSEE/'
WEIGHTS_DIR = DATA_DIR + 'weights_DAiSEE/'
TABLE_NAME = '02_DAiSEE_2_classes.xlsx'

## enet_b0_8_best_afew

In [175]:
base_model_key = 'enet_b0_8_best_afew.pt'

In [181]:
ENGAGE_WILD_DATASET, BIN_CLASSIFICATION, N_CLASSES, DATASET_NAME, BASE_DATASET_DIR, ext = define_dataset(False, True)
video2label, labels_list = get_video2label()
print(labels_list)
print(video2label)

['engaged', 'distracted']
{'1100011002': 0, '1100011003': 0, '1100011004': 0, '1100011005': 0, '1100011006': 0, '1100011007': 0, '1100011008': 0, '1100011009': 0, '1100011010': 0, '1100011011': 0, '1100011012': 0, '1100011013': 0, '1100011014': 0, '1100011015': 0, '1100011016': 0, '1100011017': 0, '1100011018': 0, '1100011019': 0, '1100011020': 0, '1100011021': 0, '1100011022': 0, '1100011023': 0, '1100011025': 0, '1100011026': 0, '1100011027': 0, '1100011028': 0, '1100011029': 0, '1100011031': 0, '1100011032': 0, '1100011034': 0, '1100011035': 0, '1100011037': 0, '1100011038': 0, '1100011040': 0, '1100011046': 0, '1100011047': 0, '1100011048': 0, '1100011049': 0, '1100011050': 0, '1100011051': 0, '1100011052': 0, '1100011053': 0, '1100011054': 0, '1100011055': 0, '1100011056': 0, '1100011057': 0, '1100011058': 0, '1100011059': 0, '1100011060': 0, '1100011062': 0, '1100011063': 0, '1100011064': 0, '1100011066': 0, '1100011067': 0, '1100011068': 0, '1100011069': 0, '1100011070': 0, '110

### Std

In [182]:
stat_func = np.std
stat_name = "std"

#### Traditional split

In [185]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
val_pickle = FEATURES_DIR + 'val_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)
filename2features_val = load_features(val_pickle)

In [186]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))
print("Val: ", len(filename2features_val))

Train:  5482
Test:  1723
Val:  1720


In [187]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))
print("val:")
print_classes(get_labels(filename2features_val))

train:
0 5234/5482: 95.4761036118205%
1 248/5482: 4.523896388179496%
test:
0 1638/1723: 95.06674405107371%
1 85/1723: 4.933255948926291%
val:
0 1537/1720: 89.36046511627907%
1 183/1720: 10.63953488372093%


##### Single attention

In [188]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_traditional


In [189]:
CONCATENATE_STAT = True

In [190]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2560) (5482,)
(1723, 128, 2560) (1723,)
(1720, 128, 2560) (1720,)


In [191]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                           

In [192]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 5s 68ms/step - loss: 0.2087 - acc: 0.9409 - auc: 0.5783 - binary_accuracy: 0.9409 - recall: 0.0040 - precision: 0.0128 - val_loss: 0.3481 - val_acc: 0.8936 - val_auc: 0.6314 - val_binary_accuracy: 0.8936 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 2/20
43/43 [==============================] - 1s 31ms/step - loss: 0.1639 - acc: 0.9548 - auc: 0.7567 - binary_accuracy: 0.9548 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.3484 - val_acc: 0.8936 - val_auc: 0.6606 - val_binary_accuracy: 0.8936 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 3/20
43/43 [==============================] - 1s 30ms/step - loss: 0.1580 - acc: 0.9548 - auc: 0.7838 - binary_accuracy: 0.9548 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.3312 - val_acc: 0.8936 - val_auc: 0.6682 - val_binary_accuracy: 0.8936 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 4/20
43/43 [==============================] - 1s 30

In [193]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [194]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_traditional Accuracy:  0.8961114335461404 MSE:  0.10388856645385955 UAR:  0.6497737556561086 Recall:  0.3764705882352941 Precision:  0.20253164556962025 F1:  0.2633744855967078
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_traditional Accuracy:  0.938479396401625 MSE:  0.061520603598374926 UAR:  0.5605149752208576 Recall:  0.1411764705882353 Precision:  0.26666666666666666 F1:  0.1846153846153846
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_traditional Accuracy:  0.9454439930354034 MSE:  0.054556006964596636 UAR:  0.5251382604323781 Recall:  0.058823529411764705 Precision:  0.2631578947368421 F1:  0.09615384615384615
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_traditional Accuracy:  0.9500870574579222 MSE:  0.04991294254207777 UAR:  0.5164260576025282 Recall:  0.03529411764705882 Precision:  0.42857142857142855 F1:  0.06521739130434782
Metric_name:  enet_

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/pyth

In [195]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [196]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [197]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_traditional_best Accuracy:  0.8786999419616948 MSE:  0.12130005803830528 UAR:  0.6740788623141565 Recall:  0.4470588235294118 Precision:  0.19 F1:  0.26666666666666666
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_traditional_best Accuracy:  0.9338363319791062 MSE:  0.0661636680208938 UAR:  0.619421101774043 Recall:  0.27058823529411763 Precision:  0.30666666666666664 F1:  0.2875
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_traditional_best Accuracy:  0.9489262913522926 MSE:  0.05107370864770749 UAR:  0.5771636859872155 Recall:  0.16470588235294117 Precision:  0.45161290322580644 F1:  0.2413793103448276
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_traditional_best Accuracy:  0.9518282066163668 MSE:  0.0481717933836332 UAR:  0.5173418085182792 Recall:  0.03529411764705882 Precision:  0.75 F1:  0.06741573033707865
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/pyth

##### Self attention

In [198]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional


In [199]:
CONCATENATE_STAT = True

In [200]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2560) (5482,)
(1723, 128, 2560) (1723,)
(1720, 128, 2560) (1720,)


In [201]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_6 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']        

In [202]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 7s 127ms/step - loss: 0.3520 - acc: 0.9435 - auc: 0.5372 - binary_accuracy: 0.9435 - recall_1: 0.0121 - precision_1: 0.0441 - val_loss: 0.3655 - val_acc: 0.8936 - val_auc: 0.6054 - val_binary_accuracy: 0.8936 - val_recall_1: 0.0000e+00 - val_precision_1: 0.0000e+00
Epoch 2/20
43/43 [==============================] - 4s 89ms/step - loss: 0.1676 - acc: 0.9544 - auc: 0.7332 - binary_accuracy: 0.9544 - recall_1: 0.0081 - precision_1: 0.3333 - val_loss: 0.4156 - val_acc: 0.8936 - val_auc: 0.6291 - val_binary_accuracy: 0.8936 - val_recall_1: 0.0000e+00 - val_precision_1: 0.0000e+00
Epoch 3/20
43/43 [==============================] - 4s 89ms/step - loss: 0.1589 - acc: 0.9533 - auc: 0.7800 - binary_accuracy: 0.9533 - recall_1: 0.0081 - precision_1: 0.1667 - val_loss: 0.3692 - val_acc: 0.8907 - val_auc: 0.6405 - val_binary_accuracy: 0.8907 - val_recall_1: 0.0328 - val_precision_1: 0.3529
Epoch 4/20
43/43 [==============================] - 4s 9

In [203]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [204]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional Accuracy:  0.8415554265815438 MSE:  0.1584445734184562 UAR:  0.6043489190548015 Recall:  0.3411764705882353 Precision:  0.11788617886178862 F1:  0.17522658610271902
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional Accuracy:  0.930354033662217 MSE:  0.06964596633778293 UAR:  0.5450872656755009 Recall:  0.11764705882352941 Precision:  0.18181818181818182 F1:  0.14285714285714285
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional Accuracy:  0.9437028438769588 MSE:  0.05629715612304121 UAR:  0.5409538174244057 Recall:  0.09411764705882353 Precision:  0.2857142857142857 F1:  0.1415929203539823
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional Accuracy:  0.947185142193848 MSE:  0.05281485780615206 UAR:  0.526054011348129 Recall:  0.058823529411764705 Precision:  0.3125 F1:  0.09900990099009901
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE

In [205]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [206]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [207]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.8340104468949506 MSE:  0.16598955310504934 UAR:  0.6282661782661783 Recall:  0.4 Precision:  0.12639405204460966 F1:  0.192090395480226
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.9297736506094022 MSE:  0.0702263493905978 UAR:  0.5949759390935861 Recall:  0.2235294117647059 Precision:  0.25675675675675674 F1:  0.2389937106918239
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.9477655252466628 MSE:  0.0522344747533372 UAR:  0.5653989801048624 Recall:  0.1411764705882353 Precision:  0.41379310344827586 F1:  0.21052631578947367
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.9524085896691816 MSE:  0.04759141033081834 UAR:  0.5566867772750126 Recall:  0.11764705882352941 Precision:  0.5882352941176471 F1:  0.19607843137254902
Metric_name:  enet_b0_8_best

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Balanced split

In [208]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [209]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  5482
Test:  1723


In [210]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 5234/5482: 95.4761036118205%
1 248/5482: 4.523896388179496%
val:
0 1638/1723: 95.06674405107371%
1 85/1723: 4.933255948926291%


In [211]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.76)

(5482,) (5482,)
(1723,) (1723,)
(7205,) (7205,)
[0 1] [6872  333]
7205
5475
1730


In [212]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 5222/5475: 95.37899543378995%
1 253/5475: 4.621004566210046%
val:
0 1650/1730: 95.3757225433526%
1 80/1730: 4.624277456647399%


##### Single attention

In [213]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_balanced


In [214]:
CONCATENATE_STAT = True

In [215]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5475, 128, 2560) (5475,)
(1730, 128, 2560) (1730,)


In [216]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                           

In [217]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 4s 66ms/step - loss: 0.2223 - acc: 0.9315 - auc: 0.5639 - binary_accuracy: 0.9315 - recall_2: 0.0158 - precision_2: 0.0308 - val_loss: 0.1833 - val_acc: 0.9532 - val_auc: 0.6921 - val_binary_accuracy: 0.9532 - val_recall_2: 0.0000e+00 - val_precision_2: 0.0000e+00
Epoch 2/20
43/43 [==============================] - 1s 29ms/step - loss: 0.1685 - acc: 0.9543 - auc: 0.7300 - binary_accuracy: 0.9543 - recall_2: 0.0119 - precision_2: 1.0000 - val_loss: 0.1702 - val_acc: 0.9532 - val_auc: 0.7677 - val_binary_accuracy: 0.9532 - val_recall_2: 0.0000e+00 - val_precision_2: 0.0000e+00
Epoch 3/20
43/43 [==============================] - 1s 30ms/step - loss: 0.1614 - acc: 0.9543 - auc: 0.7606 - binary_accuracy: 0.9543 - recall_2: 0.0158 - precision_2: 0.8000 - val_loss: 0.1628 - val_acc: 0.9532 - val_auc: 0.7720 - val_binary_accuracy: 0.9532 - val_recall_2: 0.0000e+00 - val_precision_2: 0.0000e+00
Epoch 4/20
43/43 [==============================]

In [218]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [219]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_balanced Accuracy:  0.9086705202312139 MSE:  0.09132947976878612 UAR:  0.7023484848484849 Recall:  0.475 Precision:  0.24675324675324675 F1:  0.3247863247863248
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_balanced Accuracy:  0.9421965317919075 MSE:  0.057803468208092484 UAR:  0.6128787878787878 Recall:  0.25 Precision:  0.3333333333333333 F1:  0.28571428571428575
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_balanced Accuracy:  0.9508670520231214 MSE:  0.049132947976878616 UAR:  0.5698484848484848 Recall:  0.15 Precision:  0.41379310344827586 F1:  0.22018348623853212
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_balanced Accuracy:  0.9497109826589596 MSE:  0.050289017341040465 UAR:  0.5335606060606061 Recall:  0.075 Precision:  0.3157894736842105 F1:  0.1212121212121212
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_balanced Accuracy:  

In [220]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [221]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [222]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_balanced_best Accuracy:  0.8971098265895954 MSE:  0.10289017341040463 UAR:  0.7081818181818182 Recall:  0.5 Precision:  0.2247191011235955 F1:  0.31007751937984496
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_balanced_best Accuracy:  0.9416184971098266 MSE:  0.05838150289017341 UAR:  0.624469696969697 Recall:  0.275 Precision:  0.3384615384615385 F1:  0.30344827586206896
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_balanced_best Accuracy:  0.9502890173410404 MSE:  0.04971098265895954 UAR:  0.5873863636363637 Recall:  0.1875 Precision:  0.4166666666666667 F1:  0.2586206896551724
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_balanced_best Accuracy:  0.9514450867052023 MSE:  0.048554913294797684 UAR:  0.5344696969696969 Recall:  0.075 Precision:  0.375 F1:  0.12499999999999999
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_single_attention_balanced_best Ac

##### Self attention

In [223]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced


In [224]:
CONCATENATE_STAT = True

In [225]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5475, 128, 2560) (5475,)
(1730, 128, 2560) (1730,)


In [226]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_7 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']        

In [227]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 7s 125ms/step - loss: 0.3764 - acc: 0.9412 - auc: 0.5374 - binary_accuracy: 0.9412 - recall_3: 0.0277 - precision_3: 0.0843 - val_loss: 0.1875 - val_acc: 0.9526 - val_auc: 0.6787 - val_binary_accuracy: 0.9526 - val_recall_3: 0.0125 - val_precision_3: 0.2500
Epoch 2/20
43/43 [==============================] - 4s 89ms/step - loss: 0.1737 - acc: 0.9540 - auc: 0.7014 - binary_accuracy: 0.9540 - recall_3: 0.0514 - precision_3: 0.5200 - val_loss: 0.1701 - val_acc: 0.9526 - val_auc: 0.7431 - val_binary_accuracy: 0.9526 - val_recall_3: 0.0250 - val_precision_3: 0.3333
Epoch 3/20
43/43 [==============================] - 4s 88ms/step - loss: 0.1627 - acc: 0.9536 - auc: 0.7562 - binary_accuracy: 0.9536 - recall_3: 0.0435 - precision_3: 0.4783 - val_loss: 0.1690 - val_acc: 0.9526 - val_auc: 0.7421 - val_binary_accuracy: 0.9526 - val_recall_3: 0.0000e+00 - val_precision_3: 0.0000e+00
Epoch 4/20
43/43 [==============================] - 4s 88ms/step

In [228]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [229]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.9445086705202312 MSE:  0.055491329479768786 UAR:  0.5903030303030303 Recall:  0.2 Precision:  0.3333333333333333 F1:  0.25
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.9520231213872833 MSE:  0.04797687861271676 UAR:  0.5526136363636364 Recall:  0.1125 Precision:  0.42857142857142855 F1:  0.1782178217821782
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.953757225433526 MSE:  0.046242774566473986 UAR:  0.5297348484848485 Recall:  0.0625 Precision:  0.5 F1:  0.1111111111111111
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.954335260115607 MSE:  0.04566473988439306 UAR:  0.5181439393939394 Recall:  0.0375 Precision:  0.6 F1:  0.07058823529411765
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced Accuracy:  0.9531791907514451 MSE:  0.04682080924855491 UAR:  0.505

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [230]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [231]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [232]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.8549132947976879 MSE:  0.14508670520231215 UAR:  0.6979545454545455 Recall:  0.525 Precision:  0.16470588235294117 F1:  0.2507462686567164
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.9260115606936417 MSE:  0.07398843930635839 UAR:  0.6460227272727272 Recall:  0.3375 Precision:  0.2647058823529412 F1:  0.29670329670329665
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.946242774566474 MSE:  0.05375722543352601 UAR:  0.5852651515151515 Recall:  0.1875 Precision:  0.3488372093023256 F1:  0.2439024390243903
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.954335260115607 MSE:  0.04566473988439306 UAR:  0.5716666666666667 Recall:  0.15 Precision:  0.5217391304347826 F1:  0.23300970873786406
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_std_self_attention_balanced_best

### Max

In [233]:
stat_func = np.max
stat_name = "max"

#### Traditional split

In [234]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
val_pickle = FEATURES_DIR + 'val_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)
filename2features_val = load_features(val_pickle)

In [235]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))
print("Val: ", len(filename2features_val))

Train:  5482
Test:  1723
Val:  1720


In [236]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))
print("val:")
print_classes(get_labels(filename2features_val))

train:
0 5234/5482: 95.4761036118205%
1 248/5482: 4.523896388179496%
test:
0 1638/1723: 95.06674405107371%
1 85/1723: 4.933255948926291%
val:
0 1537/1720: 89.36046511627907%
1 183/1720: 10.63953488372093%


##### Single attention

In [237]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_traditional


In [238]:
CONCATENATE_STAT = True

In [239]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2560) (5482,)
(1723, 128, 2560) (1723,)
(1720, 128, 2560) (1720,)


In [240]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_16"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                           

In [241]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 4s 67ms/step - loss: 0.1984 - acc: 0.9528 - auc: 0.6336 - binary_accuracy: 0.9528 - recall_4: 0.0081 - precision_4: 0.1333 - val_loss: 0.3389 - val_acc: 0.8936 - val_auc: 0.6639 - val_binary_accuracy: 0.8936 - val_recall_4: 0.0000e+00 - val_precision_4: 0.0000e+00
Epoch 2/20
43/43 [==============================] - 1s 29ms/step - loss: 0.1617 - acc: 0.9548 - auc: 0.7635 - binary_accuracy: 0.9548 - recall_4: 0.0000e+00 - precision_4: 0.0000e+00 - val_loss: 0.3587 - val_acc: 0.8936 - val_auc: 0.6692 - val_binary_accuracy: 0.8936 - val_recall_4: 0.0000e+00 - val_precision_4: 0.0000e+00
Epoch 3/20
43/43 [==============================] - 1s 30ms/step - loss: 0.1549 - acc: 0.9548 - auc: 0.8004 - binary_accuracy: 0.9548 - recall_4: 0.0040 - precision_4: 0.5000 - val_loss: 0.3423 - val_acc: 0.8936 - val_auc: 0.6735 - val_binary_accuracy: 0.8936 - val_recall_4: 0.0000e+00 - val_precision_4: 0.0000e+00
Epoch 4/20
43/43 [=======================

In [242]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [243]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_traditional Accuracy:  0.9071387115496228 MSE:  0.09286128845037725 UAR:  0.5551856640091934 Recall:  0.16470588235294117 Precision:  0.13592233009708737 F1:  0.14893617021276592
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_traditional Accuracy:  0.942542077771329 MSE:  0.057457922228670924 UAR:  0.5347662141779789 Recall:  0.08235294117647059 Precision:  0.25 F1:  0.12389380530973451
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_traditional Accuracy:  0.9454439930354034 MSE:  0.054556006964596636 UAR:  0.5139840551605258 Recall:  0.03529411764705882 Precision:  0.2 F1:  0.060000000000000005
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_traditional Accuracy:  0.9495066744051074 MSE:  0.05049332559489263 UAR:  0.5161208072972779 Recall:  0.03529411764705882 Precision:  0.375 F1:  0.06451612903225806
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_at

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [244]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [245]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [246]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_traditional_best Accuracy:  0.8340104468949506 MSE:  0.16598955310504934 UAR:  0.6896143072613661 Recall:  0.5294117647058824 Precision:  0.15463917525773196 F1:  0.23936170212765956
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_traditional_best Accuracy:  0.9315147997678468 MSE:  0.06848520023215322 UAR:  0.5624290741937801 Recall:  0.15294117647058825 Precision:  0.22033898305084745 F1:  0.18055555555555555
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_traditional_best Accuracy:  0.9489262913522926 MSE:  0.05107370864770749 UAR:  0.5437010701716585 Recall:  0.09411764705882353 Precision:  0.42105263157894735 F1:  0.15384615384615383
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_traditional_best Accuracy:  0.951247823563552 MSE:  0.04875217643644805 UAR:  0.5058823529411764 Recall:  0.011764705882352941 Precision:  1.0 F1:  0.023255813953488372
Metric_name:

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/pyth

##### Self attention

In [247]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_2_max_self_attention_traditional


In [248]:
CONCATENATE_STAT = True

In [249]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2560) (5482,)
(1723, 128, 2560) (1723,)
(1720, 128, 2560) (1720,)


In [250]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_17"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_8 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']        

In [251]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 7s 126ms/step - loss: 0.5691 - acc: 0.9159 - auc: 0.5587 - binary_accuracy: 0.9159 - recall_5: 0.0726 - precision_5: 0.0723 - val_loss: 0.5374 - val_acc: 0.8750 - val_auc: 0.5344 - val_binary_accuracy: 0.8750 - val_recall_5: 0.0546 - val_precision_5: 0.1923
Epoch 2/20
43/43 [==============================] - 4s 89ms/step - loss: 0.1979 - acc: 0.9497 - auc: 0.6736 - binary_accuracy: 0.9497 - recall_5: 0.0403 - precision_5: 0.2083 - val_loss: 0.3844 - val_acc: 0.8942 - val_auc: 0.6325 - val_binary_accuracy: 0.8942 - val_recall_5: 0.0219 - val_precision_5: 0.5714
Epoch 3/20
43/43 [==============================] - 4s 89ms/step - loss: 0.1687 - acc: 0.9533 - auc: 0.7470 - binary_accuracy: 0.9533 - recall_5: 0.0323 - precision_5: 0.3333 - val_loss: 0.3648 - val_acc: 0.8930 - val_auc: 0.6631 - val_binary_accuracy: 0.8930 - val_recall_5: 0.0219 - val_precision_5: 0.4444
Epoch 4/20
43/43 [==============================] - 4s 88ms/step - loss:

In [252]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [253]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_self_attention_traditional Accuracy:  0.8067324434126524 MSE:  0.19326755658734765 UAR:  0.5971881060116354 Recall:  0.36470588235294116 Precision:  0.1 F1:  0.15696202531645567
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_self_attention_traditional Accuracy:  0.8874056877539176 MSE:  0.11259431224608242 UAR:  0.5615384615384615 Recall:  0.2 Precision:  0.11888111888111888 F1:  0.14912280701754385
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_self_attention_traditional Accuracy:  0.9239698200812536 MSE:  0.07603017991874637 UAR:  0.5528837175896 Recall:  0.1411764705882353 Precision:  0.17142857142857143 F1:  0.15483870967741936
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_self_attention_traditional Accuracy:  0.9408009286128846 MSE:  0.0591990713871155 UAR:  0.5561588738059327 Recall:  0.12941176470588237 Precision:  0.28205128205128205 F1:  0.1774193548387097
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_self_attenti

In [254]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [255]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [256]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_self_attention_traditional_best Accuracy:  0.8717353453279164 MSE:  0.1282646546720836 UAR:  0.5923364217481865 Recall:  0.2823529411764706 Precision:  0.13043478260869565 F1:  0.17843866171003714
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_self_attention_traditional_best Accuracy:  0.9373186302959954 MSE:  0.06268136970400465 UAR:  0.5766357825181354 Recall:  0.17647058823529413 Precision:  0.2830188679245283 F1:  0.2173913043478261
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_self_attention_traditional_best Accuracy:  0.9437028438769588 MSE:  0.05629715612304121 UAR:  0.5688393306040365 Recall:  0.15294117647058825 Precision:  0.34210526315789475 F1:  0.21138211382113825
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_self_attention_traditional_best Accuracy:  0.9500870574579222 MSE:  0.04991294254207777 UAR:  0.5498886734180852 Recall:  0.10588235294117647 Precision:  0.47368421052631576 F1:  0.17307692307692304
Metric_n

#### Balanced split

In [257]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [258]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  5482
Test:  1723


In [259]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 5234/5482: 95.4761036118205%
1 248/5482: 4.523896388179496%
val:
0 1638/1723: 95.06674405107371%
1 85/1723: 4.933255948926291%


In [260]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.76)

(5482,) (5482,)
(1723,) (1723,)
(7205,) (7205,)
[0 1] [6872  333]
7205
5475
1730


In [261]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 5222/5475: 95.37899543378995%
1 253/5475: 4.621004566210046%
val:
0 1650/1730: 95.3757225433526%
1 80/1730: 4.624277456647399%


##### Single attention

In [262]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_balanced


In [263]:
CONCATENATE_STAT = True

In [264]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5475, 128, 2560) (5475,)
(1730, 128, 2560) (1730,)


In [265]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_18"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2561      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                           

In [266]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 4s 67ms/step - loss: 0.2116 - acc: 0.9344 - auc: 0.6204 - binary_accuracy: 0.9344 - recall_6: 0.0514 - precision_6: 0.0985 - val_loss: 0.1730 - val_acc: 0.9538 - val_auc: 0.7392 - val_binary_accuracy: 0.9538 - val_recall_6: 0.0000e+00 - val_precision_6: 0.0000e+00
Epoch 2/20
43/43 [==============================] - 1s 31ms/step - loss: 0.1649 - acc: 0.9531 - auc: 0.7579 - binary_accuracy: 0.9531 - recall_6: 0.0040 - precision_6: 0.1667 - val_loss: 0.1642 - val_acc: 0.9532 - val_auc: 0.7711 - val_binary_accuracy: 0.9532 - val_recall_6: 0.0000e+00 - val_precision_6: 0.0000e+00
Epoch 3/20
43/43 [==============================] - 1s 30ms/step - loss: 0.1572 - acc: 0.9543 - auc: 0.7928 - binary_accuracy: 0.9543 - recall_6: 0.0277 - precision_6: 0.6364 - val_loss: 0.1614 - val_acc: 0.9532 - val_auc: 0.7868 - val_binary_accuracy: 0.9532 - val_recall_6: 0.0000e+00 - val_precision_6: 0.0000e+00
Epoch 4/20
43/43 [==============================]

In [267]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [268]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_balanced Accuracy:  0.8745664739884393 MSE:  0.1254335260115607 UAR:  0.7082575757575758 Recall:  0.525 Precision:  0.19004524886877827 F1:  0.27906976744186046
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_balanced Accuracy:  0.9161849710982659 MSE:  0.0838150289017341 UAR:  0.6706060606060606 Recall:  0.4 Precision:  0.24806201550387597 F1:  0.3062200956937799
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_balanced Accuracy:  0.9358381502890173 MSE:  0.06416184971098265 UAR:  0.621439393939394 Recall:  0.275 Precision:  0.29333333333333333 F1:  0.2838709677419355
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_balanced Accuracy:  0.9416184971098266 MSE:  0.05838150289017341 UAR:  0.5947348484848485 Recall:  0.2125 Precision:  0.3090909090909091 F1:  0.2518518518518518
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_balanced Accuracy:  0.947

In [269]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [270]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [271]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_balanced_best Accuracy:  0.9 MSE:  0.1 UAR:  0.7096969696969697 Recall:  0.5 Precision:  0.23121387283236994 F1:  0.31620553359683795
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_balanced_best Accuracy:  0.9433526011560693 MSE:  0.056647398843930635 UAR:  0.6313257575757576 Recall:  0.2875 Precision:  0.359375 F1:  0.3194444444444444
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_balanced_best Accuracy:  0.9473988439306359 MSE:  0.05260115606936416 UAR:  0.5382954545454546 Recall:  0.0875 Precision:  0.28 F1:  0.13333333333333333
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_balanced_best Accuracy:  0.9497109826589596 MSE:  0.050289017341040465 UAR:  0.5038257575757575 Recall:  0.0125 Precision:  0.1111111111111111 F1:  0.022471910112359553
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_single_attention_balanced_best Accuracy:  0.9526011560693641 MSE:  0.

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### Self attention

In [272]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_2_max_self_attention_balanced


In [273]:
CONCATENATE_STAT = True

In [274]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5475, 128, 2560) (5475,)
(1730, 128, 2560) (1730,)


In [275]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_19"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2560)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2560)           6553600   ['image_set[0][0]']           
                                                                                                  
 attention_9 (Attention)     (None, None, 2560)           0         ['query[0][0]',               
                                                                     'value[0][0]']        

In [276]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 7s 126ms/step - loss: 0.5038 - acc: 0.9419 - auc: 0.5467 - binary_accuracy: 0.9419 - recall_7: 0.0435 - precision_7: 0.1264 - val_loss: 0.1838 - val_acc: 0.9520 - val_auc: 0.7074 - val_binary_accuracy: 0.9520 - val_recall_7: 0.0250 - val_precision_7: 0.2857
Epoch 2/20
43/43 [==============================] - 4s 89ms/step - loss: 0.1727 - acc: 0.9542 - auc: 0.7127 - binary_accuracy: 0.9542 - recall_7: 0.0593 - precision_7: 0.5357 - val_loss: 0.1684 - val_acc: 0.9526 - val_auc: 0.7593 - val_binary_accuracy: 0.9526 - val_recall_7: 0.0125 - val_precision_7: 0.2500
Epoch 3/20
43/43 [==============================] - 4s 88ms/step - loss: 0.1666 - acc: 0.9543 - auc: 0.7581 - binary_accuracy: 0.9543 - recall_7: 0.0514 - precision_7: 0.5652 - val_loss: 0.1904 - val_acc: 0.9532 - val_auc: 0.7580 - val_binary_accuracy: 0.9532 - val_recall_7: 0.0625 - val_precision_7: 0.4545
Epoch 4/20
43/43 [==============================] - 4s 88ms/step - loss:

In [277]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [278]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_self_attention_balanced Accuracy:  0.8317919075144509 MSE:  0.16820809248554913 UAR:  0.6798863636363637 Recall:  0.5125 Precision:  0.13993174061433447 F1:  0.21983914209115282
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_self_attention_balanced Accuracy:  0.9109826589595376 MSE:  0.08901734104046242 UAR:  0.6559848484848485 Recall:  0.375 Precision:  0.22388059701492538 F1:  0.28037383177570097
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_self_attention_balanced Accuracy:  0.9317919075144508 MSE:  0.06820809248554913 UAR:  0.6133712121212122 Recall:  0.2625 Precision:  0.2625 F1:  0.2625
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_self_attention_balanced Accuracy:  0.9450867052023122 MSE:  0.05491329479768786 UAR:  0.584659090909091 Recall:  0.1875 Precision:  0.3333333333333333 F1:  0.24000000000000005
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_self_attention_balanced Accuracy:  0.9497109826589596 MSE:  0.05028

In [279]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [280]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [281]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_self_attention_balanced_best Accuracy:  0.8745664739884393 MSE:  0.1254335260115607 UAR:  0.6904166666666667 Recall:  0.4875 Precision:  0.1813953488372093 F1:  0.26440677966101694
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_self_attention_balanced_best Accuracy:  0.9323699421965318 MSE:  0.0676300578034682 UAR:  0.6374621212121212 Recall:  0.3125 Precision:  0.28735632183908044 F1:  0.2994011976047904
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_self_attention_balanced_best Accuracy:  0.9468208092485549 MSE:  0.05317919075144509 UAR:  0.5915151515151515 Recall:  0.2 Precision:  0.36363636363636365 F1:  0.25806451612903225
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_self_attention_balanced_best Accuracy:  0.9491329479768786 MSE:  0.05086705202312139 UAR:  0.568939393939394 Recall:  0.15 Precision:  0.375 F1:  0.21428571428571425
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_max_self_attention_balanced_best Accuracy:  0.

### STAT

In [282]:
stat_func = compute_descriptor
stat_name = "STAT"

#### Traditional split

In [283]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
val_pickle = FEATURES_DIR + 'val_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)
filename2features_val = load_features(val_pickle)

In [284]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))
print("Val: ", len(filename2features_val))

Train:  5482
Test:  1723
Val:  1720


In [285]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))
print("val:")
print_classes(get_labels(filename2features_val))

train:
0 5234/5482: 95.4761036118205%
1 248/5482: 4.523896388179496%
test:
0 1638/1723: 95.06674405107371%
1 85/1723: 4.933255948926291%
val:
0 1537/1720: 89.36046511627907%
1 183/1720: 10.63953488372093%


##### Single attention

In [286]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_traditional


In [287]:
CONCATENATE_STAT = True

In [288]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 3840) (5482,)
(1723, 128, 3840) (1723,)
(1720, 128, 3840) (1720,)


In [289]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_20"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3840)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3841      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                           

In [290]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 6s 97ms/step - loss: 0.2071 - acc: 0.9464 - auc: 0.6059 - binary_accuracy: 0.9464 - recall_8: 0.0040 - precision_8: 0.0208 - val_loss: 0.3544 - val_acc: 0.8936 - val_auc: 0.6199 - val_binary_accuracy: 0.8936 - val_recall_8: 0.0000e+00 - val_precision_8: 0.0000e+00
Epoch 2/20
43/43 [==============================] - 2s 48ms/step - loss: 0.1626 - acc: 0.9548 - auc: 0.7616 - binary_accuracy: 0.9548 - recall_8: 0.0000e+00 - precision_8: 0.0000e+00 - val_loss: 0.3673 - val_acc: 0.8936 - val_auc: 0.6570 - val_binary_accuracy: 0.8936 - val_recall_8: 0.0000e+00 - val_precision_8: 0.0000e+00
Epoch 3/20
43/43 [==============================] - 2s 44ms/step - loss: 0.1553 - acc: 0.9548 - auc: 0.7932 - binary_accuracy: 0.9548 - recall_8: 0.0000e+00 - precision_8: 0.0000e+00 - val_loss: 0.3347 - val_acc: 0.8936 - val_auc: 0.6722 - val_binary_accuracy: 0.8936 - val_recall_8: 0.0000e+00 - val_precision_8: 0.0000e+00
Epoch 4/20
43/43 [===============

In [291]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [292]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_traditional Accuracy:  0.7945443993035404 MSE:  0.20545560069645966 UAR:  0.6130862601450837 Recall:  0.4117647058823529 Precision:  0.10324483775811209 F1:  0.1650943396226415
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_traditional Accuracy:  0.8705745792222868 MSE:  0.1294254207777133 UAR:  0.5861488185017597 Recall:  0.27058823529411763 Precision:  0.125 F1:  0.17100371747211895
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_traditional Accuracy:  0.9042367962855484 MSE:  0.09576320371445154 UAR:  0.5703907203907204 Recall:  0.2 Precision:  0.14912280701754385 F1:  0.17085427135678394
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_traditional Accuracy:  0.9355774811375508 MSE:  0.06442251886244922 UAR:  0.5422574157868275 Recall:  0.10588235294117647 Precision:  0.20454545454545456 F1:  0.13953488372093026
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [293]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [294]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [295]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_traditional_best Accuracy:  0.9053975623911782 MSE:  0.09460243760882182 UAR:  0.6546577605401135 Recall:  0.3764705882352941 Precision:  0.22535211267605634 F1:  0.28193832599118945
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_traditional_best Accuracy:  0.9460243760882182 MSE:  0.053975623911781775 UAR:  0.5923687423687424 Recall:  0.2 Precision:  0.40476190476190477 F1:  0.26771653543307083
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_traditional_best Accuracy:  0.9524085896691816 MSE:  0.04759141033081834 UAR:  0.5566867772750126 Recall:  0.11764705882352941 Precision:  0.5882352941176471 F1:  0.19607843137254902
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_traditional_best Accuracy:  0.9524085896691816 MSE:  0.04759141033081834 UAR:  0.5232241614594556 Recall:  0.047058823529411764 Precision:  0.8 F1:  0.08888888888888889
Metric_name:  enet_b0_8_

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/pyth

##### Self attention

In [296]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_2_STAT_self_attention_traditional


In [297]:
CONCATENATE_STAT = True

In [298]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 3840) (5482,)
(1723, 128, 3840) (1723,)
(1720, 128, 3840) (1720,)


In [299]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_21"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3840)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 3840)           1474560   ['image_set[0][0]']           
                                                          0                                       
                                                                                                  
 value (Dense)               (None, None, 3840)           1474560   ['image_set[0][0]']           
                                                          0                                       
                                                                                           

In [300]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 12s 227ms/step - loss: 0.6625 - acc: 0.9239 - auc: 0.5221 - binary_accuracy: 0.9239 - recall_9: 0.0605 - precision_9: 0.0754 - val_loss: 0.3818 - val_acc: 0.8924 - val_auc: 0.5921 - val_binary_accuracy: 0.8924 - val_recall_9: 0.0055 - val_precision_9: 0.2500
Epoch 2/20
43/43 [==============================] - 8s 176ms/step - loss: 0.1709 - acc: 0.9531 - auc: 0.7334 - binary_accuracy: 0.9531 - recall_9: 0.0444 - precision_9: 0.3548 - val_loss: 0.3445 - val_acc: 0.8936 - val_auc: 0.6673 - val_binary_accuracy: 0.8936 - val_recall_9: 0.0055 - val_precision_9: 0.5000
Epoch 3/20
43/43 [==============================] - 7s 173ms/step - loss: 0.1627 - acc: 0.9538 - auc: 0.7598 - binary_accuracy: 0.9538 - recall_9: 0.0161 - precision_9: 0.3077 - val_loss: 0.4119 - val_acc: 0.8936 - val_auc: 0.6508 - val_binary_accuracy: 0.8936 - val_recall_9: 0.0055 - val_precision_9: 0.5000
Epoch 4/20
43/43 [==============================] - 7s 174ms/step - l

In [301]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [302]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_self_attention_traditional Accuracy:  0.9338363319791062 MSE:  0.0661636680208938 UAR:  0.5357645622351505 Recall:  0.09411764705882353 Precision:  0.17777777777777778 F1:  0.12307692307692307
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_self_attention_traditional Accuracy:  0.950667440510737 MSE:  0.04933255948926291 UAR:  0.5390397184514831 Recall:  0.08235294117647059 Precision:  0.5 F1:  0.1414141414141414
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_self_attention_traditional Accuracy:  0.9518282066163668 MSE:  0.0481717933836332 UAR:  0.5284960137901314 Recall:  0.058823529411764705 Precision:  0.625 F1:  0.10752688172043011
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_self_attention_traditional Accuracy:  0.9518282066163668 MSE:  0.0481717933836332 UAR:  0.5229189111542053 Recall:  0.047058823529411764 Precision:  0.6666666666666666 F1:  0.08791208791208792
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_sel

In [303]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [304]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [305]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_self_attention_traditional_best Accuracy:  0.9048171793383634 MSE:  0.09518282066163668 UAR:  0.6208898944193062 Recall:  0.3058823529411765 Precision:  0.1984732824427481 F1:  0.24074074074074073
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_self_attention_traditional_best Accuracy:  0.9442832269297736 MSE:  0.05571677307022635 UAR:  0.5914529914529915 Recall:  0.2 Precision:  0.37777777777777777 F1:  0.26153846153846155
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_self_attention_traditional_best Accuracy:  0.951247823563552 MSE:  0.04875217643644805 UAR:  0.5672304819363643 Recall:  0.1411764705882353 Precision:  0.5217391304347826 F1:  0.22222222222222218
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_self_attention_traditional_best Accuracy:  0.951247823563552 MSE:  0.04875217643644805 UAR:  0.5337678661208073 Recall:  0.07058823529411765 Precision:  0.5454545454545454 F1:  0.125
Metric_name:  enet_b0_8_best_afew.pt_

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Balanced split

In [306]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [307]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  5482
Test:  1723


In [308]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 5234/5482: 95.4761036118205%
1 248/5482: 4.523896388179496%
val:
0 1638/1723: 95.06674405107371%
1 85/1723: 4.933255948926291%


In [309]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.76)

(5482, 2560) (5482,)
(1723, 2560) (1723,)
(7205, 2560) (7205,)
[0 1] [6872  333]
7205
5475
1730


In [310]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 5222/5475: 95.37899543378995%
1 253/5475: 4.621004566210046%
val:
0 1650/1730: 95.3757225433526%
1 80/1730: 4.624277456647399%


##### Single attention

In [311]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_balanced


In [312]:
CONCATENATE_STAT = True

In [313]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5475, 128, 3840) (5475,)
(1730, 128, 3840) (1730,)


In [314]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_22"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3840)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3841      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                           

In [315]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 5s 95ms/step - loss: 0.2116 - acc: 0.9357 - auc: 0.6157 - binary_accuracy: 0.9357 - recall_10: 0.0435 - precision_10: 0.0909 - val_loss: 0.1770 - val_acc: 0.9538 - val_auc: 0.7389 - val_binary_accuracy: 0.9538 - val_recall_10: 0.0000e+00 - val_precision_10: 0.0000e+00
Epoch 2/20
43/43 [==============================] - 2s 44ms/step - loss: 0.1681 - acc: 0.9538 - auc: 0.7434 - binary_accuracy: 0.9538 - recall_10: 0.0119 - precision_10: 0.5000 - val_loss: 0.1664 - val_acc: 0.9532 - val_auc: 0.7714 - val_binary_accuracy: 0.9532 - val_recall_10: 0.0000e+00 - val_precision_10: 0.0000e+00
Epoch 3/20
43/43 [==============================] - 2s 43ms/step - loss: 0.1588 - acc: 0.9545 - auc: 0.7815 - binary_accuracy: 0.9545 - recall_10: 0.0198 - precision_10: 0.8333 - val_loss: 0.1620 - val_acc: 0.9532 - val_auc: 0.7787 - val_binary_accuracy: 0.9532 - val_recall_10: 0.0125 - val_precision_10: 0.3333
Epoch 4/20
43/43 [===========================

In [316]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [317]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_balanced Accuracy:  0.8456647398843931 MSE:  0.15433526011560694 UAR:  0.7168939393939393 Recall:  0.575 Precision:  0.16487455197132617 F1:  0.2562674094707521
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_balanced Accuracy:  0.9208092485549133 MSE:  0.07919075144508671 UAR:  0.6908712121212122 Recall:  0.4375 Precision:  0.2755905511811024 F1:  0.3381642512077295
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_balanced Accuracy:  0.9404624277456647 MSE:  0.05953757225433526 UAR:  0.6535984848484848 Recall:  0.3375 Precision:  0.35064935064935066 F1:  0.34394904458598724
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_balanced Accuracy:  0.9479768786127167 MSE:  0.05202312138728324 UAR:  0.5980681818181818 Recall:  0.2125 Precision:  0.38636363636363635 F1:  0.27419354838709675
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_balanced Acc

In [318]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [319]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [320]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_balanced_best Accuracy:  0.8780346820809248 MSE:  0.12196531791907514 UAR:  0.7219696969696969 Recall:  0.55 Precision:  0.2009132420091324 F1:  0.294314381270903
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_balanced_best Accuracy:  0.9387283236994219 MSE:  0.06127167630057803 UAR:  0.6407954545454546 Recall:  0.3125 Precision:  0.32894736842105265 F1:  0.32051282051282054
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_balanced_best Accuracy:  0.953757225433526 MSE:  0.046242774566473986 UAR:  0.6010984848484848 Recall:  0.2125 Precision:  0.5 F1:  0.2982456140350877
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_balanced_best Accuracy:  0.9560693641618497 MSE:  0.04393063583815029 UAR:  0.5666287878787879 Recall:  0.1375 Precision:  0.6111111111111112 F1:  0.22448979591836737
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_single_attention_balanced_be

##### Self attention

In [321]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

enet_b0_8_best_afew.pt_DAiSEE_2_STAT_self_attention_balanced


In [322]:
CONCATENATE_STAT = True

In [323]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5475, 128, 3840) (5475,)
(1730, 128, 3840) (1730,)


In [324]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_23"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3840)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 3840)           1474560   ['image_set[0][0]']           
                                                          0                                       
                                                                                                  
 value (Dense)               (None, None, 3840)           1474560   ['image_set[0][0]']           
                                                          0                                       
                                                                                           

In [325]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 11s 222ms/step - loss: 0.6550 - acc: 0.9253 - auc: 0.5266 - binary_accuracy: 0.9253 - recall_11: 0.0751 - precision_11: 0.0979 - val_loss: 0.2056 - val_acc: 0.9445 - val_auc: 0.7181 - val_binary_accuracy: 0.9445 - val_recall_11: 0.1125 - val_precision_11: 0.2647
Epoch 2/20
43/43 [==============================] - 7s 173ms/step - loss: 0.1759 - acc: 0.9514 - auc: 0.7221 - binary_accuracy: 0.9514 - recall_11: 0.0395 - precision_11: 0.3030 - val_loss: 0.1680 - val_acc: 0.9538 - val_auc: 0.7627 - val_binary_accuracy: 0.9538 - val_recall_11: 0.0125 - val_precision_11: 0.5000
Epoch 3/20
43/43 [==============================] - 7s 172ms/step - loss: 0.1580 - acc: 0.9549 - auc: 0.7791 - binary_accuracy: 0.9549 - recall_11: 0.0553 - precision_11: 0.6364 - val_loss: 0.1674 - val_acc: 0.9503 - val_auc: 0.7787 - val_binary_accuracy: 0.9503 - val_recall_11: 0.0625 - val_precision_11: 0.3125
Epoch 4/20
43/43 [==============================] - 7s 17

In [326]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [327]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_self_attention_balanced Accuracy:  0.8890173410404625 MSE:  0.11098265895953757 UAR:  0.6742045454545454 Recall:  0.4375 Precision:  0.19230769230769232 F1:  0.26717557251908397
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_self_attention_balanced Accuracy:  0.9329479768786128 MSE:  0.06705202312138728 UAR:  0.5842424242424242 Recall:  0.2 Precision:  0.23529411764705882 F1:  0.2162162162162162
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_self_attention_balanced Accuracy:  0.9473988439306359 MSE:  0.05260115606936416 UAR:  0.568030303030303 Recall:  0.15 Precision:  0.34285714285714286 F1:  0.20869565217391303
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_self_attention_balanced Accuracy:  0.9514450867052023 MSE:  0.048554913294797684 UAR:  0.5285227272727273 Recall:  0.0625 Precision:  0.35714285714285715 F1:  0.10638297872340426
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_self_attention_balanced Accuracy:  0.95

In [328]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [329]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [330]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_self_attention_balanced_best Accuracy:  0.869364161849711 MSE:  0.130635838150289 UAR:  0.687689393939394 Recall:  0.4875 Precision:  0.17410714285714285 F1:  0.2565789473684211
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_self_attention_balanced_best Accuracy:  0.9300578034682081 MSE:  0.06994219653179191 UAR:  0.6481439393939394 Recall:  0.3375 Precision:  0.28421052631578947 F1:  0.30857142857142855
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_self_attention_balanced_best Accuracy:  0.946242774566474 MSE:  0.05375722543352601 UAR:  0.5793181818181818 Recall:  0.175 Precision:  0.34146341463414637 F1:  0.23140495867768593
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_self_attention_balanced_best Accuracy:  0.9502890173410404 MSE:  0.04971098265895954 UAR:  0.5517045454545455 Recall:  0.1125 Precision:  0.375 F1:  0.1730769230769231
Metric_name:  enet_b0_8_best_afew.pt_DAiSEE_2_STAT_self_attention_balanced_best Accura

## MobileNet_7

In [331]:
base_model_key = 'mobilenet_7.h5'

In [337]:
ENGAGE_WILD_DATASET, BIN_CLASSIFICATION, N_CLASSES, DATASET_NAME, BASE_DATASET_DIR, ext = define_dataset(False, True)
video2label, labels_list = get_video2label()
print(labels_list)
print(video2label)

['engaged', 'distracted']
{'1100011002': 0, '1100011003': 0, '1100011004': 0, '1100011005': 0, '1100011006': 0, '1100011007': 0, '1100011008': 0, '1100011009': 0, '1100011010': 0, '1100011011': 0, '1100011012': 0, '1100011013': 0, '1100011014': 0, '1100011015': 0, '1100011016': 0, '1100011017': 0, '1100011018': 0, '1100011019': 0, '1100011020': 0, '1100011021': 0, '1100011022': 0, '1100011023': 0, '1100011025': 0, '1100011026': 0, '1100011027': 0, '1100011028': 0, '1100011029': 0, '1100011031': 0, '1100011032': 0, '1100011034': 0, '1100011035': 0, '1100011037': 0, '1100011038': 0, '1100011040': 0, '1100011046': 0, '1100011047': 0, '1100011048': 0, '1100011049': 0, '1100011050': 0, '1100011051': 0, '1100011052': 0, '1100011053': 0, '1100011054': 0, '1100011055': 0, '1100011056': 0, '1100011057': 0, '1100011058': 0, '1100011059': 0, '1100011060': 0, '1100011062': 0, '1100011063': 0, '1100011064': 0, '1100011066': 0, '1100011067': 0, '1100011068': 0, '1100011069': 0, '1100011070': 0, '110

### Std

In [338]:
stat_func = np.std
stat_name = "std"

#### Traditional split

In [339]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
val_pickle = FEATURES_DIR + 'val_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)
filename2features_val = load_features(val_pickle)

In [340]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))
print("Val: ", len(filename2features_val))

Train:  5482
Test:  1723
Val:  1720


In [341]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))
print("val:")
print_classes(get_labels(filename2features_val))

train:
0 5234/5482: 95.4761036118205%
1 248/5482: 4.523896388179496%
test:
0 1638/1723: 95.06674405107371%
1 85/1723: 4.933255948926291%
val:
0 1537/1720: 89.36046511627907%
1 183/1720: 10.63953488372093%


##### Single attention

In [342]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

mobilenet_7.h5_DAiSEE_2_std_single_attention_traditional


In [343]:
CONCATENATE_STAT = True

In [344]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2048) (5482,)
(1723, 128, 2048) (1723,)
(1720, 128, 2048) (1720,)


In [345]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_24"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                           

In [346]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 4s 55ms/step - loss: 0.2167 - acc: 0.9548 - auc: 0.5371 - binary_accuracy: 0.9548 - recall_12: 0.0000e+00 - precision_12: 0.0000e+00 - val_loss: 0.3532 - val_acc: 0.8936 - val_auc: 0.5805 - val_binary_accuracy: 0.8936 - val_recall_12: 0.0000e+00 - val_precision_12: 0.0000e+00
Epoch 2/20
43/43 [==============================] - 1s 26ms/step - loss: 0.1690 - acc: 0.9548 - auc: 0.7114 - binary_accuracy: 0.9548 - recall_12: 0.0000e+00 - precision_12: 0.0000e+00 - val_loss: 0.3347 - val_acc: 0.8936 - val_auc: 0.6479 - val_binary_accuracy: 0.8936 - val_recall_12: 0.0000e+00 - val_precision_12: 0.0000e+00
Epoch 3/20
43/43 [==============================] - 1s 25ms/step - loss: 0.1592 - acc: 0.9548 - auc: 0.7697 - binary_accuracy: 0.9548 - recall_12: 0.0000e+00 - precision_12: 0.0000e+00 - val_loss: 0.3497 - val_acc: 0.8936 - val_auc: 0.6575 - val_binary_accuracy: 0.8936 - val_recall_12: 0.0000e+00 - val_precision_12: 0.0000e+00
Epoch 4/20
43

In [347]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [348]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_traditional Accuracy:  0.7492745211839814 MSE:  0.2507254788160186 UAR:  0.594853838971486 Recall:  0.4235294117647059 Precision:  0.08591885441527446 F1:  0.14285714285714285
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_traditional Accuracy:  0.8757980266976204 MSE:  0.12420197330237957 UAR:  0.57774186597716 Recall:  0.24705882352941178 Precision:  0.12280701754385964 F1:  0.1640625
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_traditional Accuracy:  0.926871735345328 MSE:  0.07312826465467208 UAR:  0.5599870717517776 Recall:  0.15294117647058825 Precision:  0.19402985074626866 F1:  0.17105263157894737
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_traditional Accuracy:  0.9483459082994776 MSE:  0.05165409170052235 UAR:  0.5378187172304819 Recall:  0.08235294117647059 Precision:  0.3888888888888889 F1:  0.13592233009708737
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_tradit

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/pyth

In [349]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [350]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [351]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_traditional_best Accuracy:  0.8432965757399884 MSE:  0.1567034242600116 UAR:  0.6666127989657402 Recall:  0.47058823529411764 Precision:  0.1509433962264151 F1:  0.2285714285714286
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_traditional_best Accuracy:  0.9500870574579222 MSE:  0.04991294254207777 UAR:  0.5777741865977161 Recall:  0.16470588235294117 Precision:  0.4827586206896552 F1:  0.24561403508771928
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_traditional_best Accuracy:  0.951247823563552 MSE:  0.04875217643644805 UAR:  0.5170365582130289 Recall:  0.03529411764705882 Precision:  0.6 F1:  0.06666666666666667
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_traditional_best Accuracy:  0.9495066744051074 MSE:  0.05049332559489263 UAR:  0.4993894993894994 Recall:  0.0 Precision:  0.0 F1:  0.0
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_traditional_best Accuracy:  0.95066744

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/pyth

##### Self attention

In [352]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

mobilenet_7.h5_DAiSEE_2_std_self_attention_traditional


In [353]:
CONCATENATE_STAT = True

In [354]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2048) (5482,)
(1723, 128, 2048) (1723,)
(1720, 128, 2048) (1720,)


In [355]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_25"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 attention_12 (Attention)    (None, None, 2048)           0         ['query[0][0]',               
                                                                     'value[0][0]']        

In [356]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 5s 95ms/step - loss: 0.3712 - acc: 0.9365 - auc: 0.5789 - binary_accuracy: 0.9365 - recall_13: 0.0323 - precision_13: 0.0690 - val_loss: 0.4060 - val_acc: 0.8936 - val_auc: 0.5761 - val_binary_accuracy: 0.8936 - val_recall_13: 0.0000e+00 - val_precision_13: 0.0000e+00
Epoch 2/20
43/43 [==============================] - 3s 63ms/step - loss: 0.1655 - acc: 0.9548 - auc: 0.7363 - binary_accuracy: 0.9548 - recall_13: 0.0081 - precision_13: 0.5000 - val_loss: 0.3365 - val_acc: 0.8831 - val_auc: 0.6547 - val_binary_accuracy: 0.8831 - val_recall_13: 0.0219 - val_precision_13: 0.1538
Epoch 3/20
43/43 [==============================] - 3s 67ms/step - loss: 0.1707 - acc: 0.9535 - auc: 0.7258 - binary_accuracy: 0.9535 - recall_13: 0.0524 - precision_13: 0.3939 - val_loss: 0.3455 - val_acc: 0.8919 - val_auc: 0.6341 - val_binary_accuracy: 0.8919 - val_recall_13: 0.0109 - val_precision_13: 0.2857
Epoch 4/20
43/43 [==============================] - 3

In [357]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [358]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_std_self_attention_traditional Accuracy:  0.820661636680209 MSE:  0.17933836331979106 UAR:  0.5320117790706026 Recall:  0.21176470588235294 Precision:  0.06923076923076923 F1:  0.10434782608695653
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_self_attention_traditional Accuracy:  0.9239698200812536 MSE:  0.07603017991874637 UAR:  0.5528837175896 Recall:  0.1411764705882353 Precision:  0.17142857142857143 F1:  0.15483870967741936
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_self_attention_traditional Accuracy:  0.9402205455600696 MSE:  0.05977945443993035 UAR:  0.5335452129569777 Recall:  0.08235294117647059 Precision:  0.21875 F1:  0.11965811965811965
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_self_attention_traditional Accuracy:  0.9483459082994776 MSE:  0.05165409170052235 UAR:  0.5322416145945558 Recall:  0.07058823529411765 Precision:  0.375 F1:  0.11881188118811882
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_self_attention_traditional Accuracy:  0.94950

In [359]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [360]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [361]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.575739988392339 MSE:  0.4242600116076611 UAR:  0.609548947784242 Recall:  0.6470588235294118 Precision:  0.07275132275132275 F1:  0.13079667063020212
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.840975043528729 MSE:  0.15902495647127105 UAR:  0.6598146951088127 Recall:  0.4588235294117647 Precision:  0.14606741573033707 F1:  0.2215909090909091
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.9210679048171794 MSE:  0.07893209518282066 UAR:  0.6238598003303886 Recall:  0.29411764705882354 Precision:  0.24752475247524752 F1:  0.26881720430107525
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_self_attention_traditional_best Accuracy:  0.9437028438769588 MSE:  0.05629715612304121 UAR:  0.5967248437836673 Recall:  0.21176470588235294 Precision:  0.375 F1:  0.2706766917293233
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_self_attention_t

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Balanced split

In [362]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [363]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  5482
Test:  1723


In [364]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 5234/5482: 95.4761036118205%
1 248/5482: 4.523896388179496%
val:
0 1638/1723: 95.06674405107371%
1 85/1723: 4.933255948926291%


In [365]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.76)

(5482,) (5482,)
(1723,) (1723,)
(7205,) (7205,)
[0 1] [6872  333]
7205
5475
1730


In [366]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 5222/5475: 95.37899543378995%
1 253/5475: 4.621004566210046%
val:
0 1650/1730: 95.3757225433526%
1 80/1730: 4.624277456647399%


##### Single attention

In [367]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

mobilenet_7.h5_DAiSEE_2_std_single_attention_balanced


In [368]:
CONCATENATE_STAT = True

In [369]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5475, 128, 2048) (5475,)
(1730, 128, 2048) (1730,)


In [370]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_26"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                           

In [371]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 4s 56ms/step - loss: 0.2223 - acc: 0.9443 - auc: 0.5189 - binary_accuracy: 0.9443 - recall_14: 0.0000e+00 - precision_14: 0.0000e+00 - val_loss: 0.1775 - val_acc: 0.9538 - val_auc: 0.6726 - val_binary_accuracy: 0.9538 - val_recall_14: 0.0000e+00 - val_precision_14: 0.0000e+00
Epoch 2/20
43/43 [==============================] - 1s 26ms/step - loss: 0.1723 - acc: 0.9538 - auc: 0.7023 - binary_accuracy: 0.9538 - recall_14: 0.0000e+00 - precision_14: 0.0000e+00 - val_loss: 0.1682 - val_acc: 0.9538 - val_auc: 0.7322 - val_binary_accuracy: 0.9538 - val_recall_14: 0.0000e+00 - val_precision_14: 0.0000e+00
Epoch 3/20
43/43 [==============================] - 1s 25ms/step - loss: 0.1618 - acc: 0.9540 - auc: 0.7720 - binary_accuracy: 0.9540 - recall_14: 0.0040 - precision_14: 1.0000 - val_loss: 0.1675 - val_acc: 0.9538 - val_auc: 0.7538 - val_binary_accuracy: 0.9538 - val_recall_14: 0.0000e+00 - val_precision_14: 0.0000e+00
Epoch 4/20
43/43 [===

In [372]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [373]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_balanced Accuracy:  0.8560693641618498 MSE:  0.1439306358381503 UAR:  0.7045075757575757 Recall:  0.5375 Precision:  0.16862745098039217 F1:  0.2567164179104478
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_balanced Accuracy:  0.9179190751445087 MSE:  0.08208092485549133 UAR:  0.6715151515151515 Recall:  0.4 Precision:  0.25396825396825395 F1:  0.3106796116504854
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_balanced Accuracy:  0.9416184971098266 MSE:  0.05838150289017341 UAR:  0.6125757575757576 Recall:  0.25 Precision:  0.32786885245901637 F1:  0.28368794326241137
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_balanced Accuracy:  0.9508670520231214 MSE:  0.049132947976878616 UAR:  0.5817424242424243 Recall:  0.175 Precision:  0.42424242424242425 F1:  0.247787610619469
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_balanced Accuracy:  0.9514450867052023 MSE:  0.0485549132947976

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [374]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [375]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [376]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_balanced_best Accuracy:  0.8791907514450867 MSE:  0.12080924855491329 UAR:  0.7225757575757576 Recall:  0.55 Precision:  0.20276497695852536 F1:  0.2962962962962963
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_balanced_best Accuracy:  0.9427745664739884 MSE:  0.05722543352601156 UAR:  0.636969696969697 Recall:  0.3 Precision:  0.3582089552238806 F1:  0.32653061224489793
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_balanced_best Accuracy:  0.9502890173410404 MSE:  0.04971098265895954 UAR:  0.5814393939393939 Recall:  0.175 Precision:  0.4117647058823529 F1:  0.2456140350877193
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_balanced_best Accuracy:  0.954335260115607 MSE:  0.04566473988439306 UAR:  0.5359848484848485 Recall:  0.075 Precision:  0.5454545454545454 F1:  0.13186813186813187
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_single_attention_balanced_best Accuracy:  0.953757225433526 MSE:

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/pyth

##### Self attention

In [377]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

mobilenet_7.h5_DAiSEE_2_std_self_attention_balanced


In [378]:
CONCATENATE_STAT = True

In [379]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5475, 128, 2048) (5475,)
(1730, 128, 2048) (1730,)


In [380]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_27"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 attention_13 (Attention)    (None, None, 2048)           0         ['query[0][0]',               
                                                                     'value[0][0]']        

In [381]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 5s 95ms/step - loss: 0.3851 - acc: 0.9410 - auc: 0.5399 - binary_accuracy: 0.9410 - recall_15: 0.0593 - precision_15: 0.1500 - val_loss: 0.1973 - val_acc: 0.9538 - val_auc: 0.6883 - val_binary_accuracy: 0.9538 - val_recall_15: 0.0000e+00 - val_precision_15: 0.0000e+00
Epoch 2/20
43/43 [==============================] - 3s 63ms/step - loss: 0.1714 - acc: 0.9534 - auc: 0.7176 - binary_accuracy: 0.9534 - recall_15: 0.0277 - precision_15: 0.4375 - val_loss: 0.1686 - val_acc: 0.9549 - val_auc: 0.7377 - val_binary_accuracy: 0.9549 - val_recall_15: 0.0375 - val_precision_15: 0.7500
Epoch 3/20
43/43 [==============================] - 3s 63ms/step - loss: 0.1590 - acc: 0.9543 - auc: 0.7782 - binary_accuracy: 0.9543 - recall_15: 0.0632 - precision_15: 0.5517 - val_loss: 0.1681 - val_acc: 0.9520 - val_auc: 0.7399 - val_binary_accuracy: 0.9520 - val_recall_15: 0.0250 - val_precision_15: 0.2857
Epoch 4/20
43/43 [==============================] - 3

In [382]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [383]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_std_self_attention_balanced Accuracy:  0.8849710982658959 MSE:  0.11502890173410404 UAR:  0.7137121212121212 Recall:  0.525 Precision:  0.20689655172413793 F1:  0.2968197879858658
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_self_attention_balanced Accuracy:  0.9300578034682081 MSE:  0.06994219653179191 UAR:  0.6065151515151515 Recall:  0.25 Precision:  0.24691358024691357 F1:  0.2484472049689441
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_self_attention_balanced Accuracy:  0.9387283236994219 MSE:  0.06127167630057803 UAR:  0.5753787878787878 Recall:  0.175 Precision:  0.25925925925925924 F1:  0.208955223880597
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_self_attention_balanced Accuracy:  0.9410404624277456 MSE:  0.058959537572254334 UAR:  0.5409090909090909 Recall:  0.1 Precision:  0.21052631578947367 F1:  0.13559322033898305
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_self_attention_balanced Accuracy:  0.9479768786127167 MSE:  0.05202312138728324 UAR:  0.

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [384]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [385]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [386]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.8641618497109826 MSE:  0.13583815028901733 UAR:  0.6849621212121212 Recall:  0.4875 Precision:  0.16738197424892703 F1:  0.24920127795527158
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.9179190751445087 MSE:  0.08208092485549133 UAR:  0.6179924242424242 Recall:  0.2875 Precision:  0.21296296296296297 F1:  0.2446808510638298
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.9387283236994219 MSE:  0.06127167630057803 UAR:  0.5813257575757576 Recall:  0.1875 Precision:  0.26785714285714285 F1:  0.22058823529411767
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.9514450867052023 MSE:  0.048554913294797684 UAR:  0.5582575757575757 Recall:  0.125 Precision:  0.4166666666666667 F1:  0.1923076923076923
Metric_name:  mobilenet_7.h5_DAiSEE_2_std_self_attention_balanced_best Accuracy:  0.9514450867052023 MS

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Max

In [387]:
stat_func = np.max
stat_name = "max"

#### Traditional split

In [388]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
val_pickle = FEATURES_DIR + 'val_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)
filename2features_val = load_features(val_pickle)

In [389]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))
print("Val: ", len(filename2features_val))

Train:  5482
Test:  1723
Val:  1720


In [390]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))
print("val:")
print_classes(get_labels(filename2features_val))

train:
0 5234/5482: 95.4761036118205%
1 248/5482: 4.523896388179496%
test:
0 1638/1723: 95.06674405107371%
1 85/1723: 4.933255948926291%
val:
0 1537/1720: 89.36046511627907%
1 183/1720: 10.63953488372093%


##### Single attention

In [391]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

mobilenet_7.h5_DAiSEE_2_max_single_attention_traditional


In [392]:
CONCATENATE_STAT = True

In [393]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2048) (5482,)
(1723, 128, 2048) (1723,)
(1720, 128, 2048) (1720,)


In [394]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_28"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                           

In [395]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 4s 55ms/step - loss: 0.2203 - acc: 0.9538 - auc: 0.5377 - binary_accuracy: 0.9538 - recall_16: 0.0000e+00 - precision_16: 0.0000e+00 - val_loss: 0.3500 - val_acc: 0.8936 - val_auc: 0.6077 - val_binary_accuracy: 0.8936 - val_recall_16: 0.0000e+00 - val_precision_16: 0.0000e+00
Epoch 2/20
43/43 [==============================] - 1s 26ms/step - loss: 0.1698 - acc: 0.9544 - auc: 0.7120 - binary_accuracy: 0.9544 - recall_16: 0.0000e+00 - precision_16: 0.0000e+00 - val_loss: 0.3314 - val_acc: 0.8936 - val_auc: 0.6549 - val_binary_accuracy: 0.8936 - val_recall_16: 0.0000e+00 - val_precision_16: 0.0000e+00
Epoch 3/20
43/43 [==============================] - 1s 25ms/step - loss: 0.1608 - acc: 0.9548 - auc: 0.7555 - binary_accuracy: 0.9548 - recall_16: 0.0040 - precision_16: 0.5000 - val_loss: 0.3560 - val_acc: 0.8936 - val_auc: 0.6486 - val_binary_accuracy: 0.8936 - val_recall_16: 0.0000e+00 - val_precision_16: 0.0000e+00
Epoch 4/20
43/43 [===

In [396]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [397]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_traditional Accuracy:  0.5455600696459664 MSE:  0.4544399303540337 UAR:  0.6215614450908569 Recall:  0.7058823529411765 Precision:  0.07334963325183375 F1:  0.132890365448505
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_traditional Accuracy:  0.7260591990713872 MSE:  0.2739408009286129 UAR:  0.6718774689362925 Recall:  0.611764705882353 Precision:  0.10590631364562118 F1:  0.18055555555555555
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_traditional Accuracy:  0.8154381892048752 MSE:  0.1845618107951248 UAR:  0.6631149895855779 Recall:  0.49411764705882355 Precision:  0.13249211356466878 F1:  0.208955223880597
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_traditional Accuracy:  0.8670922809053976 MSE:  0.13290771909460244 UAR:  0.6512425483013718 Recall:  0.4117647058823529 Precision:  0.16355140186915887 F1:  0.23411371237458192
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_

In [398]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [399]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [400]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_traditional_best Accuracy:  0.5908299477655252 MSE:  0.4091700522344748 UAR:  0.6453709689003807 Recall:  0.7058823529411765 Precision:  0.08108108108108109 F1:  0.14545454545454548
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_traditional_best Accuracy:  0.8026697620429484 MSE:  0.19733023795705165 UAR:  0.6396681749622926 Recall:  0.4588235294117647 Precision:  0.11711711711711711 F1:  0.18660287081339713
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_traditional_best Accuracy:  0.9071387115496228 MSE:  0.09286128845037725 UAR:  0.6276879982762336 Recall:  0.3176470588235294 Precision:  0.20930232558139536 F1:  0.2523364485981308
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_traditional_best Accuracy:  0.946604759141033 MSE:  0.053395240858966915 UAR:  0.5815197874021404 Recall:  0.17647058823529413 Precision:  0.40540540540540543 F1:  0.24590163934426235
Metric_name:  mobilenet_7.h5_DAiS

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### Self attention

In [401]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

mobilenet_7.h5_DAiSEE_2_max_self_attention_traditional


In [402]:
CONCATENATE_STAT = True

In [403]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 2048) (5482,)
(1723, 128, 2048) (1723,)
(1720, 128, 2048) (1720,)


In [404]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_29"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 attention_14 (Attention)    (None, None, 2048)           0         ['query[0][0]',               
                                                                     'value[0][0]']        

In [405]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 5s 94ms/step - loss: 0.9046 - acc: 0.9259 - auc: 0.5388 - binary_accuracy: 0.9259 - recall_17: 0.0685 - precision_17: 0.0885 - val_loss: 0.5634 - val_acc: 0.8936 - val_auc: 0.5168 - val_binary_accuracy: 0.8936 - val_recall_17: 0.0000e+00 - val_precision_17: 0.0000e+00
Epoch 2/20
43/43 [==============================] - 3s 64ms/step - loss: 0.1865 - acc: 0.9526 - auc: 0.6456 - binary_accuracy: 0.9526 - recall_17: 0.0121 - precision_17: 0.1667 - val_loss: 0.3380 - val_acc: 0.8919 - val_auc: 0.6463 - val_binary_accuracy: 0.8919 - val_recall_17: 0.0055 - val_precision_17: 0.2000
Epoch 3/20
43/43 [==============================] - 3s 63ms/step - loss: 0.1605 - acc: 0.9542 - auc: 0.7603 - binary_accuracy: 0.9542 - recall_17: 0.0363 - precision_17: 0.4286 - val_loss: 0.3453 - val_acc: 0.8895 - val_auc: 0.6549 - val_binary_accuracy: 0.8895 - val_recall_17: 0.0546 - val_precision_17: 0.3704
Epoch 4/20
43/43 [==============================] - 3

In [ ]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [ ]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

In [ ]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [409]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

In [410]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_max_self_attention_traditional_best Accuracy:  0.7417295414973882 MSE:  0.2582704585026117 UAR:  0.6076168929110105 Recall:  0.4588235294117647 Precision:  0.08904109589041095 F1:  0.14913957934990438
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_self_attention_traditional_best Accuracy:  0.9251305861868834 MSE:  0.07486941381311665 UAR:  0.6148423471952884 Recall:  0.27058823529411763 Precision:  0.25555555555555554 F1:  0.2628571428571429
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_self_attention_traditional_best Accuracy:  0.9518282066163668 MSE:  0.0481717933836332 UAR:  0.6009983480571716 Recall:  0.21176470588235294 Precision:  0.5294117647058824 F1:  0.3025210084033613
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_self_attention_traditional_best Accuracy:  0.950667440510737 MSE:  0.04933255948926291 UAR:  0.5390397184514831 Recall:  0.08235294117647059 Precision:  0.5 F1:  0.1414141414141414
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_self_attention_tra

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Balanced split

In [411]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [412]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  5482
Test:  1723


In [413]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 5234/5482: 95.4761036118205%
1 248/5482: 4.523896388179496%
val:
0 1638/1723: 95.06674405107371%
1 85/1723: 4.933255948926291%


In [414]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.76)

(5482,) (5482,)
(1723,) (1723,)
(7205,) (7205,)
[0 1] [6872  333]
7205
5475
1730


In [415]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 5222/5475: 95.37899543378995%
1 253/5475: 4.621004566210046%
val:
0 1650/1730: 95.3757225433526%
1 80/1730: 4.624277456647399%


##### Single attention

In [416]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

mobilenet_7.h5_DAiSEE_2_max_single_attention_balanced


In [417]:
CONCATENATE_STAT = True

In [418]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5475, 128, 2048) (5475,)
(1730, 128, 2048) (1730,)


In [419]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_30"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              2049      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                           

In [420]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 4s 57ms/step - loss: 0.2681 - acc: 0.9370 - auc: 0.5105 - binary_accuracy: 0.9370 - recall_18: 0.0158 - precision_18: 0.0400 - val_loss: 0.1837 - val_acc: 0.9538 - val_auc: 0.6330 - val_binary_accuracy: 0.9538 - val_recall_18: 0.0000e+00 - val_precision_18: 0.0000e+00
Epoch 2/20
43/43 [==============================] - 1s 25ms/step - loss: 0.1751 - acc: 0.9538 - auc: 0.6807 - binary_accuracy: 0.9538 - recall_18: 0.0000e+00 - precision_18: 0.0000e+00 - val_loss: 0.1708 - val_acc: 0.9538 - val_auc: 0.7270 - val_binary_accuracy: 0.9538 - val_recall_18: 0.0000e+00 - val_precision_18: 0.0000e+00
Epoch 3/20
43/43 [==============================] - 1s 26ms/step - loss: 0.1655 - acc: 0.9538 - auc: 0.7413 - binary_accuracy: 0.9538 - recall_18: 0.0000e+00 - precision_18: 0.0000e+00 - val_loss: 0.1730 - val_acc: 0.9538 - val_auc: 0.7468 - val_binary_accuracy: 0.9538 - val_recall_18: 0.0000e+00 - val_precision_18: 0.0000e+00
Epoch 4/20
43/43 [===

In [421]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [422]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_balanced Accuracy:  0.8473988439306358 MSE:  0.15260115606936417 UAR:  0.7178030303030303 Recall:  0.575 Precision:  0.16666666666666666 F1:  0.25842696629213485
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_balanced Accuracy:  0.9277456647398844 MSE:  0.07225433526011561 UAR:  0.6766666666666667 Recall:  0.4 Precision:  0.29357798165137616 F1:  0.3386243386243386
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_balanced Accuracy:  0.945664739884393 MSE:  0.05433526011560694 UAR:  0.6206439393939394 Recall:  0.2625 Precision:  0.375 F1:  0.3088235294117648
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_balanced Accuracy:  0.9497109826589596 MSE:  0.050289017341040465 UAR:  0.5395075757575757 Recall:  0.0875 Precision:  0.3333333333333333 F1:  0.13861386138613863
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_balanced Accuracy:  0.953757225433526 MSE:  0.046242774566473986 UAR:  0.5

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/pyth

In [423]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [424]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [425]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_balanced_best Accuracy:  0.9121387283236995 MSE:  0.08786127167630058 UAR:  0.7101136363636363 Recall:  0.4875 Precision:  0.26 F1:  0.3391304347826087
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_balanced_best Accuracy:  0.945664739884393 MSE:  0.05433526011560694 UAR:  0.6206439393939394 Recall:  0.2625 Precision:  0.375 F1:  0.3088235294117648
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_balanced_best Accuracy:  0.9514450867052023 MSE:  0.048554913294797684 UAR:  0.5582575757575757 Recall:  0.125 Precision:  0.4166666666666667 F1:  0.1923076923076923
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_balanced_best Accuracy:  0.9531791907514451 MSE:  0.04682080924855491 UAR:  0.5056439393939394 Recall:  0.0125 Precision:  0.3333333333333333 F1:  0.024096385542168676
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_single_attention_balanced_best Accuracy:  0.953757225433526 MSE:  0.0462427745664739

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/pyth

##### Self attention

In [426]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

mobilenet_7.h5_DAiSEE_2_max_self_attention_balanced


In [427]:
CONCATENATE_STAT = True

In [428]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5475, 128, 2048) (5475,)
(1730, 128, 2048) (1730,)


In [429]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_31"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 2048)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 2048)           4194304   ['image_set[0][0]']           
                                                                                                  
 attention_15 (Attention)    (None, None, 2048)           0         ['query[0][0]',               
                                                                     'value[0][0]']        

In [430]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 5s 94ms/step - loss: 0.6588 - acc: 0.9302 - auc: 0.5810 - binary_accuracy: 0.9302 - recall_19: 0.0791 - precision_19: 0.1183 - val_loss: 0.1834 - val_acc: 0.9503 - val_auc: 0.6953 - val_binary_accuracy: 0.9503 - val_recall_19: 0.1000 - val_precision_19: 0.3636
Epoch 2/20
43/43 [==============================] - 3s 63ms/step - loss: 0.1791 - acc: 0.9527 - auc: 0.6992 - binary_accuracy: 0.9527 - recall_19: 0.0474 - precision_19: 0.4000 - val_loss: 0.1724 - val_acc: 0.9538 - val_auc: 0.7251 - val_binary_accuracy: 0.9538 - val_recall_19: 0.0125 - val_precision_19: 0.5000
Epoch 3/20
43/43 [==============================] - 3s 63ms/step - loss: 0.1765 - acc: 0.9532 - auc: 0.7144 - binary_accuracy: 0.9532 - recall_19: 0.0949 - precision_19: 0.4706 - val_loss: 0.1718 - val_acc: 0.9509 - val_auc: 0.7603 - val_binary_accuracy: 0.9509 - val_recall_19: 0.1000 - val_precision_19: 0.3810
Epoch 4/20
43/43 [==============================] - 3s 64ms/s

In [431]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [432]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_max_self_attention_balanced Accuracy:  0.9439306358381503 MSE:  0.05606936416184971 UAR:  0.6137878787878788 Recall:  0.25 Precision:  0.3508771929824561 F1:  0.291970802919708
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_self_attention_balanced Accuracy:  0.9491329479768786 MSE:  0.05086705202312139 UAR:  0.5629924242424242 Recall:  0.1375 Precision:  0.36666666666666664 F1:  0.2
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_self_attention_balanced Accuracy:  0.9497109826589596 MSE:  0.050289017341040465 UAR:  0.5216666666666666 Recall:  0.05 Precision:  0.26666666666666666 F1:  0.08421052631578949
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_self_attention_balanced Accuracy:  0.9520231213872833 MSE:  0.04797687861271676 UAR:  0.5169318181818182 Recall:  0.0375 Precision:  0.3333333333333333 F1:  0.06741573033707865
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_self_attention_balanced Accuracy:  0.953757225433526 MSE:  0.046242774566473986 UAR:  0.5178409090909

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [433]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [434]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [435]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_max_self_attention_balanced_best Accuracy:  0.9 MSE:  0.1 UAR:  0.7096969696969697 Recall:  0.5 Precision:  0.23121387283236994 F1:  0.31620553359683795
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_self_attention_balanced_best Accuracy:  0.9439306358381503 MSE:  0.05606936416184971 UAR:  0.6375757575757576 Recall:  0.3 Precision:  0.36923076923076925 F1:  0.3310344827586207
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_self_attention_balanced_best Accuracy:  0.9508670520231214 MSE:  0.049132947976878616 UAR:  0.5876893939393939 Recall:  0.1875 Precision:  0.42857142857142855 F1:  0.26086956521739124
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_self_attention_balanced_best Accuracy:  0.9502890173410404 MSE:  0.04971098265895954 UAR:  0.5279166666666666 Recall:  0.0625 Precision:  0.3125 F1:  0.10416666666666667
Metric_name:  mobilenet_7.h5_DAiSEE_2_max_self_attention_balanced_best Accuracy:  0.9514450867052023 MSE:  0.048554913294797684 UAR:  0.51068181818181

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### STAT

In [436]:
stat_func = compute_descriptor
stat_name = "STAT"

#### Traditional split

In [437]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
val_pickle = FEATURES_DIR + 'val_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)
filename2features_val = load_features(val_pickle)

In [438]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))
print("Val: ", len(filename2features_val))

Train:  5482
Test:  1723
Val:  1720


In [439]:
print("train:")
print_classes(get_labels(filename2features_train))
print("test:")
print_classes(get_labels(filename2features_test))
print("val:")
print_classes(get_labels(filename2features_val))

train:
0 5234/5482: 95.4761036118205%
1 248/5482: 4.523896388179496%
test:
0 1638/1723: 95.06674405107371%
1 85/1723: 4.933255948926291%
val:
0 1537/1720: 89.36046511627907%
1 183/1720: 10.63953488372093%


##### Single attention

In [440]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_traditional")
print(metric_name)

mobilenet_7.h5_DAiSEE_2_STAT_single_attention_traditional


In [441]:
CONCATENATE_STAT = True

In [442]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 3072) (5482,)
(1723, 128, 3072) (1723,)
(1720, 128, 3072) (1720,)


In [443]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_32"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3072)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3073      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                           

In [444]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 5s 78ms/step - loss: 0.2130 - acc: 0.9537 - auc: 0.5883 - binary_accuracy: 0.9537 - recall_20: 0.0081 - precision_20: 0.2000 - val_loss: 0.3418 - val_acc: 0.8936 - val_auc: 0.6117 - val_binary_accuracy: 0.8936 - val_recall_20: 0.0000e+00 - val_precision_20: 0.0000e+00
Epoch 2/20
43/43 [==============================] - 2s 41ms/step - loss: 0.1650 - acc: 0.9548 - auc: 0.7414 - binary_accuracy: 0.9548 - recall_20: 0.0000e+00 - precision_20: 0.0000e+00 - val_loss: 0.3509 - val_acc: 0.8936 - val_auc: 0.6489 - val_binary_accuracy: 0.8936 - val_recall_20: 0.0000e+00 - val_precision_20: 0.0000e+00
Epoch 3/20
43/43 [==============================] - 1s 35ms/step - loss: 0.1568 - acc: 0.9548 - auc: 0.7823 - binary_accuracy: 0.9548 - recall_20: 0.0000e+00 - precision_20: 0.0000e+00 - val_loss: 0.3482 - val_acc: 0.8936 - val_auc: 0.6613 - val_binary_accuracy: 0.8936 - val_recall_20: 0.0000e+00 - val_precision_20: 0.0000e+00
Epoch 4/20
43/43 [===

In [445]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [446]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_traditional Accuracy:  0.7301218804410912 MSE:  0.2698781195589089 UAR:  0.6349745026215614 Recall:  0.5294117647058824 Precision:  0.09574468085106383 F1:  0.16216216216216217
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_traditional Accuracy:  0.824143934997098 MSE:  0.1758560650029019 UAR:  0.5673058967176614 Recall:  0.2823529411764706 Precision:  0.09022556390977443 F1:  0.13675213675213674
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_traditional Accuracy:  0.8810214741729542 MSE:  0.11897852582704585 UAR:  0.5693349134525605 Recall:  0.2235294117647059 Precision:  0.12025316455696203 F1:  0.15637860082304528
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_traditional Accuracy:  0.9123621590249564 MSE:  0.08763784097504353 UAR:  0.5467787114845938 Recall:  0.1411764705882353 Precision:  0.13333333333333333 F1:  0.13714285714285712
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_a

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [447]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [448]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [449]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_traditional_best Accuracy:  0.6477074869413814 MSE:  0.3522925130586187 UAR:  0.6418228829993535 Recall:  0.6352941176470588 Precision:  0.08571428571428572 F1:  0.15104895104895105
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_traditional_best Accuracy:  0.8461984910040626 MSE:  0.1538015089959373 UAR:  0.6625619478560655 Recall:  0.4588235294117647 Precision:  0.1511627906976744 F1:  0.22740524781341107
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_traditional_best Accuracy:  0.9239698200812536 MSE:  0.07603017991874637 UAR:  0.619808949220714 Recall:  0.2823529411764706 Precision:  0.2553191489361702 F1:  0.2681564245810055
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_traditional_best Accuracy:  0.9500870574579222 MSE:  0.04991294254207777 UAR:  0.5833512892336422 Recall:  0.17647058823529413 Precision:  0.4838709677419355 F1:  0.25862068965517243
Metric_name:  mobilenet_7.h5_DAiSE

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/pyth

##### Self attention

In [450]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_traditional")
print(metric_name)

mobilenet_7.h5_DAiSEE_2_STAT_self_attention_traditional


In [451]:
CONCATENATE_STAT = True

In [452]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
X_subsample_val, y_subsample_val = get_samples(filename2features_val)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5482, 128, 3072) (5482,)
(1723, 128, 3072) (1723,)
(1720, 128, 3072) (1720,)


In [453]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_33"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3072)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 3072)           9437184   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 3072)           9437184   ['image_set[0][0]']           
                                                                                                  
 attention_16 (Attention)    (None, None, 3072)           0         ['query[0][0]',               
                                                                     'value[0][0]']        

In [454]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_val,y_subsample_val))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 8s 164ms/step - loss: 0.6305 - acc: 0.9327 - auc: 0.5362 - binary_accuracy: 0.9327 - recall_21: 0.0323 - precision_21: 0.0584 - val_loss: 0.5288 - val_acc: 0.8936 - val_auc: 0.5954 - val_binary_accuracy: 0.8936 - val_recall_21: 0.0000e+00 - val_precision_21: 0.0000e+00
Epoch 2/20
43/43 [==============================] - 5s 122ms/step - loss: 0.1687 - acc: 0.9537 - auc: 0.7200 - binary_accuracy: 0.9537 - recall_21: 0.0363 - precision_21: 0.3750 - val_loss: 0.3306 - val_acc: 0.8913 - val_auc: 0.6568 - val_binary_accuracy: 0.8913 - val_recall_21: 0.0055 - val_precision_21: 0.1667
Epoch 3/20
43/43 [==============================] - 5s 120ms/step - loss: 0.1577 - acc: 0.9548 - auc: 0.7672 - binary_accuracy: 0.9548 - recall_21: 0.0524 - precision_21: 0.5000 - val_loss: 0.3384 - val_acc: 0.8808 - val_auc: 0.6695 - val_binary_accuracy: 0.8808 - val_recall_21: 0.0765 - val_precision_21: 0.2800
Epoch 4/20
43/43 [==============================] 

In [455]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [456]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_self_attention_traditional Accuracy:  0.6297156123041208 MSE:  0.3702843876958793 UAR:  0.5710119945414063 Recall:  0.5058823529411764 Precision:  0.06729264475743349 F1:  0.11878453038674033
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_self_attention_traditional Accuracy:  0.7678467788740568 MSE:  0.2321532211259431 UAR:  0.5600050276520865 Recall:  0.32941176470588235 Precision:  0.07547169811320754 F1:  0.12280701754385966
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_self_attention_traditional Accuracy:  0.832269297736506 MSE:  0.1677307022634939 UAR:  0.5436938878115349 Recall:  0.2235294117647059 Precision:  0.07851239669421488 F1:  0.11620795107033641
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_self_attention_traditional Accuracy:  0.8618688334300638 MSE:  0.13813116656993615 UAR:  0.5202219349278173 Recall:  0.1411764705882353 Precision:  0.06779661016949153 F1:  0.0916030534351145
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_self_attention_tr

In [457]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [458]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [459]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_self_attention_traditional_best Accuracy:  0.7214161346488682 MSE:  0.2785838653511317 UAR:  0.6638583638583638 Recall:  0.6 Precision:  0.10261569416498995 F1:  0.1752577319587629
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_self_attention_traditional_best Accuracy:  0.8984329657573998 MSE:  0.10156703424260012 UAR:  0.589646627881922 Recall:  0.24705882352941178 Precision:  0.1590909090909091 F1:  0.19354838709677422
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_self_attention_traditional_best Accuracy:  0.9442832269297736 MSE:  0.05571677307022635 UAR:  0.5691445809092868 Recall:  0.15294117647058825 Precision:  0.35135135135135137 F1:  0.21311475409836064
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_self_attention_traditional_best Accuracy:  0.950667440510737 MSE:  0.04933255948926291 UAR:  0.5557710263592617 Recall:  0.11764705882352941 Precision:  0.5 F1:  0.19047619047619047
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_self_attention_traditional

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Balanced split

In [460]:
train_pickle = FEATURES_DIR + 'train_features_engagement_{}.pickle'.format(base_model_key)
test_pickle = FEATURES_DIR + 'test_features_engagement_{}.pickle'.format(base_model_key)
filename2features_train = load_features(train_pickle)
filename2features_test = load_features(test_pickle)

In [461]:
print("Train: ", len(filename2features_train))
print("Test: ", len(filename2features_test))

Train:  5482
Test:  1723


In [462]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 5234/5482: 95.4761036118205%
1 248/5482: 4.523896388179496%
val:
0 1638/1723: 95.06674405107371%
1 85/1723: 4.933255948926291%


In [463]:
filename2features_train, filename2features_test = prepare_new_dataset(filename2features_train, filename2features_test, 0.76)

(5482, 2048) (5482,)
(1723, 2048) (1723,)
(7205, 2048) (7205,)
[0 1] [6872  333]
7205
5475
1730


In [464]:
print("train:")
print_classes(get_labels(filename2features_train))
print("val:")
print_classes(get_labels(filename2features_test))

train:
0 5222/5475: 95.37899543378995%
1 253/5475: 4.621004566210046%
val:
0 1650/1730: 95.3757225433526%
1 80/1730: 4.624277456647399%


##### Single attention

In [465]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "single_attention_balanced")
print(metric_name)

mobilenet_7.h5_DAiSEE_2_STAT_single_attention_balanced


In [466]:
CONCATENATE_STAT = True

In [467]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5475, 128, 3072) (5475,)
(1730, 128, 3072) (1730,)


In [468]:
modelAtn, save_best_model = get_single_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_34"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3072)]         0         []                            
                                                                                                  
 e (Dense)                   (None, None, 1)              3073      ['image_set[0][0]']           
                                                                                                  
 alignment (Reshape)         (None, None)                 0         ['e[0][0]']                   
                                                                                                  
 alpha (Activation)          (None, None)                 0         ['alignment[0][0]']           
                                                                                           

In [469]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 5s 87ms/step - loss: 0.2170 - acc: 0.9381 - auc: 0.5603 - binary_accuracy: 0.9381 - recall_22: 0.0158 - precision_22: 0.0426 - val_loss: 0.1779 - val_acc: 0.9538 - val_auc: 0.7091 - val_binary_accuracy: 0.9538 - val_recall_22: 0.0000e+00 - val_precision_22: 0.0000e+00
Epoch 2/20
43/43 [==============================] - 2s 37ms/step - loss: 0.1684 - acc: 0.9540 - auc: 0.7284 - binary_accuracy: 0.9540 - recall_22: 0.0040 - precision_22: 1.0000 - val_loss: 0.1660 - val_acc: 0.9538 - val_auc: 0.7475 - val_binary_accuracy: 0.9538 - val_recall_22: 0.0000e+00 - val_precision_22: 0.0000e+00
Epoch 3/20
43/43 [==============================] - 2s 43ms/step - loss: 0.1601 - acc: 0.9542 - auc: 0.7749 - binary_accuracy: 0.9542 - recall_22: 0.0237 - precision_22: 0.6000 - val_loss: 0.1612 - val_acc: 0.9538 - val_auc: 0.7620 - val_binary_accuracy: 0.9538 - val_recall_22: 0.0000e+00 - val_precision_22: 0.0000e+00
Epoch 4/20
43/43 [===================

In [470]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [471]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_balanced Accuracy:  0.9439306358381503 MSE:  0.05606936416184971 UAR:  0.6197348484848485 Recall:  0.2625 Precision:  0.3559322033898305 F1:  0.302158273381295
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_balanced Accuracy:  0.9497109826589596 MSE:  0.050289017341040465 UAR:  0.575189393939394 Recall:  0.1625 Precision:  0.3939393939393939 F1:  0.23008849557522124
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_balanced Accuracy:  0.9497109826589596 MSE:  0.050289017341040465 UAR:  0.5335606060606061 Recall:  0.075 Precision:  0.3157894736842105 F1:  0.1212121212121212
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_balanced Accuracy:  0.9520231213872833 MSE:  0.04797687861271676 UAR:  0.5169318181818182 Recall:  0.0375 Precision:  0.3333333333333333 F1:  0.06741573033707865
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_balanced Accuracy:  0.9531791907514451 MSE:  0.04682080

In [472]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [473]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [474]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_balanced_best Accuracy:  0.8953757225433526 MSE:  0.1046242774566474 UAR:  0.7132196969696969 Recall:  0.5125 Precision:  0.22404371584699453 F1:  0.31178707224334595
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_balanced_best Accuracy:  0.9450867052023122 MSE:  0.05491329479768786 UAR:  0.6381818181818182 Recall:  0.3 Precision:  0.38095238095238093 F1:  0.3356643356643356
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_balanced_best Accuracy:  0.9514450867052023 MSE:  0.048554913294797684 UAR:  0.5879924242424243 Recall:  0.1875 Precision:  0.4411764705882353 F1:  0.2631578947368421
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_balanced_best Accuracy:  0.953757225433526 MSE:  0.046242774566473986 UAR:  0.5535227272727272 Recall:  0.1125 Precision:  0.5 F1:  0.18367346938775508
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_single_attention_balanced_best Accuracy:  0.953757225433526 MSE:  0

/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/echuraev/anaconda3/envs/sciense/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### Self attention

In [475]:
metric_name = get_metric_name(base_model_key, DATASET_NAME, N_CLASSES, stat_name, "self_attention_balanced")
print(metric_name)

mobilenet_7.h5_DAiSEE_2_STAT_self_attention_balanced


In [476]:
CONCATENATE_STAT = True

In [477]:
X_subsample_train, y_subsample_train = get_samples(filename2features_train)
X_subsample_test, y_subsample_test = get_samples(filename2features_test)
FEATURE_VECTOR_DIM=X_subsample_train.shape[-1]

(5475, 128, 3072) (5475,)
(1730, 128, 3072) (1730,)


In [478]:
modelAtn, save_best_model = get_self_attention(FEATURE_VECTOR_DIM, N_CLASSES)

Model: "model_35"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_set (InputLayer)      [(None, None, 3072)]         0         []                            
                                                                                                  
 query (Dense)               (None, None, 3072)           9437184   ['image_set[0][0]']           
                                                                                                  
 value (Dense)               (None, None, 3072)           9437184   ['image_set[0][0]']           
                                                                                                  
 attention_17 (Attention)    (None, None, 3072)           0         ['query[0][0]',               
                                                                     'value[0][0]']        

In [479]:
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    if USE_GENERATORS:
        modelAtn.fit(train_generator,steps_per_epoch=num_samples_train//BATCH_SIZE,
                         epochs=20, verbose=1, callbacks=[save_best_model],
                        validation_data=test_generator,validation_steps=num_samples_test//BATCH_SIZE, class_weight=class_weights)
    else:
        modelAtn.fit(X_subsample_train,y_subsample_train, batch_size=BATCH_SIZE, epochs=20, verbose=1, callbacks=[save_best_model],
                 validation_data=(X_subsample_test,y_subsample_test))#, class_weight=class_weights)
    best_model_weights = save_best_model.best_model_weights
    print(save_best_model.best)
    save_weights(modelAtn, weights_name)

Epoch 1/20
43/43 [==============================] - 8s 163ms/step - loss: 0.5890 - acc: 0.9383 - auc: 0.5378 - binary_accuracy: 0.9383 - recall_23: 0.0435 - precision_23: 0.1028 - val_loss: 0.1908 - val_acc: 0.9445 - val_auc: 0.7407 - val_binary_accuracy: 0.9445 - val_recall_23: 0.1000 - val_precision_23: 0.2500
Epoch 2/20
43/43 [==============================] - 5s 120ms/step - loss: 0.1665 - acc: 0.9529 - auc: 0.7388 - binary_accuracy: 0.9529 - recall_23: 0.0474 - precision_23: 0.4138 - val_loss: 0.1684 - val_acc: 0.9538 - val_auc: 0.7480 - val_binary_accuracy: 0.9538 - val_recall_23: 0.0625 - val_precision_23: 0.5000
Epoch 3/20
43/43 [==============================] - 5s 120ms/step - loss: 0.1583 - acc: 0.9549 - auc: 0.7844 - binary_accuracy: 0.9549 - recall_23: 0.0751 - precision_23: 0.5938 - val_loss: 0.1819 - val_acc: 0.9532 - val_auc: 0.7437 - val_binary_accuracy: 0.9532 - val_recall_23: 0.0000e+00 - val_precision_23: 0.0000e+00
Epoch 4/20
43/43 [==============================] 

In [480]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [481]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_self_attention_balanced Accuracy:  0.9364161849710982 MSE:  0.06358381502890173 UAR:  0.6098484848484849 Recall:  0.25 Precision:  0.2857142857142857 F1:  0.26666666666666666
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_self_attention_balanced Accuracy:  0.9502890173410404 MSE:  0.04971098265895954 UAR:  0.5517045454545455 Recall:  0.1125 Precision:  0.375 F1:  0.1730769230769231
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_self_attention_balanced Accuracy:  0.9520231213872833 MSE:  0.04797687861271676 UAR:  0.5347727272727273 Recall:  0.075 Precision:  0.4 F1:  0.1263157894736842
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_self_attention_balanced Accuracy:  0.953757225433526 MSE:  0.046242774566473986 UAR:  0.5237878787878788 Recall:  0.05 Precision:  0.5 F1:  0.09090909090909091
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_self_attention_balanced Accuracy:  0.953757225433526 MSE:  0.046242774566473986 UAR:  0.5059469696969697 Recall:  0.0125 Precis

In [482]:
modelAtn.set_weights(best_model_weights)
metric_name += "_best"
weights_name = get_weights_path(metric_name)
if os.path.isfile(weights_name):
    load_weights(modelAtn, weights_name)
else:
    save_weights(modelAtn, weights_name)

In [483]:
pred, test_labels = get_prediction(modelAtn, filename2features_test, False)

/media/deelvin_disk/echuraev/tmp_dir/ipykernel_1535568/227606356.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred.append(float(pred[0]))


In [484]:
max_uar = 0
max_step = 0
for i in range(1, 10):
    step = i/10
    p = prediction2bin(pred, step)
    t = TABLE_NAME.split('.')
    tn = '{}_{}.{}'.format(t[0], str(step), t[1])
    _, _, uar, _, _, _ = print_results(p, test_labels, table_name=tn, metric_name=metric_name)
    if uar > max_uar:
        max_uar = uar
        max_step = step

print(max_step, max_uar)

Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_self_attention_balanced_best Accuracy:  0.8994219653179191 MSE:  0.10057803468208093 UAR:  0.6856060606060607 Recall:  0.45 Precision:  0.21686746987951808 F1:  0.2926829268292683
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_self_attention_balanced_best Accuracy:  0.9427745664739884 MSE:  0.05722543352601156 UAR:  0.6250757575757575 Recall:  0.275 Precision:  0.3492063492063492 F1:  0.3076923076923077
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_self_attention_balanced_best Accuracy:  0.9473988439306359 MSE:  0.05260115606936416 UAR:  0.568030303030303 Recall:  0.15 Precision:  0.34285714285714286 F1:  0.20869565217391303
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_self_attention_balanced_best Accuracy:  0.953757225433526 MSE:  0.046242774566473986 UAR:  0.5654166666666667 Recall:  0.1375 Precision:  0.5 F1:  0.21568627450980396
Metric_name:  mobilenet_7.h5_DAiSEE_2_STAT_self_attention_balanced_best Accuracy:  0.953757225433526 MSE:  0.046242774566